The next script can be used to reproduce the results of the section “Illustration: Application to the PBC dataset”. The code is segmented into five sections. Section 1 entails the preliminaries, which include all the custom functions created to compute the log-likelihood functions, residuals, ROC curves, predictiveness curves, and Standardized Total Gain. The second section consists of the preprocessing step and marker definition using the data set PCB which is available in the survival package of the R program. Section 3 introduces the code needed for the computation of the residuals and the corresponding plots, which are employed as a guide for choosing a copula from a set of candidates. Section 4 presents the code for the analysis of both markers: model fitting, ROC curves, AUC, predictiveness, and Standardized Total Gain. Finally, the last section brings in all together to create the graphics that reproduce figures 2 and 3 of the paper.

# Section 1: Preeliminaries

## Rpy2 package

In [ ]:
%load_ext rpy2.ipython

## Importing R packages

In [ ]:
%%R
library(copula)
library(survival)
library(sn)
library(VGAM)
library(flux)
library(lattice)
library(gridExtra)

## Auxiliary custom functions

### a) Log-likelihood functions for Frank copula 

In [ ]:
%%R

#Fully-parametric (fp):
ll.fp.frank <- function(parametros, obs.marker, surv.times, cens.status){
   theta.par   <- parametros[1]
   xi.par    <- parametros[2]
   omega.par <- exp(parametros[3])
   alpha.par <- parametros[4]
   shape.par <- exp(parametros[5])
   scale.par <- exp(parametros[6])
   frank.cop <- frankCopula(theta.par)
   f.M.sn <- dsn(obs.marker, xi = xi.par, omega = omega.par, alpha=alpha.par)
   f.T.wei <- dweibull(surv.times, shape = shape.par, scale = scale.par)
   F.M.sn <- psn(obs.marker, xi = xi.par, omega = omega.par, alpha=alpha.par)
   F.T.wei <- pweibull(surv.times, shape = shape.par, scale = scale.par)
   f.joint.MT <- f.M.sn*f.T.wei*dCopula(cbind(F.M.sn, 1-F.T.wei), frank.cop)
   psi <- function(x, theta){-1*log((exp(-theta*x)-1)/(exp(-theta)-1))}
   psi.inv <- function(x, theta){-log(1-(1-exp(-theta))*exp(-x))/theta}
   psi.prima <- function(x, theta){
      (theta*exp(-theta*x))/(exp(-theta*x)-1)}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   -1*sum(cens.status*log(f.joint.MT))-
      sum((1-cens.status)*
             log(f.M.sn*C.prima(F.M.sn, 1-F.T.wei, theta.par)))
}


#Semi-parametric (sp):
ll.sp.frank <- function(parametro, obs.marker, surv.times, 
                        xi.M, omega.M, alpha.M, 
                        times.np, F.T.np, cens.status){
   psi <- function(x, theta){-1*log((exp(-theta*x)-1)/(exp(-theta)-1))}
   psi.inv <- function(x, theta){-log(1-(1-exp(-theta))*exp(-x))/theta}
   psi.prima <- function(x, theta){
      (theta*exp(-theta*x))/(exp(-theta*x)-1)}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   theta.par <- parametro
   c.M <- obs.marker
   n.c <- length(c.M)
   times.k <- times.np
   F.T.k <- F.T.np
   n.times <- length(times.k)
   f.joint.all <- matrix(1:(n.c*n.times), ncol=n.times)
   F.t.minus <- c(0, F.T.k[-n.times])
   F.t.current <- F.T.k
   F.M.sum.T <- 1:n.c
   f.MT <- 1:n.c
   match.times <- match(surv.times, times.np)
   f.M <- dsn(c.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   F.M <- psn(c.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   for(i in 1:n.c){
      f.M.i <- f.M[i]
      F.M.i <- F.M[i]
      f.joint.all[i,] <- f.M.i*(C.prima(F.M.i, 1-F.t.minus, theta.par) -
                                   C.prima(F.M.i, 1-F.t.current, theta.par))
      f.MT[i] <- f.joint.all[i,match.times[i]]
      F.M.sum.T[i] <- sum(f.joint.all[i,][1:match.times[i]])
   }
   S.m.T <- f.M-F.M.sum.T
   -1*sum(log(f.MT[cens.status==1]))-sum(log(S.m.T[cens.status==0]))
}

### b) Log-likelihood functions for Gumbel copula

In [ ]:
%%R 

#Fully-parametric (fp):
ll.fp.gumbel <- function(parametros, obs.marker, surv.times, cens.status)
{
   theta.par   <- exp(parametros[1])+1
   xi.par    <- parametros[2]
   omega.par <- exp(parametros[3])
   alpha.par <- parametros[4]
   shape.par <- exp(parametros[5])
   scale.par <- exp(parametros[6])
   gumbel.cop <- gumbelCopula(theta.par)
   f.M.sn <- dsn(obs.marker, xi = xi.par, omega = omega.par, alpha=alpha.par)
   f.T.wei <- dweibull(surv.times, shape = shape.par, scale = scale.par)
   F.M.sn <- psn(obs.marker, xi = xi.par, omega = omega.par, alpha=alpha.par)
   F.T.wei <- pweibull(surv.times, shape = shape.par, scale = scale.par)
   f.joint.MT <- f.M.sn*f.T.wei*dCopula(cbind(F.M.sn, 1-F.T.wei), gumbel.cop)
   psi <- function(x, theta){(-log(x))^(theta)}
   psi.inv <- function(x, theta){exp(-x^(1/theta))}
   psi.prima <- function(x, theta){-(theta/x)*(-log(x))^(theta-1)}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   -1*sum(cens.status*log(f.joint.MT))-
      sum((1-cens.status)*
             log(f.M.sn*C.prima(F.M.sn, 1-F.T.wei, theta.par)))
}

#Semi-parametric (sp):
ll.sp.gumbel <- function(parametro, obs.marker, surv.times, 
                         xi.M, omega.M, alpha.M, 
                         times.np, F.T.np, cens.status){
   psi <- function(x, theta){(-log(x))^(theta)}
   psi.inv <- function(x, theta){exp(-x^(1/theta))}
   psi.prima <- function(x, theta){-(theta/x)*(-log(x))^(theta-1)}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   theta.par   <- exp(parametro)+1
   c.M <- obs.marker
   n.c <- length(c.M)
   times.k <- times.np
   F.T.k <- F.T.np
   n.times <- length(times.k)
   f.joint.all <- matrix(1:(n.c*n.times), ncol=n.times)
   F.t.minus <- c(0, F.T.k[-n.times])
   F.t.current <- F.T.k
   F.M.sum.T <- 1:n.c
   f.MT <- 1:n.c
   match.times <- match(surv.times, times.np)
   f.M <- dsn(c.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   F.M <- psn(c.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   for(i in 1:n.c){
      f.M.i <- f.M[i]
      F.M.i <- F.M[i]
      f.joint.all[i,] <- f.M.i*(C.prima(F.M.i, 1-F.t.minus, theta.par) -
                                   C.prima(F.M.i, 1-F.t.current, theta.par))
      f.MT[i] <- f.joint.all[i,match.times[i]]
      F.M.sum.T[i] <- sum(f.joint.all[i,][1:match.times[i]])
   }
   S.m.T <- f.M-F.M.sum.T
   -1*sum(log(f.MT[cens.status==1]))-sum(log(S.m.T[cens.status==0]))
}



### c) Log-likelihood functions for Gaussian copula

In [ ]:
%%R

#Fully-parametric (fp):
ll.fp.gauss <- function(parametros, obs.marker, surv.times, cens.status){
   rho.par   <- tanh(parametros[1])
   xi.par    <- parametros[2]
   omega.par <- exp(parametros[3])
   alpha.par <- parametros[4]
   shape.par <- exp(parametros[5])
   scale.par <- exp(parametros[6])
   norm.cop <- normalCopula(rho.par)
   f.M.sn <- dsn(obs.marker, xi = xi.par, omega = omega.par, alpha=alpha.par)
   f.T.wei <- dweibull(surv.times, shape = shape.par, scale = scale.par)
   F.M.sn <- psn(obs.marker, xi = xi.par, omega = omega.par, alpha=alpha.par)
   F.T.wei <- pweibull(surv.times, shape = shape.par, scale = scale.par)
   f.joint.MT <- f.M.sn*f.T.wei*dCopula(cbind(F.M.sn, 1-F.T.wei), norm.cop)
   -1*sum(cens.status*log(f.joint.MT))-
      sum((1-cens.status)*
             log(f.M.sn*(pnorm((qnorm(1-F.T.wei)-rho.par*qnorm(F.M.sn))/sqrt(1-rho.par^2)))))
}

#Semi-parametric (sp):
ll.sp.gauss <- function(parametro, obs.marker, surv.times, 
                        xi.M, omega.M, alpha.M, 
                        times.np, F.T.np, cens.status){
   rho.C <- tanh(parametro)
   c.M <- obs.marker
   n.c <- length(c.M)
   times.k <- times.np
   F.T.k <- F.T.np
   n.times <- length(times.k)
   f.joint.all <- matrix(1:(n.c*n.times), ncol=n.times)
   F.t.minus <- c(0, F.T.k[-n.times])
   F.t.current <- F.T.k
   F.M.sum.T <- 1:n.c
   f.MT <- 1:n.c
   match.times <- match(surv.times, times.np)
   f.M <- dsn(c.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   F.M <- psn(c.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   for(i in 1:n.c){
      f.M.i <- f.M[i]
      F.M.i <- F.M[i]
      f.joint.all[i,] <- f.M.i*(
         pnorm((qnorm(1-F.t.minus)-rho.C*qnorm(F.M.i))/sqrt(1-rho.C^2))-
            pnorm((qnorm(1-F.t.current)-rho.C*qnorm(F.M.i))/sqrt(1-rho.C^2)))
      f.MT[i] <- f.joint.all[i,match.times[i]]
      F.M.sum.T[i] <- sum(f.joint.all[i,][1:match.times[i]])
   }
   S.m.T <- f.M-F.M.sum.T
   -1*sum(log(f.MT[cens.status==1]))-sum(log(S.m.T[cens.status==0]))
}

### d) Log-likehood functions for Clayton copula

In [ ]:
%%R 

#Fully-parametric (fp):
ll.fp.clayton <- function(parametros, obs.marker, surv.times, cens.status){
   theta.par   <- exp(parametros[1])
   xi.par    <- parametros[2]
   omega.par <- exp(parametros[3])
   alpha.par <- parametros[4]
   shape.par <- exp(parametros[5])
   scale.par <- exp(parametros[6])
   clayton.cop <- claytonCopula(theta.par)
   f.M.sn <- dsn(obs.marker, xi = xi.par, omega = omega.par, alpha=alpha.par)
   f.T.wei <- dweibull(surv.times, shape = shape.par, scale = scale.par)
   F.M.sn <- psn(obs.marker, xi = xi.par, omega = omega.par, alpha=alpha.par)
   F.T.wei <- pweibull(surv.times, shape = shape.par, scale = scale.par)
   f.joint.MT <- f.M.sn*f.T.wei*dCopula(cbind(F.M.sn, 1-F.T.wei), clayton.cop)
   psi <- function(x, theta){x^(-theta)-1}
   psi.inv <- function(x, theta){(1+x)^(-1/theta)}
   psi.prima <- function(x, theta){-1*theta*x^(-1*(theta+1))}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   -1*sum(cens.status*log(f.joint.MT))-
      sum((1-cens.status)*
             log(f.M.sn*C.prima(F.M.sn, 1-F.T.wei, theta.par)))
}

#Semi-parametric (sp):
ll.sp.clayton <- function(parametro, obs.marker, surv.times, 
                          xi.M, omega.M, alpha.M, 
                          times.np, F.T.np, cens.status){
   psi <- function(x, theta){x^(-theta)-1}
   psi.inv <- function(x, theta){(1+x)^(-1/theta)}
   psi.prima <- function(x, theta){-1*theta*x^(-1*(theta+1))}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   theta.par   <- exp(parametro)
   c.M <- obs.marker
   n.c <- length(c.M)
   times.k <- times.np
   F.T.k <- F.T.np
   n.times <- length(times.k)
   f.joint.all <- matrix(1:(n.c*n.times), ncol=n.times)
   F.t.minus <- c(0, F.T.k[-n.times])
   F.t.current <- F.T.k
   F.M.sum.T <- 1:n.c
   f.MT <- 1:n.c
   match.times <- match(surv.times, times.np)
   f.M <- dsn(c.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   F.M <- psn(c.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   for(i in 1:n.c){
      f.M.i <- f.M[i]
      F.M.i <- F.M[i]
      f.joint.all[i,] <- f.M.i*(C.prima(F.M.i, 1-F.t.minus, theta.par) -
                                   C.prima(F.M.i, 1-F.t.current, theta.par))
      f.MT[i] <- f.joint.all[i,match.times[i]]
      F.M.sum.T[i] <- sum(f.joint.all[i,][1:match.times[i]])
   }
   S.m.T <- f.M-F.M.sum.T
   -1*sum(log(f.MT[cens.status==1]))-sum(log(S.m.T[cens.status==0]))
}


### e) Log-likehood function for Gumbel copula with theta=2 (mayo marker)

In [ ]:
%%R

ll.fp.tau0.5.gumbel <- function(parametros, obs.marker, surv.times, cens.status){
   theta.par   <- exp(0)+1
   xi.par    <- parametros[1]
   omega.par <- exp(parametros[2])
   alpha.par <- parametros[3]
   shape.par <- exp(parametros[4])
   scale.par <- exp(parametros[5])
   gumbel.cop <- gumbelCopula(theta.par)
   f.M.sn <- dsn(obs.marker, xi = xi.par, omega = omega.par, alpha=alpha.par)
   f.T.wei <- dweibull(surv.times, shape = shape.par, scale = scale.par)
   F.M.sn <- psn(obs.marker, xi = xi.par, omega = omega.par, alpha=alpha.par)
   F.T.wei <- pweibull(surv.times, shape = shape.par, scale = scale.par)
   f.joint.MT <- f.M.sn*f.T.wei*dCopula(cbind(F.M.sn, 1-F.T.wei), gumbel.cop)
   psi <- function(x, theta){(-log(x))^(theta)}
   psi.inv <- function(x, theta){exp(-x^(1/theta))}
   psi.prima <- function(x, theta){-(theta/x)*(-log(x))^(theta-1)}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   -1*sum(cens.status*log(f.joint.MT))-
      sum((1-cens.status)*
             log(f.M.sn*C.prima(F.M.sn, 1-F.T.wei, theta.par)))
}



### f) Log-likehood function for Frank copula with exponential survival distribution (4cov marker)

In [ ]:
%%R

ll.fp.frank.exp <-function(parametros, obs.marker, surv.times, cens.status){
   theta.par   <- parametros[1]
   xi.par    <- parametros[2]
   omega.par <- exp(parametros[3])
   alpha.par <- parametros[4]
   shape.par <- exp(0)
   scale.par <- exp(parametros[5])
   frank.cop <- frankCopula(theta.par)
   f.M.sn <- dsn(obs.marker, xi = xi.par, omega = omega.par, alpha=alpha.par)
   f.T.wei <- dweibull(surv.times, shape = shape.par, scale = scale.par)
   F.M.sn <- psn(obs.marker, xi = xi.par, omega = omega.par, alpha=alpha.par)
   F.T.wei <- pweibull(surv.times, shape = shape.par, scale = scale.par)
   f.joint.MT <- f.M.sn*f.T.wei*dCopula(cbind(F.M.sn, 1-F.T.wei), frank.cop)
   psi <- function(x, theta){-1*log((exp(-theta*x)-1)/(exp(-theta)-1))}
   psi.inv <- function(x, theta){-log(1-(1-exp(-theta))*exp(-x))/theta}
   psi.prima <- function(x, theta){
      (theta*exp(-theta*x))/(exp(-theta*x)-1)}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   -1*sum(cens.status*log(f.joint.MT))-
      sum((1-cens.status)*
             log(f.M.sn*C.prima(F.M.sn, 1-F.T.wei, theta.par)))
}


### g) Functions to compute residuals for Frank copula 

In [ ]:
%%R

#Residuals for T|M fully-parametric (fp):
residuals.T.frank.fp <- function(parametros, obs.marker, surv.times, cens.status){
   theta.par   <- parametros[1]
   xi.par    <- parametros[2]
   omega.par <- exp(parametros[3])
   alpha.par <- parametros[4]
   shape.par <- exp(parametros[5])
   scale.par <- exp(parametros[6])
   frank.cop <- frankCopula(theta.par)
   f.M.sn <- dsn(obs.marker, xi = xi.par, omega = omega.par, alpha=alpha.par)
   f.T.wei <- dweibull(surv.times, shape = shape.par, scale = scale.par)
   F.M.sn <- psn(obs.marker, xi = xi.par, omega = omega.par, alpha=alpha.par)
   F.T.wei <- pweibull(surv.times, shape = shape.par, scale = scale.par)
   f.joint.MT <- f.M.sn*f.T.wei*dCopula(cbind(F.M.sn, 1-F.T.wei), frank.cop)
   psi <- function(x, theta){-1*log((exp(-theta*x)-1)/(exp(-theta)-1))}
   psi.inv <- function(x, theta){-log(1-(1-exp(-theta))*exp(-x))/theta}
   psi.prima <- function(x, theta){
      (theta*exp(-theta*x))/(exp(-theta*x)-1)}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   S.m.T <- C.prima(F.M.sn, 1-F.T.wei, theta.par)
   Lambda.m.T <- -1*log(S.m.T)
   Lambda.set <- data.frame(Lambda.m.T, cens.status)
   KM.Lambda <- survfit(Surv(Lambda.m.T, cens.status) ~ 1, data = Lambda.set)
   plot(log(KM.Lambda$time), log(-1*log(KM.Lambda$surv)), xlim=c(-6,2), ylim=c(-6,2))
   abline(0,1)
   cbind(log(KM.Lambda$time), log(-1*log(KM.Lambda$surv)))
}


#Residuals for M|T fully-parametric (fp):
residuals.M.frank.fp <- function(parametros, obs.marker, surv.times, cens.status){
   theta.par   <- parametros[1]
   xi.par    <- parametros[2]
   omega.par <- exp(parametros[3])
   alpha.par <- parametros[4]
   shape.par <- exp(parametros[5])
   scale.par <- exp(parametros[6])
   frank.cop <- frankCopula(theta.par)
   f.M.sn <- dsn(obs.marker, xi = xi.par, omega = omega.par, alpha=alpha.par)
   f.T.wei <- dweibull(surv.times, shape = shape.par, scale = scale.par)
   F.M.sn <- psn(obs.marker, xi = xi.par, omega = omega.par, alpha=alpha.par)
   F.T.wei <- pweibull(surv.times, shape = shape.par, scale = scale.par)
   f.joint.MT <- f.M.sn*f.T.wei*dCopula(cbind(F.M.sn, 1-F.T.wei), frank.cop)
   psi <- function(x, theta){-1*log((exp(-theta*x)-1)/(exp(-theta)-1))}
   psi.inv <- function(x, theta){-log(1-(1-exp(-theta))*exp(-x))/theta}
   psi.prima <- function(x, theta){
      (theta*exp(-theta*x))/(exp(-theta*x)-1)}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   F.t.obs.M <- C.prima(1-F.T.wei, F.M.sn, theta.par)
   F.t.cens.M <- pCopula(cbind(F.M.sn, 1-F.T.wei), frank.cop)/(1-F.T.wei)
   qqnorm(qnorm(c(F.t.obs.M[cens.status==1],F.t.cens.M[cens.status==0])))
   abline(0,1)
   qnorm(c(F.t.obs.M[cens.status==1],F.t.cens.M[cens.status==0]))
}


#Residuals for T|M semi-parametric (sp):
residuals.T.frank.sp <- function(parametro, obs.marker, surv.times, 
                                 xi.M, omega.M, alpha.M, 
                                 times.np, F.T.np, cens.status){
   psi <- function(x, theta){-1*log((exp(-theta*x)-1)/(exp(-theta)-1))}
   psi.inv <- function(x, theta){-log(1-(1-exp(-theta))*exp(-x))/theta}
   psi.prima <- function(x, theta){
      (theta*exp(-theta*x))/(exp(-theta*x)-1)}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   theta.par <- parametro
   c.M <- obs.marker
   n.c <- length(c.M)
   times.k <- times.np
   F.T.k <- F.T.np
   n.times <- length(times.k)
   f.joint.all <- matrix(1:(n.c*n.times), ncol=n.times)
   F.t.minus <- c(0, F.T.k[-n.times])
   F.t.current <- F.T.k
   F.M.sum.T <- 1:n.c
   f.MT <- 1:n.c
   match.times <- match(surv.times, times.np)
   f.M <- dsn(c.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   F.M <- psn(c.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   for(i in 1:n.c){
      f.M.i <- f.M[i]
      F.M.i <- F.M[i]
      f.joint.all[i,] <- f.M.i*(C.prima(F.M.i, 1-F.t.minus, theta.par) -
                                   C.prima(F.M.i, 1-F.t.current, theta.par))
      f.MT[i] <- f.joint.all[i,match.times[i]]
      F.M.sum.T[i] <- sum(f.joint.all[i,][1:match.times[i]])
   }
   S.m.T <- (f.M-F.M.sum.T)/f.M
   Lambda.m.T <- -1*log(S.m.T)
   Lambda.set <- data.frame(Lambda.m.T, cens.status)
   KM.Lambda <- survfit(Surv(Lambda.m.T, cens.status) ~ 1, data = Lambda.set)
   plot(log(KM.Lambda$time), log(-1*log(KM.Lambda$surv)), xlim=c(-6,2), ylim=c(-6,2))
   abline(0,1)
   cbind(log(KM.Lambda$time), log(-1*log(KM.Lambda$surv)))
}


#Residuals for M|T semi-parametric (sp):
residuals.M.frank.sp <- function(lim.inf, obs.marker, surv.times, cens.status,
                                 theta.par, xi.M, omega.M, alpha.M, 
                                 times.np, F.T.np){
   psi <- function(x, theta){-1*log((exp(-theta*x)-1)/(exp(-theta)-1))}
   psi.inv <- function(x, theta){-log(1-(1-exp(-theta))*exp(-x))/theta}
   psi.prima <- function(x, theta){
      (theta*exp(-theta*x))/(exp(-theta*x)-1)}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   cdf.t <- approxfun(times.np, F.T.np)
   F.T.obs <- cdf.t(surv.times)
   F.M.obs <- psn(obs.marker, xi = xi.M, omega = omega.M, alpha=alpha.M)
   sample.zs <- length(obs.marker)
   # store the joint distribution in the following object:
   F.MT.obs <- 1:sample.zs
   F.M.given.T <- 1:sample.zs
   for(h in 1:sample.zs){
      c.M <- seq(from=lim.inf, to=obs.marker[h], length=25)
      t.T <- surv.times[h]
      times.k <- times.np[times.np<=t.T]
      F.T.k <- F.T.np[times.np<=t.T]
      n.times <- length(F.T.k)
      n.c <- length(c.M)
      f.joint.all <- matrix(1:(n.c*n.times), ncol=n.times)
      F.t.minus <- c(0, F.T.k[-n.times])
      F.t.current <- F.T.k
      F.M.i <- 1:n.c
      for(i in 1:n.c){
         F.M <- psn(c.M[i], xi = xi.M, omega = omega.M, alpha=alpha.M)
         F.M.i[i] <- F.M
         f.M <- dsn(c.M[i], xi = xi.M, omega = omega.M, alpha=alpha.M)
         f.joint.all[i,] <- f.M*(C.prima(F.M, 1-F.t.minus, theta.par) -
                                    C.prima(F.M, 1-F.t.current, theta.par))
      }
      cat(h, "\n")
      if(cens.status[h]==1){
         cat("auc:", auc(c.M, f.joint.all[,n.times]), "\n")
         cat("n.times", n.times, "\n")
         cat("F.T.k[n.times]", F.T.k[n.times], "\n")
         cat("n.times-1", n.times-1, "\n")
         cat("F.T.k[n.times-1]", F.T.k[n.times-1], "\n")
         if(n.times > 1){
            F.M.given.T[h] <- auc(c.M, f.joint.all[,n.times])/(F.T.k[n.times]-
                                                                  F.T.k[n.times-1])
         }
         else{
            F.M.given.T[h] <- auc(c.M, f.joint.all[,n.times])/(F.T.k[n.times])
         }
      }
      # cat(F.T.k[n.times] - F.T.k[n.times-1], "\n")
      F.ct <- 1:(n.c-3)
      for(k in 4:n.c){
         valor <- 0
         for(j in 1:n.times){
            valor <- auc(c.M[1:k], f.joint.all[(1:k),j]) + valor
         }
         F.ct[k-3] <- valor
      }
      F.MT.obs[h] <- rev(F.ct)[1]
   }
   CDF.for.cens <- (F.M.obs-F.MT.obs)/(1-F.T.obs)
   qqnorm(qnorm(c(F.M.given.T[cens.status==1], CDF.for.cens[cens.status==0])))
   abline(0,1)
   qnorm(c(F.M.given.T[cens.status==1], CDF.for.cens[cens.status==0]))
}


### h) Functions to compute residuals for Gumbel copula 

In [ ]:
%%R

#Residuals for T|M fully-parametric (fp):
residuals.T.gumbel.fp <- function(parametros, obs.marker, surv.times, cens.status){
   theta.par   <- exp(parametros[1])+1
   xi.par    <- parametros[2]
   omega.par <- exp(parametros[3])
   alpha.par <- parametros[4]
   shape.par <- exp(parametros[5])
   scale.par <- exp(parametros[6])
   gumbel.cop <- gumbelCopula(theta.par)
   f.M.sn <- dsn(obs.marker, xi = xi.par, omega = omega.par, alpha=alpha.par)
   f.T.wei <- dweibull(surv.times, shape = shape.par, scale = scale.par)
   F.M.sn <- psn(obs.marker, xi = xi.par, omega = omega.par, alpha=alpha.par)
   F.T.wei <- pweibull(surv.times, shape = shape.par, scale = scale.par)
   f.joint.MT <- f.M.sn*f.T.wei*dCopula(cbind(F.M.sn, 1-F.T.wei), gumbel.cop)
   psi <- function(x, theta){(-log(x))^(theta)}
   psi.inv <- function(x, theta){exp(-x^(1/theta))}
   psi.prima <- function(x, theta){-(theta/x)*(-log(x))^(theta-1)}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   S.m.T <- C.prima(F.M.sn, 1-F.T.wei, theta.par)
   Lambda.m.T <- -1*log(S.m.T)
   Lambda.set <- data.frame(Lambda.m.T, cens.status)
   KM.Lambda <- survfit(Surv(Lambda.m.T, cens.status) ~ 1, data = Lambda.set)
   plot(log(KM.Lambda$time), log(-1*log(KM.Lambda$surv)), xlim=c(-6,2), ylim=c(-6,2))
   abline(0,1)
   cbind(log(KM.Lambda$time), log(-1*log(KM.Lambda$surv)))
}

#Residuals for M|T fully-parametric (fp):
residuals.M.gumbel.fp <- function(parametros, obs.marker, surv.times, cens.status){
   theta.par   <- exp(parametros[1])+1
   xi.par    <- parametros[2]
   omega.par <- exp(parametros[3])
   alpha.par <- parametros[4]
   shape.par <- exp(parametros[5])
   scale.par <- exp(parametros[6])
   gumbel.cop <- gumbelCopula(theta.par)
   f.M.sn <- dsn(obs.marker, xi = xi.par, omega = omega.par, alpha=alpha.par)
   f.T.wei <- dweibull(surv.times, shape = shape.par, scale = scale.par)
   F.M.sn <- psn(obs.marker, xi = xi.par, omega = omega.par, alpha=alpha.par)
   F.T.wei <- pweibull(surv.times, shape = shape.par, scale = scale.par)
   f.joint.MT <- f.M.sn*f.T.wei*dCopula(cbind(F.M.sn, 1-F.T.wei), gumbel.cop)
   psi <- function(x, theta){(-log(x))^(theta)}
   psi.inv <- function(x, theta){exp(-x^(1/theta))}
   psi.prima <- function(x, theta){-(theta/x)*(-log(x))^(theta-1)}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   F.t.obs.M <- C.prima(1-F.T.wei, F.M.sn, theta.par)
   F.t.cens.M <- pCopula(cbind(F.M.sn, 1-F.T.wei), gumbel.cop)/(1-F.T.wei)
   qqnorm(qnorm(c(F.t.obs.M[cens.status==1],F.t.cens.M[cens.status==0])))
   abline(0,1)
   qnorm(c(F.t.obs.M[cens.status==1],F.t.cens.M[cens.status==0]))
}

#Residuals for T|M semi-parametric (sp):
residuals.T.gumbel.sp <- function(parametro, obs.marker, surv.times, 
                                  xi.M, omega.M, alpha.M, 
                                  times.np, F.T.np, cens.status){
   psi <- function(x, theta){(-log(x))^(theta)}
   psi.inv <- function(x, theta){exp(-x^(1/theta))}
   psi.prima <- function(x, theta){-(theta/x)*(-log(x))^(theta-1)}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   theta.par   <- exp(parametro)+1
   c.M <- obs.marker
   n.c <- length(c.M)
   times.k <- times.np
   F.T.k <- F.T.np
   n.times <- length(times.k)
   f.joint.all <- matrix(1:(n.c*n.times), ncol=n.times)
   F.t.minus <- c(0, F.T.k[-n.times])
   F.t.current <- F.T.k
   F.M.sum.T <- 1:n.c
   f.MT <- 1:n.c
   match.times <- match(surv.times, times.np)
   f.M <- dsn(c.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   F.M <- psn(c.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   for(i in 1:n.c){
      f.M.i <- f.M[i]
      F.M.i <- F.M[i]
      f.joint.all[i,] <- f.M.i*(C.prima(F.M.i, 1-F.t.minus, theta.par) -
                                   C.prima(F.M.i, 1-F.t.current, theta.par))
      f.MT[i] <- f.joint.all[i,match.times[i]]
      F.M.sum.T[i] <- sum(f.joint.all[i,][1:match.times[i]])
   }
   S.m.T <- (f.M-F.M.sum.T)/f.M
   Lambda.m.T <- -1*log(S.m.T)
   Lambda.set <- data.frame(Lambda.m.T, cens.status)
   KM.Lambda <- survfit(Surv(Lambda.m.T, cens.status) ~ 1, data = Lambda.set)
   plot(log(KM.Lambda$time), log(-1*log(KM.Lambda$surv)), xlim=c(-6,2), ylim=c(-6,2))
   abline(0,1)
   cbind(log(KM.Lambda$time), log(-1*log(KM.Lambda$surv)))
}

#Residuals for M|T semi-parametric (sp):
residuals.M.gumbel.sp <- function(lim.inf, obs.marker, surv.times, cens.status,
                                  parametro, xi.M, omega.M, alpha.M, 
                                  times.np, F.T.np){
   psi <- function(x, theta){(-log(x))^(theta)}
   psi.inv <- function(x, theta){exp(-x^(1/theta))}
   psi.prima <- function(x, theta){-(theta/x)*(-log(x))^(theta-1)}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   theta.par   <- exp(parametro)+1
   cdf.t <- approxfun(times.np, F.T.np)
   F.T.obs <- cdf.t(surv.times)
   F.M.obs <- psn(obs.marker, xi = xi.M, omega = omega.M, alpha=alpha.M)
   sample.zs <- length(obs.marker)
   # store the joint distribution in the following object:
   F.MT.obs <- 1:sample.zs
   F.M.given.T <- 1:sample.zs
   for(h in 1:sample.zs){
      c.M <- seq(from=lim.inf, to=obs.marker[h], length=25)
      t.T <- surv.times[h]
      times.k <- times.np[times.np<=t.T]
      F.T.k <- F.T.np[times.np<=t.T]
      n.times <- length(F.T.k)
      n.c <- length(c.M)
      f.joint.all <- matrix(1:(n.c*n.times), ncol=n.times)
      F.t.minus <- c(0, F.T.k[-n.times])
      F.t.current <- F.T.k
      F.M.i <- 1:n.c
      for(i in 1:n.c){
         F.M <- psn(c.M[i], xi = xi.M, omega = omega.M, alpha=alpha.M)
         F.M.i[i] <- F.M
         f.M <- dsn(c.M[i], xi = xi.M, omega = omega.M, alpha=alpha.M)
         f.joint.all[i,] <- f.M*(C.prima(F.M, 1-F.t.minus, theta.par) -
                                    C.prima(F.M, 1-F.t.current, theta.par))
      }
      cat(h, "\n")
      if(cens.status[h]==1){
         cat("auc:", auc(c.M, f.joint.all[,n.times]), "\n")
         cat("n.times", n.times, "\n")
         cat("F.T.k[n.times]", F.T.k[n.times], "\n")
         cat("n.times-1", n.times-1, "\n")
         cat("F.T.k[n.times-1]", F.T.k[n.times-1], "\n")
         if(n.times > 1){
            F.M.given.T[h] <- auc(c.M, f.joint.all[,n.times])/(F.T.k[n.times]-
                                                                  F.T.k[n.times-1])
         }
         else{
            F.M.given.T[h] <- auc(c.M, f.joint.all[,n.times])/(F.T.k[n.times])
         }
      }
      # cat(F.T.k[n.times] - F.T.k[n.times-1], "\n")
      F.ct <- 1:(n.c-3)
      for(k in 4:n.c){
         valor <- 0
         for(j in 1:n.times){
            valor <- auc(c.M[1:k], f.joint.all[(1:k),j]) + valor
         }
         F.ct[k-3] <- valor
      }
      F.MT.obs[h] <- rev(F.ct)[1]
   }
   CDF.for.cens <- (F.M.obs-F.MT.obs)/(1-F.T.obs)
   qqnorm(qnorm(c(F.M.given.T[cens.status==1], CDF.for.cens[cens.status==0])))
   abline(0,1)
   qnorm(c(F.M.given.T[cens.status==1], CDF.for.cens[cens.status==0]))
}

### i) Functions to compute residuals for Gaussian copula 

In [ ]:
%%R

#Residuals for T|M fully-parametric (fp):
residuals.T.gauss.fp <- function(parametros, obs.marker, 
                                 surv.times, cens.status){
   rho.par   <- tanh(parametros[1])
   xi.par    <- parametros[2]
   omega.par <- exp(parametros[3])
   alpha.par <- parametros[4]
   shape.par <- exp(parametros[5])
   scale.par <- exp(parametros[6])
   norm.cop <- normalCopula(rho.par)
   f.M.sn <- dsn(obs.marker, xi = xi.par, omega = omega.par, alpha=alpha.par)
   f.T.wei <- dweibull(surv.times, shape = shape.par, scale = scale.par)
   F.M.sn <- psn(obs.marker, xi = xi.par, omega = omega.par, alpha=alpha.par)
   F.T.wei <- pweibull(surv.times, shape = shape.par, scale = scale.par)
   f.joint.MT <- f.M.sn*f.T.wei*dCopula(cbind(F.M.sn, 1-F.T.wei), norm.cop)
   S.m.T <- pnorm((qnorm(1-F.T.wei)-rho.par*qnorm(F.M.sn))/sqrt(1-rho.par^2))
   Lambda.m.T <- -1*log(S.m.T)
   Lambda.set <- data.frame(Lambda.m.T, cens.status)
   KM.Lambda <- survfit(Surv(Lambda.m.T, cens.status) ~ 1, data = Lambda.set)
   plot(log(KM.Lambda$time), log(-1*log(KM.Lambda$surv)), xlim=c(-6,2), ylim=c(-6,2))
   abline(0,1)
   cbind(log(KM.Lambda$time), log(-1*log(KM.Lambda$surv)))
}

#Residuals for M|T fully-parametric (fp):
residuals.M.gauss.fp <- function(parametros, obs.marker, 
                                 surv.times, cens.status){
   rho.par   <- tanh(parametros[1])
   xi.par    <- parametros[2]
   omega.par <- exp(parametros[3])
   alpha.par <- parametros[4]
   shape.par <- exp(parametros[5])
   scale.par <- exp(parametros[6])
   norm.cop <- normalCopula(rho.par)
   f.M.sn <- dsn(obs.marker, xi = xi.par, omega = omega.par, alpha=alpha.par)
   f.T.wei <- dweibull(surv.times, shape = shape.par, scale = scale.par)
   F.M.sn <- psn(obs.marker, xi = xi.par, omega = omega.par, alpha=alpha.par)
   F.T.wei <- pweibull(surv.times, shape = shape.par, scale = scale.par)
   f.joint.MT <- f.M.sn*f.T.wei*dCopula(cbind(F.M.sn, 1-F.T.wei), norm.cop)
   F.t.obs.M <- pnorm((qnorm(F.M.sn)-
                          rho.par*qnorm(1-F.T.wei))/sqrt(1-rho.par^2))
   F.t.cens.M <- pCopula(cbind(F.M.sn, 1-F.T.wei), norm.cop)/(1-F.T.wei)
   qqnorm(qnorm(c(F.t.obs.M[cens.status==1],F.t.cens.M[cens.status==0])))
   abline(0,1)
   qnorm(c(F.t.obs.M[cens.status==1],F.t.cens.M[cens.status==0]))
}

#Residuals for T|M semi-parametric (sp):
residuals.T.gauss.sp <- function(parametro, obs.marker, surv.times, 
                                 xi.M, omega.M, alpha.M, 
                                 times.np, F.T.np, cens.status){
   rho.C <- tanh(parametro)
   c.M <- obs.marker
   n.c <- length(c.M)
   times.k <- times.np
   F.T.k <- F.T.np
   n.times <- length(times.k)
   f.joint.all <- matrix(1:(n.c*n.times), ncol=n.times)
   F.t.minus <- c(0, F.T.k[-n.times])
   F.t.current <- F.T.k
   F.M.sum.T <- 1:n.c
   f.MT <- 1:n.c
   match.times <- match(surv.times, times.np)
   f.M <- dsn(c.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   F.M <- psn(c.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   for(i in 1:n.c){
      f.M.i <- f.M[i]
      F.M.i <- F.M[i]
      f.joint.all[i,] <- f.M.i*(
         pnorm((qnorm(1-F.t.minus)-rho.C*qnorm(F.M.i))/sqrt(1-rho.C^2))-
            pnorm((qnorm(1-F.t.current)-rho.C*qnorm(F.M.i))/sqrt(1-rho.C^2)))
      f.MT[i] <- f.joint.all[i,match.times[i]]
      F.M.sum.T[i] <- sum(f.joint.all[i,][1:match.times[i]])
   }
   S.m.T <- (f.M-F.M.sum.T)/f.M
   Lambda.m.T <- -1*log(S.m.T)
   Lambda.set <- data.frame(Lambda.m.T, cens.status)
   KM.Lambda <- survfit(Surv(Lambda.m.T, cens.status) ~ 1, data = Lambda.set)
   plot(log(KM.Lambda$time), log(-1*log(KM.Lambda$surv)), xlim=c(-6,2), ylim=c(-6,2))
   abline(0,1)
   cbind(log(KM.Lambda$time), log(-1*log(KM.Lambda$surv)))
}


#Residuals for M|T semi-parametric (sp):
residuals.M.gauss.sp <- function(lim.inf, obs.marker, surv.times, cens.status,
                                 rho.C, xi.M, omega.M, alpha.M, 
                                 times.np, F.T.np){
   cdf.t <- approxfun(times.np, F.T.np)
   F.T.obs <- cdf.t(surv.times)
   F.M.obs <- psn(obs.marker, xi = xi.M, omega = omega.M, alpha=alpha.M)
   sample.zs <- length(obs.marker)
   # store the joint distribution in the following object:
   F.MT.obs <- 1:sample.zs
   F.M.given.T <- 1:sample.zs
   for(h in 1:sample.zs){
      c.M <- seq(from=lim.inf, to=obs.marker[h], length=25)
      t.T <- surv.times[h]
      times.k <- times.np[times.np<=t.T]
      F.T.k <- F.T.np[times.np<=t.T]
      n.times <- length(F.T.k)
      n.c <- length(c.M)
      f.joint.all <- matrix(1:(n.c*n.times), ncol=n.times)
      F.t.minus <- c(0, F.T.k[-n.times])
      F.t.current <- F.T.k
      F.M.i <- 1:n.c
      for(i in 1:n.c){
         F.M <- psn(c.M[i], xi = xi.M, omega = omega.M, alpha=alpha.M)
         F.M.i[i] <- F.M
         f.M <- dsn(c.M[i], xi = xi.M, omega = omega.M, alpha=alpha.M)
         f.joint.all[i,] <- f.M*(
            pnorm((qnorm(1-F.t.minus)-rho.C*qnorm(F.M))/sqrt(1-rho.C^2))-
               pnorm((qnorm(1-F.t.current)-rho.C*qnorm(F.M))/sqrt(1-rho.C^2)))
      }
      cat(h, "\n")
      if(cens.status[h]==1){
         cat("auc:", auc(c.M, f.joint.all[,n.times]), "\n")
         cat("n.times", n.times, "\n")
         cat("F.T.k[n.times]", F.T.k[n.times], "\n")
         cat("n.times-1", n.times-1, "\n")
         cat("F.T.k[n.times-1]", F.T.k[n.times-1], "\n")
         if(n.times > 1){
            F.M.given.T[h] <- auc(c.M, f.joint.all[,n.times])/(F.T.k[n.times]-
                                                                  F.T.k[n.times-1])
         }
         else{
            F.M.given.T[h] <- auc(c.M, f.joint.all[,n.times])/(F.T.k[n.times])
         }
      }
      # cat(F.T.k[n.times] - F.T.k[n.times-1], "\n")
      F.ct <- 1:(n.c-3)
      for(k in 4:n.c){
         valor <- 0
         for(j in 1:n.times){
            valor <- auc(c.M[1:k], f.joint.all[(1:k),j]) + valor
         }
         F.ct[k-3] <- valor
      }
      F.MT.obs[h] <- rev(F.ct)[1]
   }
   CDF.for.cens <- (F.M.obs-F.MT.obs)/(1-F.T.obs)
   qqnorm(qnorm(c(F.M.given.T[cens.status==1], CDF.for.cens[cens.status==0])))
   abline(0,1)
   qnorm(c(F.M.given.T[cens.status==1], CDF.for.cens[cens.status==0]))
}

### j) Functions to compute residuals for Clayton copula

In [ ]:
%%R

#Residuals for T|M fully-parametric (fp):
residuals.T.clayton.fp <- function(parametros, obs.marker, surv.times, cens.status){
   theta.par   <- exp(parametros[1])
   xi.par    <- parametros[2]
   omega.par <- exp(parametros[3])
   alpha.par <- parametros[4]
   shape.par <- exp(parametros[5])
   scale.par <- exp(parametros[6])
   clayton.cop <- claytonCopula(theta.par)
   f.M.sn <- dsn(obs.marker, xi = xi.par, omega = omega.par, alpha=alpha.par)
   f.T.wei <- dweibull(surv.times, shape = shape.par, scale = scale.par)
   F.M.sn <- psn(obs.marker, xi = xi.par, omega = omega.par, alpha=alpha.par)
   F.T.wei <- pweibull(surv.times, shape = shape.par, scale = scale.par)
   f.joint.MT <- f.M.sn*f.T.wei*dCopula(cbind(F.M.sn, 1-F.T.wei), clayton.cop)
   psi <- function(x, theta){x^(-theta)-1}
   psi.inv <- function(x, theta){(1+x)^(-1/theta)}
   psi.prima <- function(x, theta){-1*theta*x^(-1*(theta+1))}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   S.m.T <- C.prima(F.M.sn, 1-F.T.wei, theta.par)
   Lambda.m.T <- -1*log(S.m.T)
   Lambda.set <- data.frame(Lambda.m.T, cens.status)
   KM.Lambda <- survfit(Surv(Lambda.m.T, cens.status) ~ 1, data = Lambda.set)
   plot(log(KM.Lambda$time), log(-1*log(KM.Lambda$surv)), xlim=c(-6,2), ylim=c(-6,2))
   abline(0,1)
   cbind(log(KM.Lambda$time), log(-1*log(KM.Lambda$surv)))
}

#Residuals for M|T fully-parametric (fp):
residuals.M.clayton.fp <- function(parametros, obs.marker, surv.times, cens.status){
   theta.par   <- exp(parametros[1])
   xi.par    <- parametros[2]
   omega.par <- exp(parametros[3])
   alpha.par <- parametros[4]
   shape.par <- exp(parametros[5])
   scale.par <- exp(parametros[6])
   clayton.cop <- claytonCopula(theta.par)
   f.M.sn <- dsn(obs.marker, xi = xi.par, omega = omega.par, alpha=alpha.par)
   f.T.wei <- dweibull(surv.times, shape = shape.par, scale = scale.par)
   F.M.sn <- psn(obs.marker, xi = xi.par, omega = omega.par, alpha=alpha.par)
   F.T.wei <- pweibull(surv.times, shape = shape.par, scale = scale.par)
   f.joint.MT <- f.M.sn*f.T.wei*dCopula(cbind(F.M.sn, 1-F.T.wei), clayton.cop)
   psi <- function(x, theta){x^(-theta)-1}
   psi.inv <- function(x, theta){(1+x)^(-1/theta)}
   psi.prima <- function(x, theta){-1*theta*x^(-1*(theta+1))}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   F.t.obs.M <- C.prima(1-F.T.wei, F.M.sn, theta.par)
   F.t.cens.M <- pCopula(cbind(F.M.sn, 1-F.T.wei), clayton.cop)/(1-F.T.wei)
   qqnorm(qnorm(c(F.t.obs.M[cens.status==1],F.t.cens.M[cens.status==0])))
   abline(0,1)
   qnorm(c(F.t.obs.M[cens.status==1],F.t.cens.M[cens.status==0]))
}

#Residuals for T|M semi-parametric (sp):
residuals.T.clayton.sp <- function(parametro, obs.marker, surv.times, 
                                   xi.M, omega.M, alpha.M, 
                                   times.np, F.T.np, cens.status){
   psi <- function(x, theta){x^(-theta)-1}
   psi.inv <- function(x, theta){(1+x)^(-1/theta)}
   psi.prima <- function(x, theta){-1*theta*x^(-1*(theta+1))}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   theta.par   <- exp(parametro)
   c.M <- obs.marker
   n.c <- length(c.M)
   times.k <- times.np
   F.T.k <- F.T.np
   n.times <- length(times.k)
   f.joint.all <- matrix(1:(n.c*n.times), ncol=n.times)
   F.t.minus <- c(0, F.T.k[-n.times])
   F.t.current <- F.T.k
   F.M.sum.T <- 1:n.c
   f.MT <- 1:n.c
   match.times <- match(surv.times, times.np)
   f.M <- dsn(c.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   F.M <- psn(c.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   for(i in 1:n.c){
      f.M.i <- f.M[i]
      F.M.i <- F.M[i]
      f.joint.all[i,] <- f.M.i*(C.prima(F.M.i, 1-F.t.minus, theta.par) -
                                   C.prima(F.M.i, 1-F.t.current, theta.par))
      f.MT[i] <- f.joint.all[i,match.times[i]]
      F.M.sum.T[i] <- sum(f.joint.all[i,][1:match.times[i]])
   }
   S.m.T <- (f.M-F.M.sum.T)/f.M
   Lambda.m.T <- -1*log(S.m.T)
   Lambda.set <- data.frame(Lambda.m.T, cens.status)
   KM.Lambda <- survfit(Surv(Lambda.m.T, cens.status) ~ 1, data = Lambda.set)
   plot(log(KM.Lambda$time), log(-1*log(KM.Lambda$surv)), xlim=c(-6,2), ylim=c(-6,2))
   abline(0,1)
   cbind(log(KM.Lambda$time), log(-1*log(KM.Lambda$surv)))
}

#Residuals for M|T semi-parametric (sp):
residuals.M.clayton.sp <- function(lim.inf, obs.marker, surv.times, cens.status,
                                   parametro, xi.M, omega.M, alpha.M, 
                                   times.np, F.T.np){
   psi <- function(x, theta){x^(-theta)-1}
   psi.inv <- function(x, theta){(1+x)^(-1/theta)}
   psi.prima <- function(x, theta){-1*theta*x^(-1*(theta+1))}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   theta.par   <- exp(parametro)
   cdf.t <- approxfun(times.np, F.T.np)
   F.T.obs <- cdf.t(surv.times)
   F.M.obs <- psn(obs.marker, xi = xi.M, omega = omega.M, alpha=alpha.M)
   sample.zs <- length(obs.marker)
   # store the joint distribution in the following object:
   F.MT.obs <- 1:sample.zs
   F.M.given.T <- 1:sample.zs
   for(h in 1:sample.zs){
      c.M <- seq(from=lim.inf, to=obs.marker[h], length=25)
      t.T <- surv.times[h]
      times.k <- times.np[times.np<=t.T]
      F.T.k <- F.T.np[times.np<=t.T]
      n.times <- length(F.T.k)
      n.c <- length(c.M)
      f.joint.all <- matrix(1:(n.c*n.times), ncol=n.times)
      F.t.minus <- c(0, F.T.k[-n.times])
      F.t.current <- F.T.k
      F.M.i <- 1:n.c
      for(i in 1:n.c){
         F.M <- psn(c.M[i], xi = xi.M, omega = omega.M, alpha=alpha.M)
         F.M.i[i] <- F.M
         f.M <- dsn(c.M[i], xi = xi.M, omega = omega.M, alpha=alpha.M)
         f.joint.all[i,] <- f.M*(C.prima(F.M, 1-F.t.minus, theta.par) -
                                    C.prima(F.M, 1-F.t.current, theta.par))
      }
      cat(h, "\n")
      if(cens.status[h]==1){
         cat("auc:", auc(c.M, f.joint.all[,n.times]), "\n")
         cat("n.times", n.times, "\n")
         cat("F.T.k[n.times]", F.T.k[n.times], "\n")
         cat("n.times-1", n.times-1, "\n")
         cat("F.T.k[n.times-1]", F.T.k[n.times-1], "\n")
         if(n.times > 1){
            F.M.given.T[h] <- auc(c.M, f.joint.all[,n.times])/(F.T.k[n.times]-
                                                                  F.T.k[n.times-1])
         }
         else{
            F.M.given.T[h] <- auc(c.M, f.joint.all[,n.times])/(F.T.k[n.times])
         }
      }
      # cat(F.T.k[n.times] - F.T.k[n.times-1], "\n")
      F.ct <- 1:(n.c-3)
      for(k in 4:n.c){
         valor <- 0
         for(j in 1:n.times){
            valor <- auc(c.M[1:k], f.joint.all[(1:k),j]) + valor
         }
         F.ct[k-3] <- valor
      }
      F.MT.obs[h] <- rev(F.ct)[1]
   }
   CDF.for.cens <- (F.M.obs-F.MT.obs)/(1-F.T.obs)
   qqnorm(qnorm(c(F.M.given.T[cens.status==1], CDF.for.cens[cens.status==0])))
   abline(0,1)
   qnorm(c(F.M.given.T[cens.status==1], CDF.for.cens[cens.status==0]))
}


### k) Fully-parametric ROC curve functions for Frank copula 

In [ ]:
%%R 

#Cumulative/Dynamic fully-parametric ROC curve:
ROC.CD.fp.frank <- function(m.M, t.T, theta.par, 
                            xi.M, omega.M, alpha.M, shape.T, scale.T){
   frank.cop <- frankCopula(theta.par)
   F.M.sn <- psn(m.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   F.T.wei <- pweibull(t.T, shape = shape.T, scale = scale.T)
   H.MT <- pCopula(cbind(F.M.sn, 1-F.T.wei), frank.cop)
   TP.C <- (H.MT-F.M.sn-(1-F.T.wei)+1)/F.T.wei
   FP.D <- 1-(H.MT/(1-F.T.wei))
   plot(c(1,FP.D,0), c(1,TP.C,0), type="l")
   data.frame(FP.D=c(1,FP.D,0), TP.C=c(1,TP.C,0))
}

#Incident/Dynamic fully-parametric ROC curve: 
ROC.ID.fp.frank <- function(m.M, t.T, theta.par, 
                            xi.M, omega.M, alpha.M, shape.T, scale.T){
   psi <- function(x, theta){-1*log((exp(-theta*x)-1)/(exp(-theta)-1))}
   psi.inv <- function(x, theta){-log(1-(1-exp(-theta))*exp(-x))/theta}
   psi.prima <- function(x, theta){
      (theta*exp(-theta*x))/(exp(-theta*x)-1)}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   frank.cop <- frankCopula(theta.par)
   F.M.sn <- psn(m.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   F.T.wei <- pweibull(t.T, shape = shape.T, scale = scale.T)
   H.MT <- pCopula(cbind(F.M.sn, 1-F.T.wei), frank.cop)
   TP.I <- 1-C.prima(1-F.T.wei, F.M.sn, theta.par)
   # pnorm((rho.C*qnorm(1-F.T.wei)-qnorm(F.M.sn))/sqrt(1-rho.C^2))
   FP.D <- 1-(H.MT/(1-F.T.wei))
   plot(c(1,FP.D,0), c(1,TP.I,0), type="l")
   data.frame(FP.D=c(1,FP.D,0), TP.I=c(1,TP.I,0))
}


### l) Semi-parametric ROC curve functions for Frank copula 

In [ ]:
%%R 

#Cumulative/Dynamic semi-parametric ROC curve: 
ROC.CD.sp.frank <- function(c.M, t.T, theta.par, xi.M, omega.M, alpha.M, 
                            times.np, F.T.np, n.event){
   psi <- function(x, theta){-1*log((exp(-theta*x)-1)/(exp(-theta)-1))}
   psi.inv <- function(x, theta){-log(1-(1-exp(-theta))*exp(-x))/theta}
   psi.prima <- function(x, theta){
      (theta*exp(-theta*x))/(exp(-theta*x)-1)}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   times.np <- times.np[n.event>0]
   F.T.np <- F.T.np[n.event>0]
   times.k <- times.np[times.np<=t.T]
   F.T.k <- F.T.np[times.np<=t.T]
   n.times <- length(F.T.k)
   n.c <- length(c.M)
   f.joint.all <- matrix(1:(n.c*n.times), ncol=n.times)
   F.t.minus <- c(0, F.T.k[-n.times])
   F.t.current <- F.T.k
   F.M.i <- 1:n.c
   for(i in 1:n.c){
      F.M <- psn(c.M[i], xi = xi.M, omega = omega.M, alpha=alpha.M)
      F.M.i[i] <- F.M
      f.M <- dsn(c.M[i], xi = xi.M, omega = omega.M, alpha=alpha.M)
      f.joint.all[i,] <- f.M*(C.prima(F.M, 1-F.t.minus, theta.par) -
                                 C.prima(F.M, 1-F.t.current, theta.par))
   }
   F.ct <- 1:(n.c-3)
   for(i in 4:n.c){
      valor <- 0
      for(j in 1:n.times){
         valor <- auc(c.M[1:i], f.joint.all[(1:i),j]) + valor
      }
      F.ct[i-3] <- valor
   }
   TP.C <- 1 - (F.ct/F.T.k[n.times])
   FP.D <- 1 - ((F.M.i[4:n.c]- F.ct)/(1-F.T.k[n.times]))
   plot(c(1,FP.D,0), c(1,TP.C,0), type="l")
   data.frame(FP.D=c(1,FP.D,0), TP.C=c(1,TP.C,0))
}

#Incident/Dynamic semi-parametric ROC curve: 
ROC.ID.sp.frank <- function(c.M, t.T, theta.par, xi.M, omega.M, alpha.M, 
                            times.np, F.T.np, n.event){
   psi <- function(x, theta){-1*log((exp(-theta*x)-1)/(exp(-theta)-1))}
   psi.inv <- function(x, theta){-log(1-(1-exp(-theta))*exp(-x))/theta}
   psi.prima <- function(x, theta){
      (theta*exp(-theta*x))/(exp(-theta*x)-1)}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   times.np <- times.np[n.event>0]
   F.T.np <- F.T.np[n.event>0]
   times.k <- times.np[times.np<=t.T]
   F.T.k <- F.T.np[times.np<=t.T]
   n.times <- length(F.T.k)
   n.c <- length(c.M)
   f.joint.all <- matrix(1:(n.c*n.times), ncol=n.times)
   F.t.minus <- c(0, F.T.k[-n.times])
   F.t.current <- F.T.k
   F.M.i <- 1:n.c
   for(i in 1:n.c){
      F.M <- psn(c.M[i], xi = xi.M, omega = omega.M, alpha=alpha.M)
      F.M.i[i] <- F.M
      f.M <- dsn(c.M[i], xi = xi.M, omega = omega.M, alpha=alpha.M)
      f.joint.all[i,] <- f.M*(C.prima(F.M, 1-F.t.minus, theta.par) -
                                 C.prima(F.M, 1-F.t.current, theta.par))
   }
   F.ct <- 1:(n.c-3)
   for(i in 4:n.c){
      valor <- 0
      for(j in 1:n.times){
         valor <- auc(c.M[1:i], f.joint.all[(1:i),j]) + valor
      }
      F.ct[i-3] <- valor
   }
   F.ct.I <- 1:(n.c-3)
   for(i in 4:n.c){
      F.ct.I[i-3] <- auc(c.M[1:i], f.joint.all[(1:i),n.times]) 
   }
   TP.I <- 1 - (F.ct.I/(F.T.k[n.times]-F.T.k[n.times-1]))
   FP.D <- 1 - ((F.M.i[4:n.c]- F.ct)/(1-F.T.k[n.times]))
   plot(c(1,FP.D,0), c(1,TP.I,0), type="l")
   data.frame(FP.D=c(1,FP.D,0), TP.I=c(1,TP.I,0))
}


### m) Fully-parametric ROC curve functions for Gumbel copula

In [ ]:
%%R

#Cumulative/Dynamic fully-parametric ROC curve:
ROC.CD.fp.gumbel <- function(m.M, t.T, theta.par, 
                             xi.M, omega.M, alpha.M, shape.T, scale.T){
   gumbel.cop <- gumbelCopula(theta.par)
   F.M.sn <- psn(m.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   F.T.wei <- pweibull(t.T, shape = shape.T, scale = scale.T)
   H.MT <- pCopula(cbind(F.M.sn, 1-F.T.wei), gumbel.cop)
   TP.C <- (H.MT-F.M.sn-(1-F.T.wei)+1)/F.T.wei
   FP.D <- 1-(H.MT/(1-F.T.wei))
   plot(c(1,FP.D,0), c(1,TP.C,0), type="l")
   data.frame(FP.D=c(1,FP.D,0), TP.C=c(1,TP.C,0))
}

#Incident/Dynamic fully-parametric ROC curve: 
ROC.ID.fp.gumbel <- function(m.M, t.T, theta.par, 
                             xi.M, omega.M, alpha.M, shape.T, scale.T){
   psi <- function(x, theta){(-log(x))^(theta)}
   psi.inv <- function(x, theta){exp(-x^(1/theta))}
   psi.prima <- function(x, theta){-(theta/x)*(-log(x))^(theta-1)}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   gumbel.cop <- gumbelCopula(theta.par)
   F.M.sn <- psn(m.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   F.T.wei <- pweibull(t.T, shape = shape.T, scale = scale.T)
   H.MT <- pCopula(cbind(F.M.sn, 1-F.T.wei), gumbel.cop)
   TP.I <- 1-C.prima(1-F.T.wei, F.M.sn, theta.par)
   # pnorm((rho.C*qnorm(1-F.T.wei)-qnorm(F.M.sn))/sqrt(1-rho.C^2))
   FP.D <- 1-(H.MT/(1-F.T.wei))
   plot(c(1,FP.D,0), c(1,TP.I,0), type="l")
   data.frame(FP.D=c(1,FP.D,0), TP.I=c(1,TP.I,0))
}

### n) Semi-parametric ROC curve functions for Gumbel copula

In [ ]:
%%R

#Cumulative/Dynamic semi-parametric ROC curve: 
ROC.CD.sp.gumbel <- function(c.M, t.T, theta.par, xi.M, omega.M, alpha.M, 
                             times.np, F.T.np, n.event){
   psi <- function(x, theta){(-log(x))^(theta)}
   psi.inv <- function(x, theta){exp(-x^(1/theta))}
   psi.prima <- function(x, theta){-(theta/x)*(-log(x))^(theta-1)}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   times.np <- times.np[n.event>0]
   F.T.np <- F.T.np[n.event>0]
   times.k <- times.np[times.np<=t.T]
   F.T.k <- F.T.np[times.np<=t.T]
   n.times <- length(F.T.k)
   n.c <- length(c.M)
   cat("n.c=", n.c, "\n")
   cat("n.times", n.times, "\n")
   f.joint.all <- matrix(1:(n.c*n.times), ncol=n.times)
   F.t.minus <- c(0, F.T.k[-n.times])
   F.t.current <- F.T.k
   F.M.i <- 1:n.c
   for(i in 1:n.c){
      F.M <- psn(c.M[i], xi = xi.M, omega = omega.M, alpha=alpha.M)
      F.M.i[i] <- F.M
      f.M <- dsn(c.M[i], xi = xi.M, omega = omega.M, alpha=alpha.M)
      f.joint.all[i,] <- f.M*(C.prima(F.M, 1-F.t.minus, theta.par) -
                                 C.prima(F.M, 1-F.t.current, theta.par))
   }
   F.ct <- 1:(n.c-3)
   for(i in 4:n.c){
      valor <- 0
      for(j in 1:n.times){
         valor <- auc(c.M[1:i], f.joint.all[(1:i),j]) + valor
      }
      F.ct[i-3] <- valor
   }
   TP.C <- 1 - (F.ct/F.T.k[n.times])
   FP.D <- 1 - ((F.M.i[4:n.c]- F.ct)/(1-F.T.k[n.times]))
   plot(c(1,FP.D,0), c(1,TP.C,0), type="l")
   data.frame(FP.D=c(1,FP.D,0), TP.C=c(1,TP.C,0))
}

#Incident/Dynamic semi-parametric ROC curve: 
ROC.ID.sp.gumbel <- function(c.M, t.T, theta.par, xi.M, omega.M, alpha.M, 
                             times.np, F.T.np, n.event){
   psi <- function(x, theta){(-log(x))^(theta)}
   psi.inv <- function(x, theta){exp(-x^(1/theta))}
   psi.prima <- function(x, theta){-(theta/x)*(-log(x))^(theta-1)}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   times.np <- times.np[n.event>0]
   F.T.np <- F.T.np[n.event>0]
   times.k <- times.np[times.np<=t.T]
   F.T.k <- F.T.np[times.np<=t.T]
   n.times <- length(F.T.k)
   n.c <- length(c.M)
   f.joint.all <- matrix(1:(n.c*n.times), ncol=n.times)
   F.t.minus <- c(0, F.T.k[-n.times])
   F.t.current <- F.T.k
   F.M.i <- 1:n.c
   for(i in 1:n.c){
      F.M <- psn(c.M[i], xi = xi.M, omega = omega.M, alpha=alpha.M)
      F.M.i[i] <- F.M
      f.M <- dsn(c.M[i], xi = xi.M, omega = omega.M, alpha=alpha.M)
      f.joint.all[i,] <- f.M*(C.prima(F.M, 1-F.t.minus, theta.par) -
                                 C.prima(F.M, 1-F.t.current, theta.par))
   }
   F.ct <- 1:(n.c-3)
   for(i in 4:n.c){
      valor <- 0
      for(j in 1:n.times){
         valor <- auc(c.M[1:i], f.joint.all[(1:i),j]) + valor
      }
      F.ct[i-3] <- valor
   }
   F.ct.I <- 1:(n.c-3)
   for(i in 4:n.c){
      F.ct.I[i-3] <- auc(c.M[1:i], f.joint.all[(1:i),n.times]) 
   }
   TP.I <- 1 - (F.ct.I/(F.T.k[n.times]-F.T.k[n.times-1]))
   FP.D <- 1 - ((F.M.i[4:n.c]- F.ct)/(1-F.T.k[n.times]))
   plot(c(1,FP.D,0), c(1,TP.I,0), type="l")
   data.frame(FP.D=c(1,FP.D,0), TP.I=c(1,TP.I,0))
}


### o) Predictiveness curves functions for Frank copula

In [ ]:
%%R 

#Fully-parametric (fp):
PRED.fp.frank <- function(m.M, t.T, theta.par, 
                          xi.M, omega.M, alpha.M, shape.T, scale.T){
   psi <- function(x, theta){-1*log((exp(-theta*x)-1)/(exp(-theta)-1))}
   psi.inv <- function(x, theta){-log(1-(1-exp(-theta))*exp(-x))/theta}
   psi.prima <- function(x, theta){
      (theta*exp(-theta*x))/(exp(-theta*x)-1)}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   frank.cop <- frankCopula(theta.par)
   F.M.sn <- psn(m.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   F.T.wei <- pweibull(t.T, shape = shape.T, scale = scale.T)
   cat("Prevalence=", F.T.wei, "\n")
   H.MT <- pCopula(cbind(F.M.sn, 1-F.T.wei), frank.cop)
   Risk.MT <- 1-C.prima(F.M.sn, 1-F.T.wei, theta.par)
   plot(F.M.sn, Risk.MT, type="l", ylim=c(0,1))
   data.frame(F.M.sn, Risk.MT)
}

#Semi-parametric (sp):
PRED.sp.frank <- function(c.M, t.T, theta.par, xi.M, omega.M, alpha.M, 
                          times.np, F.T.np, n.event){
   psi <- function(x, theta){-1*log((exp(-theta*x)-1)/(exp(-theta)-1))}
   psi.inv <- function(x, theta){-log(1-(1-exp(-theta))*exp(-x))/theta}
   psi.prima <- function(x, theta){
      (theta*exp(-theta*x))/(exp(-theta*x)-1)}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   times.np <- times.np[n.event>0]
   F.T.np <- F.T.np[n.event>0]
   times.k <- times.np[times.np<=t.T]
   F.T.k <- F.T.np[times.np<=t.T]
   n.times <- length(F.T.k)
   n.c <- length(c.M)
   f.joint.all <- matrix(1:(n.c*n.times), ncol=n.times)
   F.t.minus <- c(0, F.T.k[-n.times])
   F.t.current <- F.T.k
   F.M.i <- 1:n.c
   for(i in 1:n.c){
      F.M <- psn(c.M[i], xi = xi.M, omega = omega.M, alpha=alpha.M)
      F.M.i[i] <- F.M
      f.M <- dsn(c.M[i], xi = xi.M, omega = omega.M, alpha=alpha.M)
      f.joint.all[i,] <- f.M*(C.prima(F.M, 1-F.t.minus, theta.par) -
                                 C.prima(F.M, 1-F.t.current, theta.par))
   }
   f.M <- dsn(c.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   F.M <- psn(c.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   F.mT <- apply(f.joint.all, 1, sum)
   Risk.MT <- F.mT/f.M
   plot(F.M, Risk.MT, type="l", ylim=c(0,1))
   data.frame(F.M, Risk.MT)
}


### p) Predictiveness curves functions for Gumbel copula 

In [ ]:
%%R 

#Fully-parametric (fp):
PRED.fp.gumbel <- function(m.M, t.T, theta.par, 
                           xi.M, omega.M, alpha.M, shape.T, scale.T){
   psi <- function(x, theta){(-log(x))^(theta)}
   psi.inv <- function(x, theta){exp(-x^(1/theta))}
   psi.prima <- function(x, theta){-(theta/x)*(-log(x))^(theta-1)}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   gumbel.cop <- gumbelCopula(theta.par)
   F.M.sn <- psn(m.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   F.T.wei <- pweibull(t.T, shape = shape.T, scale = scale.T)
   H.MT <- pCopula(cbind(F.M.sn, 1-F.T.wei), gumbel.cop)
   Risk.MT <- 1-C.prima(F.M.sn, 1-F.T.wei, theta.par)
   plot(F.M.sn, Risk.MT, type="l", ylim=c(0,1))
   data.frame(F.M.sn, Risk.MT)
}

#Semi-parametric (sp):
PRED.sp.gumbel <- function(c.M, t.T, theta.par, xi.M, omega.M, alpha.M, 
                           times.np, F.T.np, n.event){
   psi <- function(x, theta){(-log(x))^(theta)}
   psi.inv <- function(x, theta){exp(-x^(1/theta))}
   psi.prima <- function(x, theta){-(theta/x)*(-log(x))^(theta-1)}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   times.np <- times.np[n.event>0]
   F.T.np <- F.T.np[n.event>0]
   times.k <- times.np[times.np<=t.T]
   F.T.k <- F.T.np[times.np<=t.T]
   n.times <- length(F.T.k)
   n.c <- length(c.M)
   f.joint.all <- matrix(1:(n.c*n.times), ncol=n.times)
   F.t.minus <- c(0, F.T.k[-n.times])
   F.t.current <- F.T.k
   F.M.i <- 1:n.c
   for(i in 1:n.c){
      F.M <- psn(c.M[i], xi = xi.M, omega = omega.M, alpha=alpha.M)
      F.M.i[i] <- F.M
      f.M <- dsn(c.M[i], xi = xi.M, omega = omega.M, alpha=alpha.M)
      f.joint.all[i,] <- f.M*(C.prima(F.M, 1-F.t.minus, theta.par) -
                                 C.prima(F.M, 1-F.t.current, theta.par))
   }
   f.M <- dsn(c.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   F.M <- psn(c.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   F.mT <- apply(f.joint.all, 1, sum)
   Risk.MT <- F.mT/f.M
   plot(F.M, Risk.MT, type="l", ylim=c(0,1))
   data.frame(F.M=F.M, Risk.MT=Risk.MT)
}

### q) Standardized Total Gain functions for Frank copula

In [ ]:
%%R 

#Fully-parametric (fp):
std.total.gain.fp.frank <- function(m.M, t.T, theta.par, 
                                    xi.M, omega.M, alpha.M, shape.T, scale.T){
   psi <- function(x, theta){-1*log((exp(-theta*x)-1)/(exp(-theta)-1))}
   psi.inv <- function(x, theta){-log(1-(1-exp(-theta))*exp(-x))/theta}
   psi.prima <- function(x, theta){
      (theta*exp(-theta*x))/(exp(-theta*x)-1)}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   frank.cop <- frankCopula(theta.par)
   F.M.sn <- psn(m.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   F.T.wei <- pweibull(t.T, shape = shape.T, scale = scale.T)
   H.MT <- pCopula(cbind(F.M.sn, 1-F.T.wei), frank.cop)
   Risk.MT <- 1-C.prima(F.M.sn, 1-F.T.wei, theta.par)
   prevalence.t <- F.T.wei
   TG <- auc(F.M.sn, abs(Risk.MT-prevalence.t))
   cat("Prevalence", prevalence.t, "\n")
   TG/(2*prevalence.t*(1-prevalence.t))
}
#Semi-parametric (sp):
std.total.gain.sp.frank <- function(c.M, t.T, theta.par, xi.M, omega.M, alpha.M, 
                                    times.np, F.T.np, n.event){
   psi <- function(x, theta){-1*log((exp(-theta*x)-1)/(exp(-theta)-1))}
   psi.inv <- function(x, theta){-log(1-(1-exp(-theta))*exp(-x))/theta}
   psi.prima <- function(x, theta){
      (theta*exp(-theta*x))/(exp(-theta*x)-1)}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   times.np <- times.np[n.event>0]
   F.T.np <- F.T.np[n.event>0]
   times.k <- times.np[times.np<=t.T]
   F.T.k <- F.T.np[times.np<=t.T]
   prevalence.t <- max(F.T.k)
   n.times <- length(F.T.k)
   n.c <- length(c.M)
   f.joint.all <- matrix(1:(n.c*n.times), ncol=n.times)
   F.t.minus <- c(0, F.T.k[-n.times])
   F.t.current <- F.T.k
   F.M.i <- 1:n.c
   for(i in 1:n.c){
      F.M <- psn(c.M[i], xi = xi.M, omega = omega.M, alpha=alpha.M)
      F.M.i[i] <- F.M
      f.M <- dsn(c.M[i], xi = xi.M, omega = omega.M, alpha=alpha.M)
      f.joint.all[i,] <- f.M*(C.prima(F.M, 1-F.t.minus, theta.par) -
                                 C.prima(F.M, 1-F.t.current, theta.par))
   }
   f.M <- dsn(c.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   F.M <- psn(c.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   F.mT <- apply(f.joint.all, 1, sum)
   Risk.MT <- F.mT/f.M
   TG <- auc(F.M, abs(Risk.MT-prevalence.t))
   TG/(2*prevalence.t*(1-prevalence.t))
}

### r) Standardized Total Gain functions for Gumbel copula

In [ ]:
%%R

#Fully-parametric (fp):
std.total.gain.fp.gumbel <- function(m.M, t.T, theta.par, 
                                     xi.M, omega.M, alpha.M, shape.T, scale.T){
   psi <- function(x, theta){(-log(x))^(theta)}
   psi.inv <- function(x, theta){exp(-x^(1/theta))}
   psi.prima <- function(x, theta){-(theta/x)*(-log(x))^(theta-1)}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   gumbel.cop <- gumbelCopula(theta.par)
   F.M.sn <- psn(m.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   F.T.wei <- pweibull(t.T, shape = shape.T, scale = scale.T)
   H.MT <- pCopula(cbind(F.M.sn, 1-F.T.wei), gumbel.cop)
   Risk.MT <- 1-C.prima(F.M.sn, 1-F.T.wei, theta.par)
   prevalence.t <- F.T.wei
   TG <- auc(F.M.sn, abs(Risk.MT-prevalence.t))
   cat("Prevalence", prevalence.t, "\n")
   TG/(2*prevalence.t*(1-prevalence.t))
}

#Semi-parametric (sp):
std.total.gain.sp.gumbel <- function(c.M, t.T, theta.par, xi.M, omega.M, alpha.M, 
                                     times.np, F.T.np, n.event){
   psi <- function(x, theta){(-log(x))^(theta)}
   psi.inv <- function(x, theta){exp(-x^(1/theta))}
   psi.prima <- function(x, theta){-(theta/x)*(-log(x))^(theta-1)}
   C.prima <- function(u,v, theta){
      C.uv <- function(u,v, theta){psi.inv(psi(u, theta)+psi(v, theta),theta)}
      psi.prima(u, theta)/psi.prima(C.uv(u,v, theta), theta)
   }
   times.np <- times.np[n.event>0]
   F.T.np <- F.T.np[n.event>0]
   times.k <- times.np[times.np<=t.T]
   F.T.k <- F.T.np[times.np<=t.T]
   prevalence.t <- max(F.T.k)
   n.times <- length(F.T.k)
   n.c <- length(c.M)
   f.joint.all <- matrix(1:(n.c*n.times), ncol=n.times)
   F.t.minus <- c(0, F.T.k[-n.times])
   F.t.current <- F.T.k
   F.M.i <- 1:n.c
   for(i in 1:n.c){
      F.M <- psn(c.M[i], xi = xi.M, omega = omega.M, alpha=alpha.M)
      F.M.i[i] <- F.M
      f.M <- dsn(c.M[i], xi = xi.M, omega = omega.M, alpha=alpha.M)
      f.joint.all[i,] <- f.M*(C.prima(F.M, 1-F.t.minus, theta.par) -
                                 C.prima(F.M, 1-F.t.current, theta.par))
   }
   f.M <- dsn(c.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   F.M <- psn(c.M, xi = xi.M, omega = omega.M, alpha=alpha.M)
   F.mT <- apply(f.joint.all, 1, sum)
   Risk.MT <- F.mT/f.M
   TG <- auc(F.M, abs(Risk.MT-prevalence.t))
   TG/(2*prevalence.t*(1-prevalence.t))
}


# Section 2

## Preprocessing step

Just the first 312 patients have a treatment condition (trt: 1 for D-penicillmain, 2 for placebo). 
The rest of the patients are disregarded for the analysis (trt: NA for not randomised).

In [ ]:
%%R
#Patient filter
pbc.mayo <- pbc[1:312,]

# The status (0 = censored, 1 = dead):
pbc.mayo$vital.status <- 1*(pbc.mayo$status >1)

# The edema status (1 = edema despite diuretic therapy, 0 = otherwise):
pbc.mayo$edema.status <- factor(pbc.mayo$edema > 0.5)

# Kaplan-Meier estimator
KM.mayo <- survfit(Surv(time, vital.status) ~  1, 
                type="kaplan-meier", conf.type="none",
                data=pbc.mayo)

# Weibull distribution for survival times:
weib.mayo <- survreg(Surv(time, vital.status) ~ 1, data=pbc.mayo,
                    dist="weibull")

# The shape parameter:
weib.shape <- 1/weib.mayo$scale

#Survival distribution
plot(KM.mayo, conf.int = FALSE, xlab=substitute(paste(italic('t'))),
            ylab=substitute(paste('S'[T],"(", italic('t'), ")")), sub="(a)")
lines(x=seq(from=0.01, to=4600, length=100),
      y=1-pweibull(seq(from=0.01, to=4600, length=100),
                   shape=weib.shape, 
                   scale=exp(coef(weib.mayo))), lty=2)
legend(500, 0.4, c("Kaplan-Meier", "Weibull"), lty=1:2, bty = "n")


## Marker definiton

### a) Mayo marker

In [ ]:
%%R 

#Cox Regression model
cox.marker.mayo <- coxph(Surv(time,vital.status) ~ poly(albumin, 1) +
                     log(bili) + log(protime) + edema.status + poly(age, 1), 
                     x=TRUE,
                     data=pbc.mayo)

#Testing the Proportional Hazards Assumption of a Cox Regression model
zph.marker.mayo <- cox.zph(cox.marker.mayo)
print(zph.marker.mayo)
plot(zph.marker.mayo)

#Mayo marker
pbc.mayo$marker.mayo <- cox.marker.mayo$x %*% c(coef(cox.marker.mayo))  


# Histogram of the marker
hist(pbc.mayo$marker.mayo, freq = F, nclass=7, col = 'gray', xlab="Marker",
     main="", ylim=c(0, 0.35), sub="(c)")
rug(pbc.mayo$marker.mayo)
lines(density(pbc.mayo$marker.mayo, kernel = "epanechnikov"), lwd=1.5)


# Skew-normal fitting
fit.marker.mayo <- selm(marker.mayo~1, family="SN", data=pbc.mayo)
summary(fit.marker.mayo, "DP", cov=TRUE, cor=TRUE)

# Parameters estimation
xi.mayo <- fit.marker.mayo@param$dp[1]
omega.mayo <- fit.marker.mayo@param$dp[2]
alpha.mayo <- fit.marker.mayo@param$dp[3]


# Adding the fitted skew-normal density to the histogram
curve(dsn(x, xi=xi.mayo, omega=omega.mayo, alpha=alpha.mayo), 
      from=5,to=14, add=T, lwd=2, lty=2)
legend(10, 0.21, c("Kernel estimator", "Skewed normal"),
       lty=1:2, lwd=c(1.5,2), bty="n")


### b) 4cov marker

In [ ]:
%%R

#Cox Regression model
cox.marker.4cov <- coxph(Surv(time,vital.status) ~ poly(albumin, 1) +
                     log(protime) + edema.status + poly(age, 1), 
                     x=TRUE,
                     data=pbc.mayo)

#Testing the Proportional Hazards Assumption of a Cox Regression model
zph.marker.4cov <- cox.zph(cox.marker.4cov)
print(zph.marker.4cov)
plot(zph.marker.4cov)

#4cov marker
pbc.mayo$marker.4cov <- cox.marker.4cov$x %*% c(coef(cox.marker.4cov))  

# Histogram of the marker
hist(pbc.mayo$marker.4cov, freq = F, nclass=7, col = 'gray', xlab="Marker",
     main="", ylim=c(0, 0.6), sub="(b)")
rug(pbc.mayo$marker.4cov)
lines(density(pbc.mayo$marker.4cov, kernel = "epanechnikov"), lwd=1.5)


# Skew-normal fitting
fit.marker.4cov <- selm(marker.4cov~1, family="SN", data=pbc.mayo)
summary(fit.marker.4cov, "DP", cov=TRUE, cor=TRUE)

# Parameters estimation
xi.4cov <- fit.marker.4cov@param$dp[1]
omega.4cov <- fit.marker.4cov@param$dp[2]
alpha.4cov <- fit.marker.4cov@param$dp[3]


# Adding the fitted skew-normal density to the histogram
curve(dsn(x, xi=xi.4cov , omega=omega.4cov , alpha=alpha.4cov), 
      from=5,to=14, add=T, lwd=2, lty=2)
legend(12, 0.21, c("Kernel estimator", "Skewed normal"),
       lty=1:2, lwd=c(1.5,2), bty="n")

# Section 3

## Model fitting for residual analysis

Residual analysis is used as a guide for choosing a copula from a set of candidates. Thus, to inspect the residuals from each conditional distribution, a model fitting for every copula is first computed.

### 1) Frank copula 

#### 1.1) Fully-parametric (fp)

In [ ]:
%%R

# Mayo marker

#Initial values
var.ini.frank.fp.mayo <- c(5, 6, log(2), 6, 0, 8) 

#MLE estimation
mle.fp.frank.mayo <- nlm(ll.fp.frank, var.ini.frank.fp.mayo,
                         obs.marker=pbc.mayo$marker.mayo, surv.times=pbc.mayo$time, 
                         cens.status=pbc.mayo$vital.status,
                         hessian=T)

print("Tau(Mayo marker)")
print(tau(frankCopula(mle.fp.frank.mayo$estimate[1], dim= 2)))

# 4-cov marker

#Initial values
var.ini.frank.fp.4cov <- c(5, 8.5, log(1.6), 4, 0, 8)

#MLE estimation
mle.fp.frank.4cov <- nlm(ll.fp.frank, var.ini.frank.fp.4cov,
                         obs.marker=pbc.mayo$marker.4cov, surv.times=pbc.mayo$time, 
                         cens.status=pbc.mayo$vital.status,
                         hessian=T)

print("Tau(4cov marker)")
print(tau(frankCopula(mle.fp.frank.4cov$estimate[1], dim= 2)))

#### 1.2) Semi-parametric (sp)

In [ ]:
%%R 
# Mayo marker:

#Marker margin
fit.marker.mayo <- selm(marker.mayo~1, family="SN", data=pbc.mayo)
summary(fit.marker.mayo, "DP", cov=TRUE, cor=TRUE)
estimates.sn.mayo <- coef(fit.marker.mayo, "DP")

#Time margin
KM.mayo.obj <- survfit(Surv(time, vital.status) ~  1, 
                       data=pbc.mayo)

#MLE estimation
mle.sp.frank.mayo <- nlm(ll.sp.frank, 6, 
                         obs.marker=c(pbc.mayo$marker.mayo),
                         surv.times=pbc.mayo$time,
                         xi.M=estimates.sn.mayo[1], 
                         omega.M=estimates.sn.mayo[2], 
                         alpha.M=estimates.sn.mayo[3],
                         times.np= KM.mayo.obj$time, F.T.np=1 - KM.mayo.obj$surv,  
                         cens.status=pbc.mayo$vital.status)

print("Tau(Mayo marker)")
print(tau(frankCopula(mle.sp.frank.mayo$estimate, dim= 2)))

# 4-cov marker

#Marker margin
fit.marker.4cov <- selm(marker.4cov~1, family="SN", data=pbc.mayo)
summary(fit.marker.4cov, "DP", cov=TRUE, cor=TRUE)
estimates.sn.4cov <- coef(fit.marker.4cov, "DP")

#Time margin
KM.mayo.obj <- survfit(Surv(time, vital.status) ~  1, 
                       data=pbc.mayo)

#MLE estimation
mle.sp.frank.4cov <- nlm(ll.sp.frank, 6,
                         obs.marker=c(pbc.mayo$marker.4cov),
                         surv.times=pbc.mayo$time,
                         xi.M=estimates.sn.4cov[1], 
                         omega.M=estimates.sn.4cov[2], 
                         alpha.M=estimates.sn.4cov[3],
                         times.np= KM.mayo.obj$time, F.T.np=1 - KM.mayo.obj$surv,  
                         cens.status=pbc.mayo$vital.status)

print("Tau(4cov marker)")
print(tau(frankCopula(mle.sp.frank.4cov$estimate, dim= 2)))



### 2) Gumbel copula 

#### 2.1) Fully-parametric (fp) 

In [ ]:
%%R

# Mayo marker:

#Initial values
var.ini.gumbel.fp.mayo <- c(0, 6, log(2), 6, 0, 8)

#MLE estimation
mle.fp.gumbel.mayo <- nlm(ll.fp.gumbel, var.ini.gumbel.fp.mayo,
                          obs.marker=pbc.mayo$marker.mayo, surv.times=pbc.mayo$time, 
                          cens.status=pbc.mayo$vital.status,
                          hessian=T)

print("Tau(Mayo marker)")
print(tau(gumbelCopula(exp(mle.fp.gumbel.mayo$estimate[1])+1, dim= 2)))

# 4-cov marker

#Initial values
var.ini.gumbel.fp.4cov <- c(0, 8.5, log(1.6), 4, 0, 8)

#MLE estimation
mle.fp.gumbel.4cov <- nlm(ll.fp.gumbel, var.ini.gumbel.fp.4cov,
                          obs.marker=pbc.mayo$marker.4cov, surv.times=pbc.mayo$time, 
                          cens.status=pbc.mayo$vital.status,
                          hessian=T)

print("Tau(4cov marker)")
print(tau(gumbelCopula(exp(mle.fp.gumbel.4cov$estimate[1])+1, dim= 2)))


#### 2.2) Semi-parametric (sp)

In [ ]:
%%R
# Mayo marker:

#Marker margin
fit.marker.mayo <- selm(marker.mayo~1, family="SN", data=pbc.mayo)
summary(fit.marker.mayo, "DP", cov=TRUE, cor=TRUE)
estimates.sn.mayo <- coef(fit.marker.mayo, "DP")

#Time margin
KM.mayo.obj <- survfit(Surv(time, vital.status) ~  1, 
                       data=pbc.mayo)

#MLE estimation
mle.sp.gumbel.mayo <- nlm(ll.sp.gumbel, 0, 
                          obs.marker=c(pbc.mayo$marker.mayo),
                          surv.times=pbc.mayo$time,
                          xi.M=estimates.sn.mayo[1], 
                          omega.M=estimates.sn.mayo[2], 
                          alpha.M=estimates.sn.mayo[3],
                          times.np= KM.mayo.obj$time, F.T.np=1 - KM.mayo.obj$surv,  
                          cens.status=pbc.mayo$vital.status)

print("Tau(Mayo marker)")
print(tau(gumbelCopula(exp(mle.sp.gumbel.mayo$estimate)+1, dim= 2)))

# 4-cov marker

#Marker margin
fit.marker.4cov <- selm(marker.4cov~1, family="SN", data=pbc.mayo)
summary(fit.marker.4cov, "DP", cov=TRUE, cor=TRUE)
estimates.sn.4cov <- coef(fit.marker.4cov, "DP")

#Time margin
KM.mayo.obj <- survfit(Surv(time, vital.status) ~  1, 
                       data=pbc.mayo)
#MLE estimation
mle.sp.gumbel.4cov <- nlm(ll.sp.gumbel, 0,
                          obs.marker=c(pbc.mayo$marker.4cov),
                          surv.times=pbc.mayo$time,
                          xi.M=estimates.sn.4cov[1], 
                          omega.M=estimates.sn.4cov[2], 
                          alpha.M=estimates.sn.4cov[3],
                          times.np= KM.mayo.obj$time, F.T.np=1 - KM.mayo.obj$surv,  
                          cens.status=pbc.mayo$vital.status)

print("Tau(4cov marker)")
print(tau(gumbelCopula(exp(mle.sp.gumbel.4cov$estimate)+1, dim= 2)))

### 3) Gaussian copula 

#### 3.1) Fully-parametric (fp) 

In [ ]:
%%R 
# Mayo marker:

#Initial values
var.ini.fp.mayo <- c(0, 6, log(2), 6, 0, 8)

#MLE estimation
mle.fp.gauss.mayo <- nlm(ll.fp.gauss, var.ini.fp.mayo, 
                         obs.marker=pbc.mayo$marker.mayo, surv.times=pbc.mayo$time, 
                         cens.status=pbc.mayo$vital.status,
                         hessian=T)

print("Tau(Mayo marker)")
print(tau(normalCopula(tanh(mle.fp.gauss.mayo$estimate[1]), dim= 2)))


# 4-cov marker

#Initial value
var.ini.fp.4cov <- c(0, 8.5, log(1.6), 4, 0, 8)

#MLE estimation
mle.fp.gauss.4cov <- nlm(ll.fp.gauss, var.ini.fp.4cov, 
                         obs.marker=pbc.mayo$marker.4cov, surv.times=pbc.mayo$time, 
                         cens.status=pbc.mayo$vital.status,
                         hessian=T)

print("Tau(4cov marker)")
print(tau(normalCopula(tanh(mle.fp.gauss.4cov$estimate[1]), dim= 2)))



#### 3.2) Semi-parametric (sp) 

In [ ]:
%%R 
# Mayo marker:

#Marker margin
fit.marker.mayo <- selm(marker.mayo~1, family="SN", data=pbc.mayo)
summary(fit.marker.mayo, "DP", cov=TRUE, cor=TRUE)
estimates.sn.mayo <- coef(fit.marker.mayo, "DP")

#Time margin
KM.mayo.obj <- survfit(Surv(time, vital.status) ~  1, 
                       data=pbc.mayo)

#MLE estimation
mle.sp.gauss.mayo <- nlm(ll.sp.gauss, 0,
                         obs.marker=c(pbc.mayo$marker.mayo),
                         surv.times=pbc.mayo$time,
                         xi.M=estimates.sn.mayo[1], 
                         omega.M=estimates.sn.mayo[2], 
                         alpha.M=estimates.sn.mayo[3],
                         times.np= KM.mayo.obj$time, F.T.np=1 - KM.mayo.obj$surv,  
                         cens.status=pbc.mayo$vital.status)

print("Tau(Mayo marker)")
print(tau(normalCopula(tanh(mle.sp.gauss.mayo$estimate), dim= 2)))

# 4-cov marker

#Marker margin
fit.marker.4cov <- selm(marker.4cov~1, family="SN", data=pbc.mayo)
summary(fit.marker.4cov, "DP", cov=TRUE, cor=TRUE)
estimates.sn.4cov <- coef(fit.marker.4cov, "DP")

#Time margin
KM.mayo.obj <- survfit(Surv(time, vital.status) ~  1, 
                       data=pbc.mayo)

#MLE estimation
mle.sp.gauss.4cov <- nlm(ll.sp.gauss, 0,
                         obs.marker=c(pbc.mayo$marker.4cov),
                         surv.times=pbc.mayo$time,
                         xi.M=estimates.sn.4cov[1], 
                         omega.M=estimates.sn.4cov[2], 
                         alpha.M=estimates.sn.4cov[3],
                         times.np= KM.mayo.obj$time, F.T.np=1 - KM.mayo.obj$surv,  
                         cens.status=pbc.mayo$vital.status)

print("Tau(4cov marker)")
tau(normalCopula(tanh(mle.sp.gauss.4cov$estimate), dim= 2))


### 4) Clayton copula 

#### 4.1) Fully-parametric (fp) 

In [ ]:
%%R 
# Mayo marker:

#Initial values
var.ini.clayton.fp.mayo <- c(0, 6, log(2), 6, 0, 8)

#MLE estimation
mle.fp.clayton.mayo <- nlm(ll.fp.clayton, var.ini.clayton.fp.mayo,
                           obs.marker=pbc.mayo$marker.mayo, surv.times=pbc.mayo$time, 
                           cens.status=pbc.mayo$vital.status,
                           hessian=T)

print("Tau(Mayo marker)")
print(tau(claytonCopula(exp(mle.fp.clayton.mayo$estimate[1]), dim= 2)))


# 4-cov marker

#Initial values
var.ini.clayton.fp.4cov <- c(0, 8.5, log(1.6), 4, 0, 8)

#MLE estimation
mle.fp.clayton.4cov <- nlm(ll.fp.clayton, var.ini.clayton.fp.4cov,
                           obs.marker=pbc.mayo$marker.4cov, surv.times=pbc.mayo$time, 
                           cens.status=pbc.mayo$vital.status,
                           hessian=T)

print("Tau(4cov marker)")
print(tau(claytonCopula(exp(mle.fp.clayton.4cov$estimate[1]), dim= 2)))


#### 4.2) Semi-parametric (sp) 

In [ ]:
%%R 
# Mayo marker:

#Marker margin
fit.marker.mayo <- selm(marker.mayo~1, family="SN", data=pbc.mayo)
summary(fit.marker.mayo, "DP", cov=TRUE, cor=TRUE)
estimates.sn.mayo <- coef(fit.marker.mayo, "DP")

#Time margin
KM.mayo.obj <- survfit(Surv(time, vital.status) ~  1, 
                       data=pbc.mayo)

#MLE estimation
mle.sp.clayton.mayo <- nlm(ll.sp.clayton, 0, 
                           obs.marker=c(pbc.mayo$marker.mayo),
                           surv.times=pbc.mayo$time,
                           xi.M=estimates.sn.mayo[1], 
                           omega.M=estimates.sn.mayo[2], 
                           alpha.M=estimates.sn.mayo[3],
                           times.np= KM.mayo.obj$time, F.T.np=1 - KM.mayo.obj$surv,  
                           cens.status=pbc.mayo$vital.status)
print("Tau(Mayo marker)")
print(tau(claytonCopula(exp(mle.sp.clayton.mayo$estimate), dim= 2)))

# 4-cov marker

#Marker margin
fit.marker.4cov <- selm(marker.4cov~1, family="SN", data=pbc.mayo)
summary(fit.marker.4cov, "DP", cov=TRUE, cor=TRUE)
estimates.sn.4cov <- coef(fit.marker.4cov, "DP")

#Time margin
KM.mayo.obj <- survfit(Surv(time, vital.status) ~  1, 
                       data=pbc.mayo)

#MLE estimation
mle.sp.clayton.4cov <- nlm(ll.sp.clayton, 0,
                           obs.marker=c(pbc.mayo$marker.4cov),
                           surv.times=pbc.mayo$time,
                           xi.M=estimates.sn.4cov[1], 
                           omega.M=estimates.sn.4cov[2], 
                           alpha.M=estimates.sn.4cov[3],
                           times.np= KM.mayo.obj$time, F.T.np=1 - KM.mayo.obj$surv,  
                           cens.status=pbc.mayo$vital.status)

print("Tau(4cov marker)")
print(tau(claytonCopula(exp(mle.sp.clayton.4cov$estimate), dim= 2)))


##  Residuals calculation

For each copula, the conditional residuals are computed. The fully parametric and semi-parametric approaches are plotted in the same graphic for comparison.

### 1) Frank copula

#### 1.1)Fully-parametric 

In [ ]:
%%R

# Residuals for T|M mayo marker
resid.T.frank.fp.mayo <- residuals.T.frank.fp(mle.fp.frank.mayo$estimate,
                                              obs.marker=pbc.mayo$marker.mayo, surv.times=pbc.mayo$time, 
                                              cens.status=pbc.mayo$vital.status)
# Residuals for T|M 4cov marker
resid.T.frank.fp.4cov <- residuals.T.frank.fp(mle.fp.frank.4cov$estimate,
                                              obs.marker=pbc.mayo$marker.4cov, surv.times=pbc.mayo$time, 
                                              cens.status=pbc.mayo$vital.status)

# Residuals for M|T mayo marker
resid.M.frank.fp.mayo <- residuals.M.frank.fp(mle.fp.frank.mayo$estimate,
                                              obs.marker=pbc.mayo$marker.mayo, surv.times=pbc.mayo$time, 
                                              cens.status=pbc.mayo$vital.status)

# Residuals for M|T 4cov marker
resid.M.frank.fp.4cov <- residuals.M.frank.fp(mle.fp.frank.4cov$estimate,
                                              obs.marker=pbc.mayo$marker.4cov, surv.times=pbc.mayo$time, 
                                              cens.status=pbc.mayo$vital.status)



#### 1.2)Semi-parametric 

In [ ]:
%%R

# Residuals for T|M mayo marker
resid.T.frank.sp.mayo <- residuals.T.frank.sp(mle.sp.frank.mayo$estimate,
                                              obs.marker=c(pbc.mayo$marker.mayo),
                                              surv.times=pbc.mayo$time,
                                              xi.M=estimates.sn.mayo[1], 
                                              omega.M=estimates.sn.mayo[2], 
                                              alpha.M=estimates.sn.mayo[3],
                                              times.np= KM.mayo.obj$time, F.T.np=1 - KM.mayo.obj$surv,  
                                              cens.status=pbc.mayo$vital.status)
# Residuals for T|M 4cov marker
resid.T.frank.sp.4cov <- residuals.T.frank.sp(mle.sp.frank.4cov$estimate,
                                              obs.marker=c(pbc.mayo$marker.4cov),
                                              surv.times=pbc.mayo$time,
                                              xi.M=estimates.sn.4cov[1], 
                                              omega.M=estimates.sn.4cov[2], 
                                              alpha.M=estimates.sn.4cov[3],
                                              times.np= KM.mayo.obj$time, F.T.np=1 - KM.mayo.obj$surv,  
                                              cens.status=pbc.mayo$vital.status)

# Residuals for M|T mayo marker
resid.M.frank.sp.mayo <- residuals.M.frank.sp(lim.inf=4.5,
                                              obs.marker=c(pbc.mayo$marker.mayo),
                                              surv.times=pbc.mayo$time,
                                              cens.status=pbc.mayo$vital.status,
                                              theta.par=mle.sp.frank.mayo$estimate,
                                              xi.M=estimates.sn.mayo[1], 
                                              omega.M=estimates.sn.mayo[2], 
                                              alpha.M=estimates.sn.mayo[3],
                                              times.np= KM.mayo.obj$time, F.T.np=1 - KM.mayo.obj$surv)

# Residuals for M|T 4cov marker
resid.M.frank.sp.4cov <- residuals.M.frank.sp(lim.inf=7.6,
                                              obs.marker=c(pbc.mayo$marker.4cov),
                                              surv.times=pbc.mayo$time,
                                              cens.status=pbc.mayo$vital.status,
                                              theta.par=mle.sp.frank.4cov$estimate,
                                              xi.M=estimates.sn.4cov[1], 
                                              omega.M=estimates.sn.4cov[2], 
                                              alpha.M=estimates.sn.4cov[3],
                                              times.np= KM.mayo.obj$time, F.T.np=1 - KM.mayo.obj$surv)


### 2) Gumbel copula

#### 2.1) Fully-parametric 

In [ ]:
%%R 
# Residuals for T|M mayo marker
resid.T.gumbel.fp.mayo <- residuals.T.gumbel.fp(mle.fp.gumbel.mayo$estimate,
                                                obs.marker=pbc.mayo$marker.mayo, surv.times=pbc.mayo$time, 
                                                cens.status=pbc.mayo$vital.status)
# Residuals for T|M 4cov marker
resid.T.gumbel.fp.4cov <- residuals.T.gumbel.fp(mle.fp.gumbel.4cov$estimate,
                                                obs.marker=pbc.mayo$marker.4cov, surv.times=pbc.mayo$time, 
                                                cens.status=pbc.mayo$vital.status)

# Residuals for M|T mayo marker
resid.M.gumbel.fp.mayo <- residuals.M.gumbel.fp(mle.fp.gumbel.mayo$estimate,
                                                obs.marker=pbc.mayo$marker.mayo, surv.times=pbc.mayo$time, 
                                                cens.status=pbc.mayo$vital.status)

# Residuals for M|T 4cov marker
resid.M.gumbel.fp.4cov <- residuals.M.gumbel.fp(mle.fp.gumbel.4cov$estimate,
                                                obs.marker=pbc.mayo$marker.4cov, surv.times=pbc.mayo$time, 
                                                cens.status=pbc.mayo$vital.status)


#### 2.2) Semi-parametric 

In [ ]:
%%R 

# Residuals for T|M mayo marker
resid.T.gumbel.sp.mayo <- residuals.T.gumbel.sp(mle.sp.gumbel.mayo$estimate,
                                                obs.marker=c(pbc.mayo$marker.mayo),
                                                surv.times=pbc.mayo$time,
                                                xi.M=estimates.sn.mayo[1], 
                                                omega.M=estimates.sn.mayo[2], 
                                                alpha.M=estimates.sn.mayo[3],
                                                times.np= KM.mayo.obj$time, F.T.np=1 - KM.mayo.obj$surv,  
                                                cens.status=pbc.mayo$vital.status)

# Residuals for T|M 4cov marker
resid.T.gumbel.sp.4cov <- residuals.T.gumbel.sp(mle.sp.gumbel.4cov$estimate,
                                                obs.marker=c(pbc.mayo$marker.4cov),
                                                surv.times=pbc.mayo$time,
                                                xi.M=estimates.sn.4cov[1], 
                                                omega.M=estimates.sn.4cov[2], 
                                                alpha.M=estimates.sn.4cov[3],
                                                times.np= KM.mayo.obj$time, F.T.np=1 - KM.mayo.obj$surv,  
                                                cens.status=pbc.mayo$vital.status)

# Residuals for M|T mayo marker
resid.M.gumbel.sp.mayo <- residuals.M.gumbel.sp(lim.inf=4.5,
                                                obs.marker=c(pbc.mayo$marker.mayo),
                                                surv.times=pbc.mayo$time,
                                                cens.status=pbc.mayo$vital.status,
                                                parametro=mle.sp.gumbel.mayo$estimate,
                                                xi.M=estimates.sn.mayo[1], 
                                                omega.M=estimates.sn.mayo[2], 
                                                alpha.M=estimates.sn.mayo[3],
                                                times.np= KM.mayo.obj$time, F.T.np=1 - KM.mayo.obj$surv)

# Residuals for M|T 4cov marker
resid.M.gumbel.sp.4cov <- residuals.M.gumbel.sp(lim.inf=7.6,
                                                obs.marker=c(pbc.mayo$marker.4cov),
                                                surv.times=pbc.mayo$time,
                                                cens.status=pbc.mayo$vital.status,
                                                parametro=mle.sp.gumbel.4cov$estimate,
                                                xi.M=estimates.sn.4cov[1], 
                                                omega.M=estimates.sn.4cov[2], 
                                                alpha.M=estimates.sn.4cov[3],
                                                times.np= KM.mayo.obj$time, F.T.np=1 - KM.mayo.obj$surv)



### 3) Gaussian copula

#### 3.1) Fully-parametric 

In [ ]:
%%R

# Residuals for T|M mayo marker
resid.T.gauss.fp.mayo <- residuals.T.gauss.fp(mle.fp.gauss.mayo$estimate, 
                                              obs.marker=pbc.mayo$marker.mayo, surv.times=pbc.mayo$time, 
                                              cens.status=pbc.mayo$vital.status)

# Residuals for T|M 4cov marker

resid.T.gauss.fp.4cov <- residuals.T.gauss.fp(mle.fp.gauss.4cov$estimate, 
                                              obs.marker=pbc.mayo$marker.4cov, surv.times=pbc.mayo$time, 
                                              cens.status=pbc.mayo$vital.status)

# Residuals for M|T mayo marker
resid.M.gauss.fp.mayo <- residuals.M.gauss.fp(mle.fp.gauss.mayo$estimate, 
                                              obs.marker=pbc.mayo$marker.mayo, surv.times=pbc.mayo$time, 
                                              cens.status=pbc.mayo$vital.status)

# Residuals for M|T 4cov marker
resid.M.gauss.fp.4cov <- residuals.M.gauss.fp(mle.fp.gauss.4cov$estimate, 
                                              obs.marker=pbc.mayo$marker.4cov, surv.times=pbc.mayo$time, 
                                              cens.status=pbc.mayo$vital.status)



#### 3.2) Semi-parametric 

In [ ]:
%%R
# Residuals for T|M mayo marker
resid.T.gauss.sp.mayo <- residuals.T.gauss.sp(mle.sp.gauss.mayo$estimate,
                                              obs.marker=c(pbc.mayo$marker.mayo),
                                              surv.times=pbc.mayo$time,
                                              xi.M=estimates.sn.mayo[1], 
                                              omega.M=estimates.sn.mayo[2], 
                                              alpha.M=estimates.sn.mayo[3],
                                              times.np= KM.mayo.obj$time, F.T.np=1 - KM.mayo.obj$surv,  
                                              cens.status=pbc.mayo$vital.status)

# Residuals for T|M 4cov marker
resid.T.gauss.sp.4cov <- residuals.T.gauss.sp(mle.sp.gauss.4cov$estimate,
                                              obs.marker=c(pbc.mayo$marker.4cov),
                                              surv.times=pbc.mayo$time,
                                              xi.M=estimates.sn.4cov[1], 
                                              omega.M=estimates.sn.4cov[2], 
                                              alpha.M=estimates.sn.4cov[3],
                                              times.np= KM.mayo.obj$time, F.T.np=1 - KM.mayo.obj$surv,  
                                              cens.status=pbc.mayo$vital.status)

# Residuals for M|T mayo marker
resid.M.gauss.sp.mayo <- residuals.M.gauss.sp(lim.inf=4.5,
                                              obs.marker=c(pbc.mayo$marker.mayo),
                                              surv.times=pbc.mayo$time,
                                              cens.status=pbc.mayo$vital.status,
                                              rho.C=tanh(mle.sp.gauss.mayo$estimate),
                                              xi.M=estimates.sn.mayo[1], 
                                              omega.M=estimates.sn.mayo[2], 
                                              alpha.M=estimates.sn.mayo[3],
                                              times.np= KM.mayo.obj$time, F.T.np=1 - KM.mayo.obj$surv)
# Residuals for M|T 4cov marker
resid.M.gauss.sp.4cov <- residuals.M.gauss.sp(lim.inf=7.6,
                                              obs.marker=c(pbc.mayo$marker.4cov),
                                              surv.times=pbc.mayo$time,
                                              cens.status=pbc.mayo$vital.status,
                                              rho.C=tanh(mle.sp.gauss.4cov$estimate),
                                              xi.M=estimates.sn.4cov[1], 
                                              omega.M=estimates.sn.4cov[2], 
                                              alpha.M=estimates.sn.4cov[3],
                                              times.np= KM.mayo.obj$time, F.T.np=1 - KM.mayo.obj$surv)


### 4) Clayton copula

#### 4.1) Fully-parametric 

In [ ]:
%%R
# Residuals for T|M mayo marker
resid.T.clayton.fp.mayo <- residuals.T.clayton.fp(mle.fp.clayton.mayo$estimate,
                                                  obs.marker=pbc.mayo$marker.mayo, surv.times=pbc.mayo$time, 
                                                  cens.status=pbc.mayo$vital.status)
# Residuals for T|M 4cov marker
resid.T.clayton.fp.4cov <- residuals.T.clayton.fp(mle.fp.clayton.4cov$estimate,
                                                  obs.marker=pbc.mayo$marker.4cov, surv.times=pbc.mayo$time, 
                                                  cens.status=pbc.mayo$vital.status)
# Residuals for M|T mayo marker
resid.M.clayton.fp.mayo <- residuals.M.clayton.fp(mle.fp.clayton.mayo$estimate,
                                                  obs.marker=pbc.mayo$marker.mayo, surv.times=pbc.mayo$time, 
                                                  cens.status=pbc.mayo$vital.status)

# Residuals for M|T 4cov marker
resid.M.clayton.fp.4cov <- residuals.M.clayton.fp(mle.fp.clayton.4cov$estimate,
                                                  obs.marker=pbc.mayo$marker.4cov, surv.times=pbc.mayo$time, 
                                                  cens.status=pbc.mayo$vital.status)


#### 4.2) Semi-parametric 

In [ ]:
%%R

# Residuals for T|M mayo marker
resid.T.clayton.sp.mayo <- residuals.T.clayton.sp(mle.sp.clayton.mayo$estimate,
                                                  obs.marker=c(pbc.mayo$marker.mayo),
                                                  surv.times=pbc.mayo$time,
                                                  xi.M=estimates.sn.mayo[1], 
                                                  omega.M=estimates.sn.mayo[2], 
                                                  alpha.M=estimates.sn.mayo[3],
                                                  times.np= KM.mayo.obj$time, F.T.np=1 - KM.mayo.obj$surv,  
                                                  cens.status=pbc.mayo$vital.status)


# Residuals for T|M 4cov marker
resid.T.clayton.sp.4cov <- residuals.T.clayton.sp(mle.sp.clayton.4cov$estimate,
                                                  obs.marker=c(pbc.mayo$marker.4cov),
                                                  surv.times=pbc.mayo$time,
                                                  xi.M=estimates.sn.4cov[1], 
                                                  omega.M=estimates.sn.4cov[2], 
                                                  alpha.M=estimates.sn.4cov[3],
                                                  times.np= KM.mayo.obj$time, F.T.np=1 - KM.mayo.obj$surv,  
                                                  cens.status=pbc.mayo$vital.status)

# Residuals for M|T mayo marker
resid.M.clayton.sp.mayo <- residuals.M.clayton.sp(lim.inf=4.5,
                                                  obs.marker=c(pbc.mayo$marker.mayo),
                                                  surv.times=pbc.mayo$time,
                                                  cens.status=pbc.mayo$vital.status,
                                                  parametro=mle.sp.clayton.mayo$estimate,
                                                  xi.M=estimates.sn.mayo[1], 
                                                  omega.M=estimates.sn.mayo[2], 
                                                  alpha.M=estimates.sn.mayo[3],
                                                  times.np= KM.mayo.obj$time, F.T.np=1 - KM.mayo.obj$surv)
# Residuals for M|T 4cov marker

resid.M.clayton.sp.4cov <- residuals.M.clayton.sp(lim.inf=7.6,
                                                  obs.marker=c(pbc.mayo$marker.4cov),
                                                  surv.times=pbc.mayo$time,
                                                  cens.status=pbc.mayo$vital.status,
                                                  parametro=mle.sp.clayton.4cov$estimate,
                                                  xi.M=estimates.sn.4cov[1], 
                                                  omega.M=estimates.sn.4cov[2], 
                                                  alpha.M=estimates.sn.4cov[3],
                                                  times.np= KM.mayo.obj$time, F.T.np=1 - KM.mayo.obj$surv)




### Dataframes for residuals plots

In [ ]:
%%R

#Data frame for T|M residuals for mayo marker
residuals.T.mayo <- data.frame(x=c(resid.T.gauss.fp.mayo[,1],
                                   resid.T.frank.fp.mayo[,1],
                                   resid.T.gumbel.fp.mayo[,1],
                                   resid.T.clayton.fp.mayo[,1],
                                   resid.T.gauss.sp.mayo[,1],
                                   resid.T.frank.sp.mayo[,1],
                                   resid.T.gumbel.sp.mayo[,1],
                                   resid.T.clayton.sp.mayo[,1]),
                               y=c(resid.T.gauss.fp.mayo[,2],
                                   resid.T.frank.fp.mayo[,2],
                                   resid.T.gumbel.fp.mayo[,2],
                                   resid.T.clayton.fp.mayo[,2],
                                   resid.T.gauss.sp.mayo[,2],
                                   resid.T.frank.sp.mayo[,2],
                                   resid.T.gumbel.sp.mayo[,2],
                                   resid.T.clayton.sp.mayo[,2]),
                               copula=factor(c(rep("Gaussian", length(resid.T.gauss.fp.mayo[,1])),
                                               rep("Frank", length(resid.T.frank.fp.mayo[,1])),
                                               rep("Gumbel", length(resid.T.gumbel.fp.mayo[,1])),
                                               rep("Clayton", length(resid.T.clayton.fp.mayo[,1])),
                                               rep("Gaussian", length(resid.T.gauss.fp.mayo[,1])),
                                               rep("Frank", length(resid.T.frank.fp.mayo[,1])),
                                               rep("Gumbel", length(resid.T.gumbel.fp.mayo[,1])),
                                               rep("Clayton", length(resid.T.clayton.fp.mayo[,1])))),
                               model=factor(c(rep("Parametric", length(resid.T.gauss.fp.mayo[,1])+
                                                     length(resid.T.frank.fp.mayo[,1])+
                                                     length(resid.T.gumbel.fp.mayo[,1])+
                                                     length(resid.T.clayton.fp.mayo[,1])),
                                              rep("Semi-parametric", length(resid.T.gauss.fp.mayo[,1])+
                                                     length(resid.T.frank.fp.mayo[,1])+
                                                     length(resid.T.gumbel.fp.mayo[,1])+
                                                     length(resid.T.clayton.fp.mayo[,1]))), 
                                            levels=c("Semi-parametric", "Parametric")))

#Data frame for M|T residuals for mayo marker
residuals.M.mayo <- data.frame(q=c(resid.M.gauss.fp.mayo,
                                   resid.M.frank.fp.mayo,
                                   resid.M.gumbel.fp.mayo,
                                   resid.M.clayton.fp.mayo,
                                   resid.M.gauss.sp.mayo,
                                   resid.M.frank.sp.mayo,
                                   resid.M.gumbel.sp.mayo,
                                   resid.M.clayton.sp.mayo),
                               copula=factor(c(rep("Gaussian", length(resid.M.gauss.fp.mayo)),
                                               rep("Frank", length(resid.M.frank.fp.mayo)),
                                               rep("Gumbel", length(resid.M.gumbel.fp.mayo)),
                                               rep("Clayton", length(resid.M.clayton.fp.mayo)),
                                               rep("Gaussian", length(resid.M.gauss.fp.mayo)),
                                               rep("Frank", length(resid.M.frank.fp.mayo)),
                                               rep("Gumbel", length(resid.M.gumbel.fp.mayo)),
                                               rep("Clayton", length(resid.M.clayton.fp.mayo)))),
                               model=factor(c(rep("Parametric", length(resid.M.gauss.fp.mayo)+
                                                     length(resid.M.frank.fp.mayo)+
                                                     length(resid.M.gumbel.fp.mayo)+
                                                     length(resid.M.clayton.fp.mayo)),
                                              rep("Semi-parametric", length(resid.M.gauss.fp.mayo)+
                                                     length(resid.M.frank.fp.mayo)+
                                                     length(resid.M.gumbel.fp.mayo)+
                                                     length(resid.M.clayton.fp.mayo))), 
                                            levels=c("Semi-parametric", "Parametric")))


#Data frame for T|M residuals for 4cov marker
residuals.T.4cov <- data.frame(x=c(resid.T.gauss.fp.4cov[,1],
                                   resid.T.frank.fp.4cov[,1],
                                   resid.T.gumbel.fp.4cov[,1],
                                   resid.T.clayton.fp.4cov[,1],
                                   resid.T.gauss.sp.4cov[,1],
                                   resid.T.frank.sp.4cov[,1],
                                   resid.T.gumbel.sp.4cov[,1],
                                   resid.T.clayton.sp.4cov[,1]),
                               y=c(resid.T.gauss.fp.4cov[,2],
                                   resid.T.frank.fp.4cov[,2],
                                   resid.T.gumbel.fp.4cov[,2],
                                   resid.T.clayton.fp.4cov[,2],
                                   resid.T.gauss.sp.4cov[,2],
                                   resid.T.frank.sp.4cov[,2],
                                   resid.T.gumbel.sp.4cov[,2],
                                   resid.T.clayton.sp.4cov[,2]),
                               copula=factor(c(rep("Gaussian", length(resid.T.gauss.fp.4cov[,1])),
                                               rep("Frank", length(resid.T.frank.fp.4cov[,1])),
                                               rep("Gumbel", length(resid.T.gumbel.fp.4cov[,1])),
                                               rep("Clayton", length(resid.T.clayton.fp.4cov[,1])),
                                               rep("Gaussian", length(resid.T.gauss.fp.4cov[,1])),
                                               rep("Frank", length(resid.T.frank.fp.4cov[,1])),
                                               rep("Gumbel", length(resid.T.gumbel.fp.4cov[,1])),
                                               rep("Clayton", length(resid.T.clayton.fp.4cov[,1])))),
                               model=factor(c(rep("Parametric", length(resid.T.gauss.fp.4cov[,1])+
                                                     length(resid.T.frank.fp.4cov[,1])+
                                                     length(resid.T.gumbel.fp.4cov[,1])+
                                                     length(resid.T.clayton.fp.4cov[,1])),
                                              rep("Semi-parametric", length(resid.T.gauss.fp.4cov[,1])+
                                                     length(resid.T.frank.fp.4cov[,1])+
                                                     length(resid.T.gumbel.fp.4cov[,1])+
                                                     length(resid.T.clayton.fp.4cov[,1]))), 
                                            levels=c("Semi-parametric", "Parametric")))

#Data frame for M|T residuals for 4cov marker
residuals.M.4cov <- data.frame(q=c(resid.M.gauss.fp.4cov,
                                   resid.M.frank.fp.4cov,
                                   resid.M.gumbel.fp.4cov,
                                   resid.M.clayton.fp.4cov,
                                   resid.M.gauss.sp.4cov,
                                   resid.M.frank.sp.4cov,
                                   resid.M.gumbel.sp.4cov,
                                   resid.M.clayton.sp.4cov),
                               copula=factor(c(rep("Gaussian", length(resid.M.gauss.fp.4cov)),
                                               rep("Frank", length(resid.M.frank.fp.4cov)),
                                               rep("Gumbel", length(resid.M.gumbel.fp.4cov)),
                                               rep("Clayton", length(resid.M.clayton.fp.4cov)),
                                               rep("Gaussian", length(resid.M.gauss.fp.4cov)),
                                               rep("Frank", length(resid.M.frank.fp.4cov)),
                                               rep("Gumbel", length(resid.M.gumbel.fp.4cov)),
                                               rep("Clayton", length(resid.M.clayton.fp.4cov)))),
                               model=factor(c(rep("Parametric", length(resid.M.gauss.fp.4cov)+
                                                     length(resid.M.frank.fp.4cov)+
                                                     length(resid.M.gumbel.fp.4cov)+
                                                     length(resid.M.clayton.fp.4cov)),
                                              rep("Semi-parametric", length(resid.M.gauss.fp.4cov)+
                                                     length(resid.M.frank.fp.4cov)+
                                                     length(resid.M.gumbel.fp.4cov)+
                                                     length(resid.M.clayton.fp.4cov))), 
                                            levels=c("Semi-parametric", "Parametric")))                               


### Residuals plots

In [ ]:
%%R

# Mayo Marker
plot.mayo.T <- xyplot(y~x|copula*model, data=residuals.T.mayo,
                      panel = function(x, y) {
                         panel.xyplot(x, y, pch=19, col=gray(0.35))
                         panel.abline(0,1, lwd=2)},
                      par.settings = list(strip.background=list(col=gray(0.95))), 
                      main="Residuals for T|M corresponding to the mayo marker",
                      xlab="log(y)", ylab="log{-log[S(y)]}")

plot.mayo.M <- qqmath(~ q|copula*model, data=residuals.M.mayo, 
                      panel = function(x, y) {
                         panel.qqmath(x, pch=19, col=gray(0.35))
                         panel.abline(0,1, lwd=2)},
                      par.settings = list(strip.background=list(col=gray(0.95))), 
                      main="Residuals for M|T corresponding to the mayo marker",
                      xlab = "Theoretical Quantiles", ylab = "Sample Quantiles")

# 4cov Marker
plot.4cov.T <- xyplot(y~x|copula*model, data=residuals.T.4cov,
                      panel = function(x, y) {
                         panel.xyplot(x, y, pch=19, col=gray(0.35))
                         panel.abline(0,1, lwd=2)},
                      par.settings = list(strip.background=list(col=gray(0.95))), 
                      main="Residuals for T|M corresponding to the 4cov marker",
                      xlab="log(y)", ylab="log{-log[S(y)]}")

plot.4cov.M <- qqmath(~ q|copula*model, data=residuals.M.4cov, 
                      panel = function(x, y) {
                         panel.qqmath(x, pch=19, col=gray(0.35))
                         panel.abline(0,1, lwd=2)},
                      par.settings = list(strip.background=list(col=gray(0.95))), 
                      main="Residuals for M|T corresponding to the 4cov marker",
                      xlab = "Theoretical Quantiles", ylab = "Sample Quantiles")

#Combining residuals plots
grid.arrange(plot.4cov.T, plot.4cov.M, plot.mayo.T, plot.mayo.M, ncol=2)


# Section 4

## a) 4cov marker analysis 

### Fully-parametric (fp) model fitting

In [ ]:
%%R 
#Initial values for optimization
var.ini.frank.fp.4cov <- c(5, 8.5, log(1.6), 4, 0, 8)

#MLE estimation
mle.fp.frank.4cov <- nlm(ll.fp.frank, var.ini.frank.fp.4cov,
                         obs.marker=pbc.mayo$marker.4cov, surv.times=pbc.mayo$time, 
                         cens.status=pbc.mayo$vital.status,
                         hessian=T)
print("Kendall's Tau")
print(tau(frankCopula(mle.fp.frank.4cov$estimate[1], dim= 2)))

#Wald statistics
print("Wald statistics")
Z0.frank.fp.4cov <- mle.fp.frank.4cov$estimate/sqrt(diag(solve(mle.fp.frank.4cov$hessian)))
print(Z0.frank.fp.4cov)
print("P-values")
print(round(2*pnorm(abs(Z0.frank.fp.4cov), lower.tail = FALSE),4))

#Exponential distribution for surivival time

#Initial values for optimation
var.ini.frank.fp.4cov.exp <- c(5, 8.5, log(1.6), 4, 8)

#MLE estimation
mle.fp.frank.4cov.exp <- nlm(ll.fp.frank.exp, var.ini.frank.fp.4cov.exp,
                             obs.marker=pbc.mayo$marker.4cov, surv.times=pbc.mayo$time, 
                             cens.status=pbc.mayo$vital.status,
                             hessian=T)

#Kendall's Tau
print("Kendall's Tau")
print(tau(frankCopula(mle.fp.frank.4cov.exp$estimate[1], dim= 2)))

#Wald test
print("Wald statistics")
Z0.frank.fp.4cov.exp <- mle.fp.frank.4cov.exp$estimate/sqrt(diag(solve(mle.fp.frank.4cov.exp$hessian)))
print(Z0.frank.fp.4cov.exp)
print("P-values")
print(round(2*pnorm(abs(Z0.frank.fp.4cov.exp), lower.tail = FALSE),4))


#Estimator parameters

print("theta")
print(round(mle.fp.frank.4cov.exp$estimate[1],3))
print(round(sqrt(diag(solve(mle.fp.frank.4cov.exp$hessian)))[1],3)) #SE

print("omega1")
print(round(mle.fp.frank.4cov.exp$estimate[2],3))
print(round(sqrt(diag(solve(mle.fp.frank.4cov.exp$hessian)))[2],3)) #SE

print("log(omega2)")
print(round(mle.fp.frank.4cov.exp$estimate[3],3))
print(round(sqrt(diag(solve(mle.fp.frank.4cov.exp$hessian)))[3],3)) #SE

print("omega3")
print(round(mle.fp.frank.4cov.exp$estimate[4],3))
print(round(sqrt(diag(solve(mle.fp.frank.4cov.exp$hessian)))[4],3)) #SE

print("log(lambda2)")
print(round(mle.fp.frank.4cov.exp$estimate[5],3))
print(round(sqrt(diag(solve(mle.fp.frank.4cov.exp$hessian)))[5],3)) #SE


# Bootstrap for confidence intervals:

sample.size <- 312
estimates.boot.fp.4cov <- NULL
system.time(
for (i in 1:200){
   estimated.C.fp.4cov <- mvdc(frankCopula(mle.fp.frank.4cov.exp$estimate[1],
                                           dim= 2), 
                               c("unif", "unif"), 
                               list(list(min = 0, max = 1), 
                                    list(min = 0, max = 1)), 
                               marginsIdentical = T)
   sim.U.C.par <- rMvdc(sample.size, estimated.C.fp.4cov)
   #
   marker.par.boot <- qsn(sim.U.C.par[,1], 
                          xi=mle.fp.frank.4cov.exp$estimate[2], 
                          omega=exp(mle.fp.frank.4cov.exp$estimate[3]), 
                          alpha=mle.fp.frank.4cov.exp$estimate[4])
   cuantil.t.par <- 1-sim.U.C.par[,2]
   t.par <- qweibull(cuantil.t.par, 
                     shape = exp(0), 
                     scale = exp(mle.fp.frank.4cov.exp$estimate[5]))
  
   cuantil.c <- runif(sample.size)
   C.dist <- survfit(Surv(time-0.001*vital.status, 1-vital.status) ~ 1, 
                     data = pbc.mayo)
   c.km <- quantile(C.dist, probs = cuantil.c, conf.int = F)
   t.and.c.par <- matrix(c(t.par, c.km), ncol=2, byrow=F)
   t.star.par <- apply(t.and.c.par, 1, min, na.rm = TRUE)
   substract.sim.par <- t.and.c.par-t.star.par
   cens.in.sim.par <- rep(0, nrow(substract.sim.par))
   cens.in.sim.par[substract.sim.par[,1] == 0] <- 1
   
   mle.sim.parametric.boot <- nlm(ll.fp.frank.exp, 
                                  var.ini.frank.fp.4cov.exp,
                                  obs.marker=marker.par.boot,
                                  surv.times=t.star.par,
                                  cens.status=cens.in.sim.par,
                                  hessian=F)
   estimates.boot.fp.4cov <- rbind(estimates.boot.fp.4cov,
                                   c(mle.sim.parametric.boot$estimate))
}
)

#Boostrap standard errors
se.4cov.exp <- apply(estimates.boot.fp.4cov, 2, sd)
print("Boostrap SE:")
print(se.4cov.exp)

      
#Confidence interval for theta
thetas.boot.4cov <- estimates.boot.fp.4cov[,1]
quantile.thetas.boot.4cov <- quantile(thetas.boot.4cov, probs=c(0.025,0.975))
CI.thetas.4cov.par <- 2*mle.fp.frank.4cov.exp$estimate[1]- quantile.thetas.boot.4cov


#Confidence interval for kendall's tau
print("Boostrap CI for kendall's tau")
print(c(tau(frankCopula(CI.thetas.4cov.par[1], dim= 2)),tau(frankCopula(CI.thetas.4cov.par[2], dim= 2))))
   

### Semi-parametric (sp) model fitting

In [ ]:
%%R

#Marker margin
fit.marker.4cov <- selm(marker.4cov~1, family="SN", data=pbc.mayo)
estimates.sn.4cov <- coef(fit.marker.4cov, "DP")
summary.fit.marker.4cov <- summary(fit.marker.4cov, "DP", cov=TRUE, cor=TRUE)

#Time margin
KM.mayo.obj <- survfit(Surv(time, vital.status) ~  1, 
                       data=pbc.mayo)

#MLE Estimation
mle.sp.frank.4cov <- nlm(ll.sp.frank, 6, 
                         obs.marker=c(pbc.mayo$marker.4cov),
                         surv.times=pbc.mayo$time,
                         xi.M=estimates.sn.4cov[1], 
                         omega.M=estimates.sn.4cov[2], 
                         alpha.M=estimates.sn.4cov[3],
                         times.np= KM.mayo.obj$time, F.T.np=1 - KM.mayo.obj$surv,  
                         cens.status=pbc.mayo$vital.status, hessian = TRUE)

#Kendall's Tau
print("Kendall's Tau")
print(tau(frankCopula(mle.sp.frank.4cov$estimate, dim= 2)))


# Bootstrap for confidence intervals
sample.size <- 312

estimates.boot.sp.4cov.w.dependence <- NULL
for (i in 1:200){
   estimated.C.sp.4cov.w.dependence <- mvdc(frankCopula(mle.sp.frank.4cov$estimate, 
                                                        dim= 2), 
                                            c("unif", "unif"), 
                                            list(list(min = 0, max = 1), 
                                                 list(min = 0, max = 1)), 
                                            marginsIdentical = T)
   sim.U.C <- rMvdc(sample.size, estimated.C.sp.4cov.w.dependence)
   simulated.marker.boot <- qsn(sim.U.C[,1], 
                                xi=estimates.sn.4cov[1], 
                                omega=estimates.sn.4cov[2], 
                                alpha=estimates.sn.4cov[3])
   fit.sn.sim.boot <- selm(marker~1, family="SN", 
                           data=data.frame(marker=simulated.marker.boot))
   estimates.sn.sim.boot <- coef(fit.sn.sim.boot, "DP")
   cuantil.t <- 1-sim.U.C[,2]
   t.km <- quantile(KM.mayo.obj, probs = cuantil.t, conf.int = F)
   cuantil.c <- runif(sample.size)
   C.sim <- survfit(Surv(time-0.001*vital.status, 1-vital.status) ~ 1, 
                    data = pbc.mayo)
   c.km <- quantile(C.sim, probs = cuantil.c, conf.int = F)
   t.and.c <- matrix(c(t.km, c.km), ncol=2, byrow=F)
   t.star <- apply(t.and.c, 1, min, na.rm = TRUE)
   substract.sim <- t.and.c-t.star
   cens.in.sim <- rep(0, nrow(substract.sim))
   cens.in.sim[substract.sim[,1] == 0] <- 1
   #
   sim.set.boot <- data.frame(ztimes=t.star, status=cens.in.sim)
   KM.sim.boot <- survfit(Surv(ztimes, status) ~ 1, data = sim.set.boot)
   #
   mle.sim.boot <- nlm(ll.sp.frank, 6, 
                       obs.marker=simulated.marker.boot,
                       surv.times=sim.set.boot$ztimes,
                       xi.M=estimates.sn.sim.boot[1], 
                       omega.M=estimates.sn.sim.boot[2], 
                       alpha.M=estimates.sn.sim.boot[3],
                       times.np= KM.sim.boot$time, F.T.np=1 - KM.sim.boot$surv,  
                       cens.status=sim.set.boot$status,
                       hessian=F)
   cat(mle.sim.boot$estimate, "\n")
   #
   estimates.boot.sp.4cov.w.dependence <- rbind(estimates.boot.sp.4cov.w.dependence,
                                                c(mle.sim.boot$estimate, 
                                                  estimates.sn.sim.boot[1], 
                                                  log(estimates.sn.sim.boot[2]), 
                                                  estimates.sn.sim.boot[3]))
}

# Bootstrap standard errors
se.sp.4cov <-apply(estimates.boot.sp.4cov.w.dependence, 2, sd)

#Estimated parameters

print("omega1")
print(summary.fit.marker.4cov@param.table[1,1]) #Estimator
print(se.sp.4cov[2]) #SE


print("log(omega2)")
print(log(summary.fit.marker.4cov@param.table[2,1])) #Estimator
print(se.sp.4cov[3]) #SE

print("omega3")
print(summary.fit.marker.4cov@param.table[3,1]) #Estimator
print(se.sp.4cov[4]) #SE

#Confidence interval for theta
thetas.boot.4cov.sp <- estimates.boot.sp.4cov.w.dependence[,1]
quantile.thetas.boot.4cov.sp <- quantile(thetas.boot.4cov.sp, probs=c(0.025,0.975))
CI.thetas.4cov.par.sp <- 2*mle.sp.frank.4cov$estimate -  quantile.thetas.boot.4cov.sp

#Confidence interval for tau
print("CI for kendall's tau")
print(c(tau(frankCopula(CI.thetas.4cov.par.sp[1], dim= 2)), tau(frankCopula(CI.thetas.4cov.par.sp[2], dim= 2))))



### Fully-parametric Cumulative/Dynamic AUC 

In [ ]:
%%R

#Vectors for time and mayo marker
time.marginal <- seq(365, 4380, length.out = 30)
mayo.marginal <- seq(5.4, 13.1, length.out = 225)
xy.mayo <- list(time=time.marginal, mayo=mayo.marginal)
grid.mayo <- expand.grid(xy.mayo)

# Point estimate
auc.CD.fp.mayo <- 1:30
tp.cd.fp.mayo <- NULL
fp.cd.fp.mayo <- NULL
for(i in 1:30){
   roc.t.i <- ROC.CD.fp.gumbel(m.M=mayo.marginal, t.T=time.marginal[i], 
                               theta.par=2, xi.M=mle.fp.gumbel.tau0.5.mayo$estimate[1], 
                               omega.M=exp(mle.fp.gumbel.tau0.5.mayo$estimate[2]), 
                               alpha.M=mle.fp.gumbel.tau0.5.mayo$estimate[3], 
                               shape.T=exp(mle.fp.gumbel.tau0.5.mayo$estimate[4]), 
                               scale.T=exp(mle.fp.gumbel.tau0.5.mayo$estimate[5]))
   auc.CD.fp.mayo[i] <- auc(roc.t.i$FP.D, roc.t.i$TP.C)
   cat("time=", time.marginal[i], "	AUC=", auc.CD.fp.mayo[i], "\n")
   tp.cd.fp.mayo <- c(tp.cd.fp.mayo, roc.t.i$TP.C)
   fp.cd.fp.mayo <- c(fp.cd.fp.mayo, roc.t.i$FP.D)
}


# Bootstrap for CD AUC 
boot.auc.CD.fp.mayo <- matrix(1:(200*30), ncol=30)

for(i in 1:200){
   lim.mark.boot <- qsn(c(0.001,0.999), 
                        xi=estimates.boot.fp.mayo[i,1], 
                        omega=exp(estimates.boot.fp.mayo[i,2]), 
                        alpha=estimates.boot.fp.mayo[i,3])
   for(j in 1:30){
      roc.t.i <- ROC.CD.fp.gumbel(m.M=seq(from=lim.mark.boot[1], 
                                          to=lim.mark.boot[2], length=50), 
                                  t.T=time.marginal[j], 
                                  theta.par=2, xi.M=estimates.boot.fp.mayo[i,1], 
                                  omega.M=exp(estimates.boot.fp.mayo[i,2]), 
                                  alpha.M=estimates.boot.fp.mayo[i,3], 
                                  shape.T=exp(estimates.boot.fp.mayo[i,4]), 
                                  scale.T=exp(estimates.boot.fp.mayo[i,5]))
      boot.auc.CD.fp.mayo[i,j] <- auc(roc.t.i$FP.D, roc.t.i$TP.C)
   }
}

#Boostrap quantiles
quantile.boot.auc.CD.fp.mayo <- apply(boot.auc.CD.fp.mayo, 
                                      2, quantile, probs=c(0.025,0.975))
#Inferior and superior limits
sup.auc.CD.fp.mayo <- 2*auc.CD.fp.mayo - quantile.boot.auc.CD.fp.mayo[1,]
inf.auc.CD.fp.mayo <- 2*auc.CD.fp.mayo - quantile.boot.auc.CD.fp.mayo[2,]

#AUC plot
plot(time.marginal, auc.CD.fp.mayo, type="l", ylim=c(0.7,1), lwd=2)
lines(time.marginal, inf.auc.CD.fp.mayo, col="gray", lwd=2)
lines(time.marginal, sup.auc.CD.fp.mayo, col="gray", lwd=2)

### Fully-parametric Cumulative/Dynamic ROC curves

In [ ]:
%%R 
#Point estimate
q.roc.plot <- seq(from=0.01, to=0.99, length=23)

roc.CD.fp.4cov.3plots <- matrix(1:(23*3), ncol=3)
actual.roc.CD.fp.4cov.3plots <- matrix(1:(227*3), ncol=3)
aqtual.q.roc.CD.fp.4cov.3plots <- matrix(1:(227*3), ncol=3)
par(mfrow=c(1,3))

times.3 <- c(365,4*365,6*365)
for(i in 1:3){
   roc.CD.fp.4cov <- ROC.CD.fp.frank(m.M=m4cov.marginal, t.T=times.3[i], 
                                     theta.par=mle.fp.frank.4cov.exp$estimate[1],
                                     xi.M=mle.fp.frank.4cov.exp$estimate[2], 
                                     omega.M=exp(mle.fp.frank.4cov.exp$estimate[3]), 
                                     alpha.M=mle.fp.frank.4cov.exp$estimate[4], 
                                     shape.T=exp(0), 
                                     scale.T=exp(mle.fp.frank.4cov.exp$estimate[5]))
   cat(auc(roc.CD.fp.4cov$FP.D, roc.CD.fp.4cov$TP.C),"\n")
   aqtual.q.roc.CD.fp.4cov.3plots[,i] <- roc.CD.fp.mayo$FP.D
   actual.roc.CD.fp.4cov.3plots[,i] <- roc.CD.fp.mayo$TP.C
   roc.approx <- approx(roc.CD.fp.4cov$FP.D, roc.CD.fp.4cov$TP.C,
                        xout = q.roc.plot)
   lines(roc.approx$x, roc.approx$y, lwd=1.5)
   roc.CD.fp.4cov.3plots[,i] <- roc.approx$y
}

# Bootstrap for confidence bounds
boot.ROC.CD.fp.4cov <- array(1:(200*3*23), dim=c(200, 3, 23))
times.3 <- c(365,4*365,6*365)

for(i in 1:200){
   lim.mark.boot <- qsn(c(0.001,0.999), 
                        xi=estimates.boot.fp.4cov[i,2], 
                        omega=exp(estimates.boot.fp.4cov[i,3]), 
                        alpha=estimates.boot.fp.4cov[i,4])
   for(j in 1:3){
      roc.CD.fp.4cov.boot <- ROC.CD.fp.frank(m.M=seq(from=lim.mark.boot[1], 
                                                     to=lim.mark.boot[2], length=225), 
                                             t.T=times.3[j],
                                             theta.par=estimates.boot.fp.4cov[i,1],
                                             xi.M=estimates.boot.fp.4cov[i,2], 
                                             omega.M=exp(estimates.boot.fp.4cov[i,3]), 
                                             alpha.M=estimates.boot.fp.4cov[i,4], 
                                             shape.T=exp(0), 
                                             scale.T=exp(estimates.boot.fp.4cov[i,5]))
      roc.approx <- approx(roc.CD.fp.4cov.boot$FP.D, roc.CD.fp.4cov.boot$TP.C,
                           xout = q.roc.plot)
      boot.ROC.CD.fp.4cov[i,j,] <- roc.approx$y
   }}

#Boostrap quantiles
q.boot.ROC.CD.fp.4cov <- apply(boot.ROC.CD.fp.4cov, 
                               2:3, quantile, probs=c(0.025,0.975))

#Inferior and superior limits
roc.sup.CD.fp.4cov.3plots <- matrix(1:(23*3), ncol=3)
roc.inf.CD.fp.4cov.3plots <- matrix(1:(23*3), ncol=3)

#ROC curves plots
par(mfrow=c(1,3))
for(i in 1:3){
   plot(q.roc.plot, roc.CD.fp.4cov.3plots[,i], 
        type="l", lwd=2, ylim=c(0,1))
   roc.sup.CD.fp.4cov.3plots[,i] <- 2*roc.CD.fp.4cov.3plots[,i] - 
      q.boot.ROC.CD.fp.4cov[1,i,]
   roc.inf.CD.fp.4cov.3plots[,i] <- 2*roc.CD.fp.4cov.3plots[,i] - 
      q.boot.ROC.CD.fp.4cov[2,i,]
   lines(c(0,q.roc.plot,1), c(0,roc.sup.CD.fp.4cov.3plots[,i],1), 
         lwd=2, col="gray")
   lines(c(0,q.roc.plot,1), c(0,roc.inf.CD.fp.4cov.3plots[,i],1), 
         lwd=2, col="gray")
   lines(c(0,q.roc.plot,1), c(0,roc.CD.fp.4cov.3plots[,i],1), lwd=2)
}


### Fully-parametric Incident/Dynamic AUC 

In [ ]:
%%R 

#Point estimate
auc.ID.fp.4cov <- 1:30
for(i in 1:30){
   roc.t.i <- ROC.ID.fp.frank(m.M=m4cov.marginal, t.T=time.marginal[i],
                              theta.par=mle.fp.frank.4cov.exp$estimate[1],
                              xi.M    <- mle.fp.frank.4cov.exp$estimate[2],
                              omega.M <- exp(mle.fp.frank.4cov.exp$estimate[3]),
                              alpha.M <- mle.fp.frank.4cov.exp$estimate[4],
                              shape.T <- exp(0),
                              scale.T <- exp(mle.fp.frank.4cov.exp$estimate[5]))
   auc.ID.fp.4cov[i] <- auc(roc.t.i$FP.D, roc.t.i$TP.I)
   # Sys.sleep(0.5)
}


# Bootstrap for confidence bounds
boot.auc.ID.fp.4cov <- matrix(1:(200*30), ncol=30)

for(i in 1:200){
   lim.mark.boot <- qsn(c(0.001,0.999), 
                        xi=estimates.boot.fp.4cov[i,2], 
                        omega=exp(estimates.boot.fp.4cov[i,3]), 
                        alpha=estimates.boot.fp.4cov[i,4])
   for(j in 1:30){
      roc.t.i <- ROC.ID.fp.frank(m.M=seq(from=lim.mark.boot[1], 
                                         to=lim.mark.boot[2], length=225), 
                                 t.T=time.marginal[j], 
                                 theta.par=estimates.boot.fp.4cov[i,1],
                                 xi.M=estimates.boot.fp.4cov[i,2], 
                                 omega.M=exp(estimates.boot.fp.4cov[i,3]), 
                                 alpha.M=estimates.boot.fp.4cov[i,4], 
                                 shape.T=exp(0), 
                                 scale.T=exp(estimates.boot.fp.4cov[i,5]))
      boot.auc.ID.fp.4cov[i,j] <- auc(roc.t.i$FP.D, roc.t.i$TP.I)
   }
}

#Boostrap quantiles
quantile.boot.auc.ID.fp.4cov <- apply(boot.auc.ID.fp.4cov, 
                                      2, quantile, probs=c(0.025,0.975))

#Inferior and superior limits
sup.auc.ID.fp.4cov <- 2*auc.ID.fp.4cov - quantile.boot.auc.ID.fp.4cov[1,]
inf.auc.ID.fp.4cov <- 2*auc.ID.fp.4cov - quantile.boot.auc.ID.fp.4cov[2,]

#AUC plot
plot(time.marginal, auc.ID.fp.4cov, type="l", ylim=c(0,1), lwd=2)
lines(time.marginal, inf.auc.ID.fp.4cov, col="gray", lwd=2)
lines(time.marginal, sup.auc.ID.fp.4cov, col="gray", lwd=2)

### Fully-parametric Incident/Dynamic ROC curves 

In [ ]:
%%R

#Point estimate
roc.ID.fp.4cov.3plots <- matrix(1:(23*3), ncol=3)
par(mfrow=c(1,3))
times.3 <- c(365,4*365,6*365)
for(i in 1:3){
   roc.ID.fp.4cov <- ROC.ID.fp.frank(m.M=m4cov.marginal, t.T=times.3[i], 
                                     theta.par=mle.fp.frank.4cov.exp$estimate[1],
                                     xi.M=mle.fp.frank.4cov.exp$estimate[2], 
                                     omega.M=exp(mle.fp.frank.4cov.exp$estimate[3]), 
                                     alpha.M=mle.fp.frank.4cov.exp$estimate[4], 
                                     shape.T=exp(0), 
                                     scale.T=exp(mle.fp.frank.4cov.exp$estimate[5]))
   cat(auc(roc.ID.fp.4cov$FP.D, roc.ID.fp.4cov$TP.I),"\n")
   roc.approx <- approx(roc.ID.fp.4cov$FP.D, roc.ID.fp.4cov$TP.I,
                        xout = q.roc.plot)
   lines(roc.approx$x, roc.approx$y, col=2)
   roc.ID.fp.4cov.3plots[,i] <- roc.approx$y
}

# Bootstrap for confidence bounds
boot.ROC.ID.fp.4cov <- array(1:(200*3*23), dim=c(200, 3, 23))
times.3 <- c(365,4*365,6*365)

for(i in 1:200){
   lim.mark.boot <- qsn(c(0.001,0.999), 
                        xi=estimates.boot.fp.4cov[i,2], 
                        omega=exp(estimates.boot.fp.4cov[i,3]), 
                        alpha=estimates.boot.fp.4cov[i,4])
   for(j in 1:3){
      roc.ID.fp.4cov.boot <- ROC.ID.fp.frank(m.M=seq(from=lim.mark.boot[1], 
                                                     to=lim.mark.boot[2], length=225), 
                                             t.T=times.3[j],
                                             theta.par=estimates.boot.fp.4cov[i,1],
                                             xi.M=estimates.boot.fp.4cov[i,2], 
                                             omega.M=exp(estimates.boot.fp.4cov[i,3]), 
                                             alpha.M=estimates.boot.fp.4cov[i,4], 
                                             shape.T=exp(0), 
                                             scale.T=exp(estimates.boot.fp.4cov[i,5]))
      roc.approx <- approx(roc.ID.fp.4cov.boot$FP.D, roc.ID.fp.4cov.boot$TP.I,
                           xout = q.roc.plot)
      boot.ROC.ID.fp.4cov[i,j,] <- roc.approx$y
   }}

#Boostrap quantiles
q.boot.ROC.ID.fp.4cov <- apply(boot.ROC.ID.fp.4cov, 
                               2:3, quantile, probs=c(0.025,0.975))

#Inferior and superior limits
roc.sup.ID.fp.4cov.3plots <- matrix(1:(23*3), ncol=3)
roc.inf.ID.fp.4cov.3plots <- matrix(1:(23*3), ncol=3)

#ROC curves plots
par(mfrow=c(1,3))
for(i in 1:3){
   plot(c(0, q.roc.plot, 1), c(0,roc.ID.fp.4cov.3plots[,i],1),
        type="l", ylim=c(0,1))
   roc.sup.ID.fp.4cov.3plots[,i] <- 2*roc.ID.fp.4cov.3plots[,i] - 
      q.boot.ROC.ID.fp.4cov[1,i,]
   roc.inf.ID.fp.4cov.3plots[,i] <- 2*roc.ID.fp.4cov.3plots[,i] - 
      q.boot.ROC.ID.fp.4cov[2,i,]
   lines(c(0,q.roc.plot,1), c(0,roc.sup.ID.fp.4cov.3plots[,i],1), 
         lwd=2, col="gray")
   lines(c(0,q.roc.plot,1), c(0,roc.inf.ID.fp.4cov.3plots[,i],1), 
         lwd=2, col="gray")
   lines(c(0, q.roc.plot, 1), c(0,roc.ID.fp.4cov.3plots[,i],1))
}

### Fully-parametric Predictiveness curves

In [ ]:
%%R 

#Point estimate
PRED.fp.4cov.3plots <- matrix(1:(23*3), ncol=3)
par(mfrow=c(1,3))
times.3 <- c(365,4*365,6*365)
for(i in 1:3){
   PRED.fp.4cov <- PRED.fp.frank(m.M=m4cov.marginal, t.T=times.3[i], 
                                 theta.par=mle.fp.frank.4cov.exp$estimate[1],
                                 xi.M=mle.fp.frank.4cov.exp$estimate[2], 
                                 omega.M=exp(mle.fp.frank.4cov.exp$estimate[3]), 
                                 alpha.M=mle.fp.frank.4cov.exp$estimate[4], 
                                 shape.T=exp(0), 
                                 scale.T=exp(mle.fp.frank.4cov.exp$estimate[5]))
   PRED.approx <- approx(PRED.fp.4cov$F.M.sn, PRED.fp.4cov$Risk.MT,
                         xout = q.roc.plot)
   lines(PRED.approx$x, PRED.approx$y, col=2)
   PRED.fp.4cov.3plots[,i] <- PRED.approx$y
}

# Bootstrap for confidence bounds
boot.PRED.fp.4cov <- array(1:(200*3*23), dim=c(200, 3, 23))
times.3 <- c(365,4*365,6*365)
for(i in 1:200){
   lim.mark.boot <- qsn(c(0.001,0.999), 
                        xi=estimates.boot.fp.4cov[i,2], 
                        omega=exp(estimates.boot.fp.4cov[i,3]), 
                        alpha=estimates.boot.fp.4cov[i,4])
   for(j in 1:3){
      PRED.fp.4cov.boot <- PRED.fp.frank(m.M=seq(from=lim.mark.boot[1], 
                                                 to=lim.mark.boot[2], length=225), 
                                         t.T=times.3[j],
                                         theta.par=estimates.boot.fp.4cov[i,1],
                                         xi.M=estimates.boot.fp.4cov[i,2], 
                                         omega.M=exp(estimates.boot.fp.4cov[i,3]), 
                                         alpha.M=estimates.boot.fp.4cov[i,4], 
                                         shape.T=exp(0), 
                                         scale.T=exp(estimates.boot.fp.4cov[i,5]))
      PRED.approx <- approx(PRED.fp.4cov.boot$F.M.sn, PRED.fp.4cov.boot$Risk.MT,
                            xout = q.roc.plot)
      boot.PRED.fp.4cov[i,j,] <- PRED.approx$y
   }}

#Boostrap quantiles
q.boot.PRED.fp.4cov <- apply(boot.PRED.fp.4cov, 
                             2:3, quantile, probs=c(0.025,0.975))

#Inferior and superior limits
roc.sup.PRED.fp.4cov.3plots <- matrix(1:(23*3), ncol=3)
roc.inf.PRED.fp.4cov.3plots <- matrix(1:(23*3), ncol=3)

#Predictiveness curves plots
par(mfrow=c(1,3))
for(i in 1:3){
   plot(q.roc.plot, PRED.fp.4cov.3plots[,i], 
        type="l", ylim=c(0,1))
   roc.sup.PRED.fp.4cov.3plots[,i] <- 2*PRED.fp.4cov.3plots[,i] - 
      q.boot.PRED.fp.4cov[1,i,]
   roc.inf.PRED.fp.4cov.3plots[,i] <- 2*PRED.fp.4cov.3plots[,i] - 
      q.boot.PRED.fp.4cov[2,i,]
   lines(q.roc.plot, roc.sup.PRED.fp.4cov.3plots[,i], 
         lwd=2, col="gray")
   lines(q.roc.plot, roc.inf.PRED.fp.4cov.3plots[,i], 
         lwd=2, col="gray")
   lines(q.roc.plot, PRED.fp.4cov.3plots[,i])
}

###  Fully-parametric Total Gain

In [ ]:
%%R

#Point estimate
STG.fp.4cov <- 1:30
for(i in 1:30){
   STG.fp.4cov[i] <- std.total.gain.fp.frank(m.M=m4cov.marginal, 
                                             t.T=time.marginal[i], 
                                             theta.par=mle.fp.frank.4cov$estimate[1],
                                             xi.M    <- mle.fp.frank.4cov$estimate[2],
                                             omega.M <- exp(mle.fp.frank.4cov$estimate[3]),
                                             alpha.M <- mle.fp.frank.4cov$estimate[4],
                                             shape.T <- exp(mle.fp.frank.4cov$estimate[5]),
                                             scale.T <- exp(mle.fp.frank.4cov$estimate[6]))
   auc.ID.fp.4cov[i] <- auc(roc.ID.t.i$FP.D, roc.ID.t.i$TP.I)
}

# Bootstrap for confidence bounds
boot.STG.fp.4cov <- matrix(1:(200*30), ncol=30)
for(i in 1:200){
   lim.mark.boot <- qsn(c(0.001,0.999), 
                        xi=estimates.boot.fp.4cov[i,2], 
                        omega=exp(estimates.boot.fp.4cov[i,3]), 
                        alpha=estimates.boot.fp.4cov[i,4])
   for(j in 1:30){
      stg.t.i <- std.total.gain.fp.frank(m.M=seq(from=lim.mark.boot[1], 
                                                 to=lim.mark.boot[2], length=225), 
                                         t.T=time.marginal[j], 
                                         theta.par=estimates.boot.fp.4cov[i,1],
                                         xi.M=estimates.boot.fp.4cov[i,2], 
                                         omega.M=exp(estimates.boot.fp.4cov[i,3]), 
                                         alpha.M=estimates.boot.fp.4cov[i,4], 
                                         shape.T=exp(0), 
                                         scale.T=exp(estimates.boot.fp.4cov[i,5]))
      boot.STG.fp.4cov[i,j] <- stg.t.i
   }
}

#Boostrap quatiles
quantile.boot.STG.fp.4cov <- apply(boot.STG.fp.4cov, 
                                   2, quantile, probs=c(0.025,0.975))
#Inferior and superior limits
sup.STG.fp.4cov <- 2*STG.fp.4cov -  quantile.boot.STG.fp.4cov[1,]
inf.STG.fp.4cov <- 2*STG.fp.4cov - quantile.boot.STG.fp.4cov[2,]

#Total Gain plot
plot(time.marginal, STG.fp.4cov, type="l", ylim=c(0,1))
lines(time.marginal, inf.STG.fp.4cov, col="gray", lwd=2)
lines(time.marginal, sup.STG.fp.4cov, col="gray", lwd=2)

### Semi-parametric Cumulative/Dynamic AUC 

In [ ]:
%%R

#Point estimator
auc.CD.sp.4cov <- 1:30
for(i in 1:30){
   roc.CD.t.i <- ROC.CD.sp.frank(c.M=m4cov.marginal, t.T=time.marginal[i],
                                 theta.par=mle.sp.frank.4cov$estimate,
                                 xi.M=estimates.sn.4cov[1], 
                                 omega.M=estimates.sn.4cov[2], 
                                 alpha.M=estimates.sn.4cov[3],
                                 times.np=KM.mayo.obj$time, 
                                 F.T.np=1 - KM.mayo.obj$surv, KM.mayo.obj$n.event)
   auc.CD.sp.4cov[i] <- auc(roc.CD.t.i$FP.D, roc.CD.t.i$TP.C)
   cat(i, " ", date(), "\n")
}


# Bootstrap for confidence bounds
sample.size <- 312
boot.auc.CD.sp.4cov <- matrix(1:(200*30), ncol=30)

for (i in 1:200){
   estimated.C.sp.4cov.w.dependence <- mvdc(frankCopula(mle.sp.frank.4cov$estimate, 
                                                        dim= 2), 
                                            c("unif", "unif"), 
                                            list(list(min = 0, max = 1), 
                                                 list(min = 0, max = 1)), 
                                            marginsIdentical = T)
   sim.U.C <- rMvdc(sample.size, estimated.C.sp.4cov.w.dependence)
   simulated.marker.boot <- qsn(sim.U.C[,1], 
                                xi=estimates.sn.4cov[1], 
                                omega=estimates.sn.4cov[2], 
                                alpha=estimates.sn.4cov[3])
   fit.sn.sim.boot <- selm(marker~1, family="SN", 
                           data=data.frame(marker=simulated.marker.boot))
   estimates.sn.sim.boot <- coef(fit.sn.sim.boot, "DP")
   lim.mark.boot <- qsn(c(0.001,0.999), 
                        xi=estimates.sn.sim.boot[1], 
                        omega=estimates.sn.sim.boot[2], 
                        alpha=estimates.sn.sim.boot[3])
   #
   cuantil.t <- 1-sim.U.C[,2]
   t.km <- quantile(KM.mayo.obj, probs = cuantil.t, conf.int = F)
   cuantil.c <- runif(sample.size)
   C.sim <- survfit(Surv(time-0.001*vital.status, 1-vital.status) ~ 1, 
                    data = pbc.mayo)
   c.km <- quantile(C.sim, probs = cuantil.c, conf.int = F)
   t.and.c <- matrix(c(t.km, c.km), ncol=2, byrow=F)
   t.star <- apply(t.and.c, 1, min, na.rm = TRUE)
   substract.sim <- t.and.c-t.star
   cens.in.sim <- rep(0, nrow(substract.sim))
   cens.in.sim[substract.sim[,1] == 0] <- 1
   #
   sim.set.boot <- data.frame(ztimes=t.star, status=cens.in.sim)
   KM.sim.boot <- survfit(Surv(ztimes, status) ~ 1, data = sim.set.boot)
   #
   mle.sim.boot <- nlm(ll.sp.frank, 6, 
                       obs.marker=simulated.marker.boot,
                       surv.times=sim.set.boot$ztimes,
                       xi.M=estimates.sn.sim.boot[1], 
                       omega.M=estimates.sn.sim.boot[2], 
                       alpha.M=estimates.sn.sim.boot[3],
                       times.np= KM.sim.boot$time, F.T.np=1 - KM.sim.boot$surv,  
                       cens.status=sim.set.boot$status,
                       hessian=F)
   cat(mle.sim.boot$estimate, "\n")
   #
   for(j in 1:30){
      roc.t.i <- ROC.CD.sp.frank(c.M=seq(from=lim.mark.boot[1], 
                                         to=lim.mark.boot[2], length=41), 
                                 t.T=time.marginal[j],
                                 theta.par=mle.sim.boot$estimate, 
                                 xi.M=estimates.sn.sim.boot[1], 
                                 omega.M=estimates.sn.sim.boot[2], 
                                 alpha.M=estimates.sn.sim.boot[3],
                                 times.np=KM.sim.boot$time, 
                                 F.T.np=1 - KM.sim.boot$surv, KM.sim.boot$n.event)
      boot.auc.CD.sp.4cov[i,j] <- auc(roc.t.i$FP.D, roc.t.i$TP.C)
   }
}

#Boostrap quantiles
quantile.boot.auc.CD.sp.4cov <- apply(boot.auc.CD.sp.4cov, 
                                      2, quantile, probs=c(0.025,0.975))

#Inferior and superior limits
sup.auc.CD.sp.4cov <- 2*auc.CD.sp.4cov - quantile.boot.auc.CD.sp.4cov[1,]
inf.auc.CD.sp.4cov <- 2*auc.CD.sp.4cov -  quantile.boot.auc.CD.sp.4cov[2,]

#AUC plot
plot(time.marginal, auc.CD.sp.4cov, type="l", ylim=c(0.7,1))
lines(time.marginal, inf.auc.CD.sp.4cov, col="gray", lwd=2)
lines(time.marginal, sup.auc.CD.sp.4cov, col="gray", lwd=2)
lines(time.marginal, auc.CD.sp.4cov, lwd=2)

### Semi-parametric Cumulative/Dynamic ROC curves

In [ ]:
%%R

#Point estimate
roc.CD.sp.4cov.3plots <- matrix(1:(23*3), ncol=3)
actual.roc.CD.sp.4cov.3plots <- matrix(1:(227*3), ncol=3)
aqtual.q.roc.CD.sp.4cov.3plots <- matrix(1:(227*3), ncol=3)

par(mfrow=c(1,3))
times.3 <- c(365,4*365,6*365)
for(i in 1:3){
   roc.CD.sp.4cov <- ROC.CD.sp.frank(c.M=m4cov.marginal, t.T=times.3[i], 
                                     theta.par=mle.sp.frank.4cov$estimate, 
                                     xi.M=estimates.sn.4cov[1], 
                                     omega.M=estimates.sn.4cov[2], 
                                     alpha.M=estimates.sn.4cov[3],
                                     times.np=KM.mayo.obj$time, 
                                     F.T.np=1 - KM.mayo.obj$surv, KM.mayo.obj$n.event)
   cat(auc(roc.CD.sp.4cov$FP.D, roc.CD.sp.4cov$TP.C),"\n")
   aqtual.q.roc.CD.sp.4cov.3plots[,i] <- roc.CD.fp.mayo$FP.D
   actual.roc.CD.sp.4cov.3plots[,i] <- roc.CD.fp.mayo$TP.C
   roc.approx <- approx(roc.CD.sp.4cov$FP.D, roc.CD.sp.4cov$TP.C,
                        xout = q.roc.plot)
   lines(roc.approx$x, roc.approx$y, col=2)
   roc.CD.sp.4cov.3plots[,i] <- roc.approx$y
}


#Bootstrap for confidence bounds
boot.ROC.CD.sp.4cov <- array(1:(200*3*23), dim=c(200, 3, 23))
times.3 <- c(365,4*365,6*365)

for(i in 1:200){
   for(j in 1:3){
      estimated.C.sp.4cov.w.dependence <- mvdc(frankCopula(mle.sp.frank.4cov$estimate, 
                                                           dim= 2), 
                                               c("unif", "unif"), 
                                               list(list(min = 0, max = 1), 
                                                    list(min = 0, max = 1)), 
                                               marginsIdentical = T)
      sim.U.C <- rMvdc(sample.size, estimated.C.sp.4cov.w.dependence)
      simulated.marker.boot <- qsn(sim.U.C[,1], 
                                   xi=estimates.sn.4cov[1], 
                                   omega=estimates.sn.4cov[2], 
                                   alpha=estimates.sn.4cov[3])
      fit.sn.sim.boot <- selm(marker~1, family="SN", 
                              data=data.frame(marker=simulated.marker.boot))
      estimates.sn.sim.boot <- coef(fit.sn.sim.boot, "DP")
      lim.mark.boot <- qsn(c(0.001,0.999), 
                           xi=estimates.sn.sim.boot[1], 
                           omega=estimates.sn.sim.boot[2], 
                           alpha=estimates.sn.sim.boot[3])
      #
      cuantil.t <- 1-sim.U.C[,2]
      t.km <- quantile(KM.mayo.obj, probs = cuantil.t, conf.int = F)
      cuantil.c <- runif(sample.size)
      C.sim <- survfit(Surv(time-0.001*vital.status, 1-vital.status) ~ 1, 
                       data = pbc.mayo)
      c.km <- quantile(C.sim, probs = cuantil.c, conf.int = F)
      t.and.c <- matrix(c(t.km, c.km), ncol=2, byrow=F)
      t.star <- apply(t.and.c, 1, min, na.rm = TRUE)
      substract.sim <- t.and.c-t.star
      cens.in.sim <- rep(0, nrow(substract.sim))
      cens.in.sim[substract.sim[,1] == 0] <- 1
      #
      sim.set.boot <- data.frame(ztimes=t.star, status=cens.in.sim)
      KM.sim.boot <- survfit(Surv(ztimes, status) ~ 1, data = sim.set.boot)
      #
      mle.sim.boot <- nlm(ll.sp.frank, 6, 
                          obs.marker=simulated.marker.boot,
                          surv.times=sim.set.boot$ztimes,
                          xi.M=estimates.sn.sim.boot[1], 
                          omega.M=estimates.sn.sim.boot[2], 
                          alpha.M=estimates.sn.sim.boot[3],
                          times.np= KM.sim.boot$time, F.T.np=1 - KM.sim.boot$surv,  
                          cens.status=sim.set.boot$status,
                          hessian=F)
      cat(mle.sim.boot$estimate, "\n")
      #
      roc.CD.sp.4cov.boot <- ROC.CD.sp.frank(c.M=seq(from=lim.mark.boot[1], 
                                                     to=lim.mark.boot[2], length=41), 
                                             t.T=times.3[j],
                                             theta.par=mle.sim.boot$estimate, 
                                             xi.M=estimates.sn.sim.boot[1], 
                                             omega.M=estimates.sn.sim.boot[2], 
                                             alpha.M=estimates.sn.sim.boot[3],
                                             times.np=KM.sim.boot$time, 
                                             F.T.np=1 - KM.sim.boot$surv, KM.sim.boot$n.event)
      roc.approx <- approx(roc.CD.sp.4cov.boot$FP.D, roc.CD.sp.4cov.boot$TP.C,
                           xout = q.roc.plot)
      boot.ROC.CD.sp.4cov[i,j,] <- roc.approx$y
   }
}

#Boostrap quantiles
q.boot.ROC.CD.sp.4cov <- apply(boot.ROC.CD.sp.4cov, 
                               2:3, quantile, probs=c(0.025,0.975))

#Inferior and superior limits
roc.sup.CD.sp.4cov.3plots <- matrix(1:(23*3), ncol=3)
roc.inf.CD.sp.4cov.3plots <- matrix(1:(23*3), ncol=3)

#ROC curves plots
par(mfrow=c(1,3))
for(i in 1:3){
   plot(c(0, q.roc.plot, 1), c(0,roc.CD.sp.4cov.3plots[,i],1),
        type="l", ylim=c(0,1))
   roc.sup.CD.sp.4cov.3plots[,i] <- 2*roc.CD.sp.4cov.3plots[,i] - 
      q.boot.ROC.CD.sp.4cov[1,i,]
   roc.inf.CD.sp.4cov.3plots[,i] <- 2*roc.CD.sp.4cov.3plots[,i] - 
      q.boot.ROC.CD.sp.4cov[2,i,]
   lines(c(0,q.roc.plot,1), c(0,roc.sup.CD.sp.4cov.3plots[,i],1), 
         lwd=2, col="gray")
   lines(c(0,q.roc.plot,1), c(0,roc.inf.CD.sp.4cov.3plots[,i],1), 
         lwd=2, col="gray")
   lines(c(0, q.roc.plot, 1), c(0,roc.CD.sp.4cov.3plots[,i],1))
}


### Semi-parametric Incident/Dynamic AUC

In [ ]:
%%R

#Point estimates
auc.ID.sp.4cov <- 1:30
for(i in 1:30){
   cat(i, "\n")
   roc.ID.t.i <- ROC.ID.sp.frank(c.M=m4cov.marginal, t.T=time.marginal[i],
                                 theta.par=mle.sp.frank.4cov$estimate,
                                 xi.M=estimates.sn.4cov[1], 
                                 omega.M=estimates.sn.4cov[2], 
                                 alpha.M=estimates.sn.4cov[3],
                                 times.np=KM.mayo.obj$time, 
                                 F.T.np=1 - KM.mayo.obj$surv, KM.mayo.obj$n.event)
   auc.ID.sp.4cov[i] <- auc(roc.ID.t.i$FP.D, roc.ID.t.i$TP.I)
}

# Bootstrap for confidence bounds
sample.size <- 312
boot.auc.ID.sp.4cov <- matrix(1:(200*30), ncol=30)


for (i in 1:200){

   estimated.C.sp.4cov.w.dependence <- mvdc(frankCopula(mle.sp.frank.4cov$estimate, 
                                                        dim= 2), 
                                            c("unif", "unif"), 
                                            list(list(min = 0, max = 1), 
                                                 list(min = 0, max = 1)), 
                                            marginsIdentical = T)
   sim.U.C <- rMvdc(sample.size, estimated.C.sp.4cov.w.dependence)
   simulated.marker.boot <- qsn(sim.U.C[,1], 
                                xi=estimates.sn.4cov[1], 
                                omega=estimates.sn.4cov[2], 
                                alpha=estimates.sn.4cov[3])
   fit.sn.sim.boot <- selm(marker~1, family="SN", 
                           data=data.frame(marker=simulated.marker.boot))
   estimates.sn.sim.boot <- coef(fit.sn.sim.boot, "DP")
   lim.mark.boot <- qsn(c(0.001,0.999), 
                        xi=estimates.sn.sim.boot[1], 
                        omega=estimates.sn.sim.boot[2], 
                        alpha=estimates.sn.sim.boot[3])
   #
   cuantil.t <- 1-sim.U.C[,2]
   t.km <- quantile(KM.mayo.obj, probs = cuantil.t, conf.int = F)
   cuantil.c <- runif(sample.size)
   C.sim <- survfit(Surv(time-0.001*vital.status, 1-vital.status) ~ 1, 
                    data = pbc.mayo)
   c.km <- quantile(C.sim, probs = cuantil.c, conf.int = F)
   t.and.c <- matrix(c(t.km, c.km), ncol=2, byrow=F)
   t.star <- apply(t.and.c, 1, min, na.rm = TRUE)
   substract.sim <- t.and.c-t.star
   cens.in.sim <- rep(0, nrow(substract.sim))
   cens.in.sim[substract.sim[,1] == 0] <- 1
   #
   sim.set.boot <- data.frame(ztimes=t.star, status=cens.in.sim)
   KM.sim.boot <- survfit(Surv(ztimes, status) ~ 1, data = sim.set.boot)
   #
   mle.sim.boot <- nlm(ll.sp.frank, 6, 
                       obs.marker=simulated.marker.boot,
                       surv.times=sim.set.boot$ztimes,
                       xi.M=estimates.sn.sim.boot[1], 
                       omega.M=estimates.sn.sim.boot[2], 
                       alpha.M=estimates.sn.sim.boot[3],
                       times.np= KM.sim.boot$time, F.T.np=1 - KM.sim.boot$surv,  
                       cens.status=sim.set.boot$status,
                       hessian=F)
   cat(mle.sim.boot$estimate, "\n")
   #
   for(j in 1:30){
      roc.t.i <- ROC.ID.sp.frank(c.M=seq(from=lim.mark.boot[1], 
                                         to=lim.mark.boot[2], length=50), 
                                 t.T=time.marginal[j],
                                 theta.par=mle.sim.boot$estimate, 
                                 xi.M=estimates.sn.sim.boot[1], 
                                 omega.M=estimates.sn.sim.boot[2], 
                                 alpha.M=estimates.sn.sim.boot[3],
                                 times.np=KM.sim.boot$time, 
                                 F.T.np=1 - KM.sim.boot$surv, KM.sim.boot$n.event)
      boot.auc.ID.sp.4cov[i,j] <- auc(roc.t.i$FP.D, roc.t.i$TP.I)
   }
}

#Boostrap quantiles
quantile.boot.auc.ID.sp.4cov <- apply(boot.auc.ID.sp.4cov, 
                                      2, quantile, probs=c(0.025,0.975))

#Inferior and superior limits
sup.auc.ID.sp.4cov <- 2*auc.ID.sp.4cov - quantile.boot.auc.ID.sp.4cov[1,]
inf.auc.ID.sp.4cov <- 2*auc.ID.sp.4cov - quantile.boot.auc.ID.sp.4cov[2,]

#AUC plot
plot(time.marginal, auc.ID.sp.4cov, type="l", ylim=c(0,1))
lines(time.marginal, inf.auc.ID.sp.4cov, col="gray", lwd=2)
lines(time.marginal, sup.auc.ID.sp.4cov, col="gray", lwd=2)
lines(time.marginal, auc.ID.sp.4cov, lwd=2)


###  Semi-parametric Incident/Dynamic ROC curves 

In [ ]:
%%R

#Point estimates
roc.ID.sp.4cov.3plots <- matrix(1:(23*3), ncol=3)
par(mfrow=c(1,3))
times.3 <- c(365,4*365,6*365)
for(i in 1:3){
   roc.ID.sp.4cov <- ROC.ID.sp.frank(c.M=m4cov.marginal, t.T=times.3[i], 
                                     theta.par=mle.sp.frank.4cov$estimate, 
                                     xi.M=estimates.sn.4cov[1], 
                                     omega.M=estimates.sn.4cov[2], 
                                     alpha.M=estimates.sn.4cov[3],
                                     times.np=KM.mayo.obj$time, 
                                     F.T.np=1 - KM.mayo.obj$surv, KM.mayo.obj$n.event)
   cat(auc(roc.ID.sp.4cov$FP.D, roc.ID.sp.4cov$TP.I),"\n")
   roc.approx <- approx(roc.ID.sp.4cov$FP.D, roc.ID.sp.4cov$TP.I,
                        xout = q.roc.plot)
   lines(roc.approx$x, roc.approx$y, col=2)
   roc.ID.sp.4cov.3plots[,i] <- roc.approx$y
}

#Bootstrap for confidence bounds
boot.ROC.ID.sp.4cov <- array(1:(200*3*23), dim=c(200, 3, 23))
times.3 <- c(365,4*365,6*365)

for(i in 1:200){
   for(j in 1:3){
      estimated.C.sp.4cov.w.dependence <- mvdc(frankCopula(mle.sp.frank.4cov$estimate, 
                                                           dim= 2), 
                                               c("unif", "unif"), 
                                               list(list(min = 0, max = 1), 
                                                    list(min = 0, max = 1)), 
                                               marginsIdentical = T)
      sim.U.C <- rMvdc(sample.size, estimated.C.sp.4cov.w.dependence)
      simulated.marker.boot <- qsn(sim.U.C[,1], 
                                   xi=estimates.sn.4cov[1], 
                                   omega=estimates.sn.4cov[2], 
                                   alpha=estimates.sn.4cov[3])
      fit.sn.sim.boot <- selm(marker~1, family="SN", 
                              data=data.frame(marker=simulated.marker.boot))
      estimates.sn.sim.boot <- coef(fit.sn.sim.boot, "DP")
      lim.mark.boot <- qsn(c(0.001,0.999), 
                           xi=estimates.sn.sim.boot[1], 
                           omega=estimates.sn.sim.boot[2], 
                           alpha=estimates.sn.sim.boot[3])
      #
      cuantil.t <- 1-sim.U.C[,2]
      t.km <- quantile(KM.mayo.obj, probs = cuantil.t, conf.int = F)
      cuantil.c <- runif(sample.size)
      C.sim <- survfit(Surv(time-0.001*vital.status, 1-vital.status) ~ 1, 
                       data = pbc.mayo)
      c.km <- quantile(C.sim, probs = cuantil.c, conf.int = F)
      t.and.c <- matrix(c(t.km, c.km), ncol=2, byrow=F)
      t.star <- apply(t.and.c, 1, min, na.rm = TRUE)
      substract.sim <- t.and.c-t.star
      cens.in.sim <- rep(0, nrow(substract.sim))
      cens.in.sim[substract.sim[,1] == 0] <- 1
      #
      sim.set.boot <- data.frame(ztimes=t.star, status=cens.in.sim)
      KM.sim.boot <- survfit(Surv(ztimes, status) ~ 1, data = sim.set.boot)
      #
      mle.sim.boot <- nlm(ll.sp.frank, 6, 
                          obs.marker=simulated.marker.boot,
                          surv.times=sim.set.boot$ztimes,
                          xi.M=estimates.sn.sim.boot[1], 
                          omega.M=estimates.sn.sim.boot[2], 
                          alpha.M=estimates.sn.sim.boot[3],
                          times.np= KM.sim.boot$time, F.T.np=1 - KM.sim.boot$surv,  
                          cens.status=sim.set.boot$status,
                          hessian=F)
      cat(mle.sim.boot$estimate, "\n")
      #
      roc.ID.sp.4cov.boot <- ROC.ID.sp.frank(c.M=seq(from=lim.mark.boot[1], 
                                                     to=lim.mark.boot[2], length=225), 
                                             t.T=times.3[j],
                                             theta.par=mle.sim.boot$estimate, 
                                             xi.M=estimates.sn.sim.boot[1], 
                                             omega.M=estimates.sn.sim.boot[2], 
                                             alpha.M=estimates.sn.sim.boot[3],
                                             times.np=KM.sim.boot$time, 
                                             F.T.np=1 - KM.sim.boot$surv, KM.sim.boot$n.event)
      roc.approx <- approx(roc.ID.sp.4cov.boot$FP.D, roc.ID.sp.4cov.boot$TP.I,
                           xout = q.roc.plot)
      boot.ROC.ID.sp.4cov[i,j,] <- roc.approx$y
   }
}

#Boostrap quantiles
q.boot.ROC.ID.sp.4cov <- apply(boot.ROC.ID.sp.4cov, 
                               2:3, quantile, probs=c(0.025,0.975))

#Inferior and superior limits
roc.sup.ID.sp.4cov.3plots <- matrix(1:(23*3), ncol=3)
roc.inf.ID.sp.4cov.3plots <- matrix(1:(23*3), ncol=3)

#ROC curves plots
par(mfrow=c(1,3))
for(i in 1:3){
   plot(c(0, q.roc.plot, 1), c(0,roc.ID.sp.4cov.3plots[,i],1),
        type="l", ylim=c(0,1))
   roc.sup.ID.sp.4cov.3plots[,i] <- 2*roc.ID.sp.4cov.3plots[,i] - 
      q.boot.ROC.ID.sp.4cov[1,i,]
   roc.inf.ID.sp.4cov.3plots[,i] <- 2*roc.ID.sp.4cov.3plots[,i] - 
      q.boot.ROC.ID.sp.4cov[2,i,]
   lines(c(0,q.roc.plot,1), c(0,roc.sup.ID.sp.4cov.3plots[,i],1), 
         lwd=2, col="gray")
   lines(c(0,q.roc.plot,1), c(0,roc.inf.ID.sp.4cov.3plots[,i],1), 
         lwd=2, col="gray")
   lines(c(0, q.roc.plot, 1), c(0,roc.ID.sp.4cov.3plots[,i],1))
}


### Semi-parametric Predictiveness curves

In [ ]:
%%R 


#Point estimate
PRED.sp.4cov.3plots <- matrix(1:(23*3), ncol=3)
par(mfrow=c(1,3))
times.3 <- c(365,4*365,6*365)

for(i in 1:3){
   PRED.sp.4cov <- PRED.sp.frank(c.M=m4cov.marginal, t.T=times.3[i],
                                 theta.par=mle.sp.frank.4cov$estimate, 
                                 xi.M=estimates.sn.4cov[1], 
                                 omega.M=estimates.sn.4cov[2], 
                                 alpha.M=estimates.sn.4cov[3],
                                 times.np=KM.mayo.obj$time, 
                                 F.T.np=1 - KM.mayo.obj$surv, KM.mayo.obj$n.event)
   PRED.approx <- approx(PRED.sp.4cov$F.M, PRED.sp.4cov$Risk.MT,
                         xout = q.roc.plot)
   lines(PRED.approx$x, PRED.approx$y, col=2)
   PRED.sp.4cov.3plots[,i] <- PRED.approx$y
}


#Bootstrap for confidence bounds
boot.PRED.sp.4cov <- array(1:(200*3*23), dim=c(200, 3, 23))
times.3 <- c(365,4*365,6*365)

for(i in 1:200){
   for(j in 1:3){
      estimated.C.sp.4cov.w.dependence <- mvdc(frankCopula(mle.sp.frank.4cov$estimate, 
                                                           dim= 2), 
                                               c("unif", "unif"), 
                                               list(list(min = 0, max = 1), 
                                                    list(min = 0, max = 1)), 
                                               marginsIdentical = T)
      sim.U.C <- rMvdc(sample.size, estimated.C.sp.4cov.w.dependence)
      simulated.marker.boot <- qsn(sim.U.C[,1], 
                                   xi=estimates.sn.4cov[1], 
                                   omega=estimates.sn.4cov[2], 
                                   alpha=estimates.sn.4cov[3])
      fit.sn.sim.boot <- selm(marker~1, family="SN", 
                              data=data.frame(marker=simulated.marker.boot))
      estimates.sn.sim.boot <- coef(fit.sn.sim.boot, "DP")
      lim.mark.boot <- qsn(c(0.001,0.999), 
                           xi=estimates.sn.sim.boot[1], 
                           omega=estimates.sn.sim.boot[2], 
                           alpha=estimates.sn.sim.boot[3])
      #
      cuantil.t <- 1-sim.U.C[,2]
      t.km <- quantile(KM.mayo.obj, probs = cuantil.t, conf.int = F)
      cuantil.c <- runif(sample.size)
      C.sim <- survfit(Surv(time-0.001*vital.status, 1-vital.status) ~ 1, 
                       data = pbc.mayo)
      c.km <- quantile(C.sim, probs = cuantil.c, conf.int = F)
      t.and.c <- matrix(c(t.km, c.km), ncol=2, byrow=F)
      t.star <- apply(t.and.c, 1, min, na.rm = TRUE)
      substract.sim <- t.and.c-t.star
      cens.in.sim <- rep(0, nrow(substract.sim))
      cens.in.sim[substract.sim[,1] == 0] <- 1
      #
      sim.set.boot <- data.frame(ztimes=t.star, status=cens.in.sim)
      KM.sim.boot <- survfit(Surv(ztimes, status) ~ 1, data = sim.set.boot)
      #
      mle.sim.boot <- nlm(ll.sp.frank, 6, 
                          obs.marker=simulated.marker.boot,
                          surv.times=sim.set.boot$ztimes,
                          xi.M=estimates.sn.sim.boot[1], 
                          omega.M=estimates.sn.sim.boot[2], 
                          alpha.M=estimates.sn.sim.boot[3],
                          times.np= KM.sim.boot$time, F.T.np=1 - KM.sim.boot$surv,  
                          cens.status=sim.set.boot$status,
                          hessian=F)
      cat(mle.sim.boot$estimate, "\n")
      #
      pred.sp.4cov.boot <- PRED.sp.frank(c.M=seq(from=lim.mark.boot[1], 
                                                 to=lim.mark.boot[2], length=225), 
                                         t.T=times.3[j],
                                         theta.par=mle.sim.boot$estimate, 
                                         xi.M=estimates.sn.sim.boot[1], 
                                         omega.M=estimates.sn.sim.boot[2], 
                                         alpha.M=estimates.sn.sim.boot[3],
                                         times.np=KM.sim.boot$time, 
                                         F.T.np=1 - KM.sim.boot$surv, KM.sim.boot$n.event)
      pred.approx <- approx(pred.sp.4cov.boot$F.M, pred.sp.4cov.boot$Risk.MT,
                            xout = q.roc.plot)
      boot.PRED.sp.4cov[i,j,] <- pred.approx$y
   }
}


#Boostrap quantiles
q.boot.PRED.sp.4cov <- apply(boot.PRED.sp.4cov, 
                             2:3, quantile, probs=c(0.025,0.975))

#Inferior and superior limits
pred.sup.sp.4cov.3plots <- matrix(1:(23*3), ncol=3)
pred.inf.sp.4cov.3plots <- matrix(1:(23*3), ncol=3)

#Preditiveness curves plots
par(mfrow=c(1,3))
for(i in 1:3){
   plot(q.roc.plot, PRED.sp.4cov.3plots[,i],
        type="l", ylim=c(0,1))
   pred.sup.sp.4cov.3plots[,i] <- 2*PRED.sp.4cov.3plots[,i] - 
      q.boot.PRED.sp.4cov[1,i,]
   pred.inf.sp.4cov.3plots[,i] <- 2*PRED.sp.4cov.3plots[,i] - 
      q.boot.PRED.sp.4cov[2,i,]
   lines(q.roc.plot, pred.sup.sp.4cov.3plots[,i], 
         lwd=2, col="gray")
   lines(q.roc.plot, pred.inf.sp.4cov.3plots[,i], 
         lwd=2, col="gray")
   lines(q.roc.plot, PRED.sp.4cov.3plots[,i])
}   

### Semi-parametric Total Gain

In [ ]:
%%R

#Point estimate
STG.sp.4cov <- 1:30

system.time
for(i in 1:30){
   STG.sp.4cov[i] <- std.total.gain.sp.frank(c.M=m4cov.marginal, t.T=time.marginal[i], 
                                             theta.par=mle.sp.frank.4cov$estimate,
                                             xi.M=estimates.sn.4cov[1], 
                                             omega.M=estimates.sn.4cov[2], 
                                             alpha.M=estimates.sn.4cov[3],
                                             times.np=KM.mayo.obj$time, 
                                             F.T.np=1 - KM.mayo.obj$surv, KM.mayo.obj$n.event)
}


# Bootstrap for confidence bounds
sample.size <- 312
boot.STG.sp.4cov <- matrix(1:(200*30), ncol=30)


for (i in 1:200){
   
   estimated.C.sp.4cov.w.dependence <- mvdc(frankCopula(mle.sp.frank.4cov$estimate, 
                                                        dim= 2), 
                                            c("unif", "unif"), 
                                            list(list(min = 0, max = 1), 
                                                 list(min = 0, max = 1)), 
                                            marginsIdentical = T)
   sim.U.C <- rMvdc(sample.size, estimated.C.sp.4cov.w.dependence)
   simulated.marker.boot <- qsn(sim.U.C[,1], 
                                xi=estimates.sn.4cov[1], 
                                omega=estimates.sn.4cov[2], 
                                alpha=estimates.sn.4cov[3])
   fit.sn.sim.boot <- selm(marker~1, family="SN", 
                           data=data.frame(marker=simulated.marker.boot))
   estimates.sn.sim.boot <- coef(fit.sn.sim.boot, "DP")
   lim.mark.boot <- qsn(c(0.001,0.999), 
                        xi=estimates.sn.sim.boot[1], 
                        omega=estimates.sn.sim.boot[2], 
                        alpha=estimates.sn.sim.boot[3])
   #
   cuantil.t <- 1-sim.U.C[,2]
   t.km <- quantile(KM.mayo.obj, probs = cuantil.t, conf.int = F)
   cuantil.c <- runif(sample.size)
   C.sim <- survfit(Surv(time-0.001*vital.status, 1-vital.status) ~ 1, 
                    data = pbc.mayo)
   c.km <- quantile(C.sim, probs = cuantil.c, conf.int = F)
   t.and.c <- matrix(c(t.km, c.km), ncol=2, byrow=F)
   t.star <- apply(t.and.c, 1, min, na.rm = TRUE)
   substract.sim <- t.and.c-t.star
   cens.in.sim <- rep(0, nrow(substract.sim))
   cens.in.sim[substract.sim[,1] == 0] <- 1
   #
   sim.set.boot <- data.frame(ztimes=t.star, status=cens.in.sim)
   KM.sim.boot <- survfit(Surv(ztimes, status) ~ 1, data = sim.set.boot)
   #
   mle.sim.boot <- nlm(ll.sp.frank, 6, 
                       obs.marker=simulated.marker.boot,
                       surv.times=sim.set.boot$ztimes,
                       xi.M=estimates.sn.sim.boot[1], 
                       omega.M=estimates.sn.sim.boot[2], 
                       alpha.M=estimates.sn.sim.boot[3],
                       times.np= KM.sim.boot$time, F.T.np=1 - KM.sim.boot$surv,  
                       cens.status=sim.set.boot$status,
                       hessian=F)
   cat(mle.sim.boot$estimate, "\n")
   #
   for(j in 1:30){
      boot.STG.sp.4cov[i,j]<- std.total.gain.sp.frank(c.M=seq(from=lim.mark.boot[1], 
                                                              to=lim.mark.boot[2], length=50), 
                                                      t.T=time.marginal[j],
                                                      theta.par=mle.sim.boot$estimate, 
                                                      xi.M=estimates.sn.sim.boot[1], 
                                                      omega.M=estimates.sn.sim.boot[2], 
                                                      alpha.M=estimates.sn.sim.boot[3],
                                                      times.np=KM.sim.boot$time, 
                                                      F.T.np=1 - KM.sim.boot$surv, KM.sim.boot$n.event)
   }
}

#Boostrap quantiles
quantile.boot.STG.sp.4cov <- apply(boot.STG.sp.4cov, 
                                   2, quantile, probs=c(0.025,0.975))

#Inferior and superior limits
sup.STG.sp.4cov <- 2*STG.sp.4cov -  quantile.boot.STG.sp.4cov[1,]
inf.STG.sp.4cov <- 2*STG.sp.4cov -    quantile.boot.STG.sp.4cov[2,]

#Total Gain plot
plot(time.marginal, STG.sp.4cov, type="l", ylim=c(0,1))
lines(time.marginal, inf.STG.sp.4cov, col="gray", lwd=2)
lines(time.marginal, sup.STG.sp.4cov, col="gray", lwd=2)
lines(time.marginal, STG.sp.4cov, lwd=2)


## b) Mayo marker analysis

### Fully-parametric (fp) model fitting

In [ ]:
%%R 

#Initial values for optimization
var.ini.gumbel.fp.mayo <- c(0, 6, log(2), 6, 0, 8)

#MLE estimation
mle.fp.gumbel.mayo <- nlm(ll.fp.gumbel, var.ini.gumbel.fp.mayo,
                          obs.marker=pbc.mayo$marker.mayo, surv.times=pbc.mayo$time, 
                          cens.status=pbc.mayo$vital.status,
                          hessian=T)

print("Kendall's Tau")
print(tau(gumbelCopula(exp(mle.fp.gumbel.mayo$estimate[1])+1, dim= 2)))


#Wald statistic
Z0.gumbel.fp.mayo <- mle.fp.gumbel.mayo$estimate/sqrt(diag(solve(mle.fp.gumbel.mayo$hessian)))
round(2*pnorm(abs(Z0.gumbel.fp.mayo), lower.tail = FALSE),4)

#Wald statistics
print("Wald statistics")
Z0.gumbel.fp.mayo <- mle.fp.gumbel.mayo$estimate/sqrt(diag(solve(mle.fp.gumbel.mayo$hessian)))
print(Z0.gumbel.fp.mayo)
print("P-values")
print(round(2*pnorm(abs(Z0.gumbel.fp.mayo), lower.tail = FALSE),4))


#Estimation with theta=2 

#Initial values for optimization
var.ini.gumbel.fp.tau0.5.mayo <- c(6, log(2), 6, 0, 8)

#MLE estimation
mle.fp.gumbel.tau0.5.mayo <- nlm(ll.fp.tau0.5.gumbel, var.ini.gumbel.fp.tau0.5.mayo,
                                 obs.marker=pbc.mayo$marker.mayo, surv.times=pbc.mayo$time, 
                                 cens.status=pbc.mayo$vital.status,
                                 hessian=T)


#Wald statistics
print("Wald statistics")
Z0.fp.gumbel.tau0.5.mayo <- mle.fp.gumbel.tau0.5.mayo$estimate/sqrt(diag(solve(mle.fp.gumbel.tau0.5.mayo$hessian)))
print(Z0.fp.gumbel.tau0.5.mayo)
print("P-values")
print(round(2*pnorm(abs(Z0.fp.gumbel.tau0.5.mayo), lower.tail = FALSE),4))


print("omega1")
print(round(mle.fp.gumbel.tau0.5.mayo$estimate[1],3))
print(round(sqrt(diag(solve(mle.fp.gumbel.tau0.5.mayo$hessian)))[1],3)) #SE

print("log(omega2)")
print(round(mle.fp.gumbel.tau0.5.mayo$estimate[2],3))
print(round(sqrt(diag(solve(mle.fp.gumbel.tau0.5.mayo$hessian)))[2],3)) #SE

print("omega3")
print(round(mle.fp.gumbel.tau0.5.mayo$estimate[3],3))
round(sqrt(diag(solve(mle.fp.gumbel.tau0.5.mayo$hessian)))[3],3) #SE

print("log(lambda1)")
print(round(mle.fp.gumbel.tau0.5.mayo$estimate[4],3))
print(round(sqrt(diag(solve(mle.fp.gumbel.tau0.5.mayo$hessian)))[4],3)) #SE

print("log(lambda2)")
print(round(mle.fp.gumbel.tau0.5.mayo$estimate[5],3))
print(round(sqrt(diag(solve(mle.fp.gumbel.tau0.5.mayo$hessian)))[5],3)) #SE


# Bootstrap for confidence intervals:

sample.size <- 312
estimates.boot.fp.mayo <- NULL

for (i in 1:200){
   estimated.C.fp.mayo <- mvdc(gumbelCopula(2, dim= 2), 
                               c("unif", "unif"), 
                               list(list(min = 0, max = 1), 
                                    list(min = 0, max = 1)), 
                               marginsIdentical = T)
   sim.U.C.par <- rMvdc(sample.size, estimated.C.fp.mayo)
 
   #
   marker.par.boot <- qsn(sim.U.C.par[,1], 
                          xi=mle.fp.gumbel.tau0.5.mayo$estimate[1], 
                          omega=exp(mle.fp.gumbel.tau0.5.mayo$estimate[2]), 
                          alpha=mle.fp.gumbel.tau0.5.mayo$estimate[3])
   cuantil.t.par <- 1-sim.U.C.par[,2]
   t.par <- qweibull(cuantil.t.par, 
                     shape = exp(mle.fp.gumbel.tau0.5.mayo$estimate[4]), 
                     scale = exp(mle.fp.gumbel.tau0.5.mayo$estimate[5]))
  
   cuantil.c <- runif(sample.size)
   C.dist <- survfit(Surv(time-0.001*vital.status, 1-vital.status) ~ 1, 
                     data = pbc.mayo)
   c.km <- quantile(C.dist, probs = cuantil.c, conf.int = F)
   t.and.c.par <- matrix(c(t.par, c.km), ncol=2, byrow=F)
   t.star.par <- apply(t.and.c.par, 1, min, na.rm = TRUE)
   substract.sim.par <- t.and.c.par-t.star.par
   cens.in.sim.par <- rep(0, nrow(substract.sim.par))
   cens.in.sim.par[substract.sim.par[,1] == 0] <- 1
   
   mle.sim.parametric.boot <- nlm(ll.fp.tau0.5.gumbel, 
                                  var.ini.gumbel.fp.tau0.5.mayo,
                                  obs.marker=marker.par.boot,
                                  surv.times=t.star.par,
                                  cens.status=cens.in.sim.par,
                                  hessian=F)
   estimates.boot.fp.mayo <- rbind(estimates.boot.fp.mayo,
                                   c(mle.sim.parametric.boot$estimate))
}

#Boostrap standard errors
se.mayo.weibull <- apply(estimates.boot.fp.mayo, 2, sd)
print("Boostrap SE:")
print(se.mayo.weibull)

### Semi-parametric (sp) model fitting

In [ ]:
%%R 

#Marker margin
fit.marker.mayo <- selm(marker.mayo~1, family="SN", data=pbc.mayo)
estimates.sn.mayo <- coef(fit.marker.mayo, "DP")
summary.fit.marker.mayo <- summary(fit.marker.mayo, "DP", cov=TRUE, cor=TRUE)

#Time margin
KM.mayo.obj <- survfit(Surv(time, vital.status) ~  1, 
                       data=pbc.mayo)

#MLE Estimation
mle.sp.gumbel.mayo <- nlm(ll.sp.gumbel, 0, 
                          obs.marker=c(pbc.mayo$marker.mayo),
                          surv.times=pbc.mayo$time,
                          xi.M=estimates.sn.mayo[1], 
                          omega.M=estimates.sn.mayo[2], 
                          alpha.M=estimates.sn.mayo[3],
                          times.np= KM.mayo.obj$time, F.T.np=1 - KM.mayo.obj$surv,  
                          cens.status=pbc.mayo$vital.status, hessian= TRUE)
#Theta 
print("theta")
print(1 + exp(round(mle.sp.gumbel.mayo$estimate[1],3)))

# Bootstrap for confidence intervals
sample.size <- 312

estimates.boot.sp.mayo.w.dependence <- NULL
for (i in 1:200){
   estimated.C.sp.mayo.w.dependence <- mvdc(gumbelCopula(exp(mle.sp.gumbel.mayo$estimate)+1, 
                                                         dim= 2), 
                                            c("unif", "unif"), 
                                            list(list(min = 0, max = 1), 
                                                 list(min = 0, max = 1)), 
                                            marginsIdentical = T)
   sim.U.C <- rMvdc(sample.size, estimated.C.sp.mayo.w.dependence)
   simulated.marker.boot <- qsn(sim.U.C[,1], 
                                xi=estimates.sn.mayo[1], 
                                omega=estimates.sn.mayo[2], 
                                alpha=estimates.sn.mayo[3])
   fit.sn.sim.boot <- selm(marker~1, family="SN", 
                           data=data.frame(marker=simulated.marker.boot))
   estimates.sn.sim.boot <- coef(fit.sn.sim.boot, "DP")
   cuantil.t <- 1-sim.U.C[,2]
   t.km <- quantile(KM.mayo.obj, probs = cuantil.t, conf.int = F)
   cuantil.c <- runif(sample.size)
   C.sim <- survfit(Surv(time-0.001*vital.status, 1-vital.status) ~ 1, 
                    data = pbc.mayo)
   c.km <- quantile(C.sim, probs = cuantil.c, conf.int = F)
   t.and.c <- matrix(c(t.km, c.km), ncol=2, byrow=F)
   t.star <- apply(t.and.c, 1, min, na.rm = TRUE)
   substract.sim <- t.and.c-t.star
   cens.in.sim <- rep(0, nrow(substract.sim))
   cens.in.sim[substract.sim[,1] == 0] <- 1
   #
   sim.set.boot <- data.frame(ztimes=t.star, status=cens.in.sim)
   KM.sim.boot <- survfit(Surv(ztimes, status) ~ 1, data = sim.set.boot)
   #
   mle.sim.boot <- nlm(ll.sp.gumbel, 0, 
                       obs.marker=simulated.marker.boot,
                       surv.times=sim.set.boot$ztimes,
                       xi.M=estimates.sn.sim.boot[1], 
                       omega.M=estimates.sn.sim.boot[2], 
                       alpha.M=estimates.sn.sim.boot[3],
                       times.np= KM.sim.boot$time, F.T.np=1 - KM.sim.boot$surv,  
                       cens.status=sim.set.boot$status,
                       hessian=F)
   estimates.boot.sp.mayo.w.dependence <- rbind(estimates.boot.sp.mayo.w.dependence,
                                                c(mle.sim.boot$estimate, 
                                                  estimates.sn.sim.boot[1], 
                                                  log(estimates.sn.sim.boot[2]), 
                                                  estimates.sn.sim.boot[3]))
}

# Bootstrap standard errors
se.sp.mayo <- apply(estimates.boot.sp.mayo.w.dependence, 2, sd)

#Estimated parameters


print("omega1")
print(summary.fit.marker.mayo@param.table[1,1]) #Estimator
print(se.sp.mayo[2]) #SE


print("log(omega2)")
print(log(summary.fit.marker.mayo@param.table[2,1])) #Estimator
print(se.sp.mayo[3]) #SE

print("omega3")
print(summary.fit.marker.mayo@param.table[3,1]) #Estimator
print(summary.fit.marker.mayo@param.table[3,2]) #SE


# Confidence interval for theta
inv.quantiles.dep.star <- quantile(estimates.boot.sp.mayo.w.dependence[,1], 
                                   c(0.025,0.975))

theta.mayo.boot.inf <- exp(2*mle.sp.gumbel.mayo$estimate- inv.quantiles.dep.star[2])+1

theta.mayo.boot.sup <- exp(2*mle.sp.gumbel.mayo$estimate- inv.quantiles.dep.star[1])+1

#Confidence interval for theta
print("Boostrap CI for theta")
print(c(theta.mayo.boot.inf, theta.mayo.boot.sup))



###  Fully-parametric Cumulative/Dynamic AUC

In [ ]:
%%R 

#Vectors for representing time and mayo marker
time.marginal <- seq(365, 4380, length.out = 30)
mayo.marginal <- seq(5.4, 13.1, length.out = 225)
xy.mayo <- list(time=time.marginal, mayo=mayo.marginal)
grid.mayo <- expand.grid(xy.mayo)

# Point estimate
auc.CD.fp.mayo <- 1:30
tp.cd.fp.mayo <- NULL
fp.cd.fp.mayo <- NULL
for(i in 1:30){
   roc.t.i <- ROC.CD.fp.gumbel(m.M=mayo.marginal, t.T=time.marginal[i], 
                               theta.par=2, xi.M=mle.fp.gumbel.tau0.5.mayo$estimate[1], 
                               omega.M=exp(mle.fp.gumbel.tau0.5.mayo$estimate[2]), 
                               alpha.M=mle.fp.gumbel.tau0.5.mayo$estimate[3], 
                               shape.T=exp(mle.fp.gumbel.tau0.5.mayo$estimate[4]), 
                               scale.T=exp(mle.fp.gumbel.tau0.5.mayo$estimate[5]))
   auc.CD.fp.mayo[i] <- auc(roc.t.i$FP.D, roc.t.i$TP.C)
   cat("time=", time.marginal[i], "	AUC=", auc.CD.fp.mayo[i], "\n")
   tp.cd.fp.mayo <- c(tp.cd.fp.mayo, roc.t.i$TP.C)
   fp.cd.fp.mayo <- c(fp.cd.fp.mayo, roc.t.i$FP.D)
}


# Bootstrap for CD AUC 
boot.auc.CD.fp.mayo <- matrix(1:(200*30), ncol=30)

for(i in 1:200){
   lim.mark.boot <- qsn(c(0.001,0.999), 
                        xi=estimates.boot.fp.mayo[i,1], 
                        omega=exp(estimates.boot.fp.mayo[i,2]), 
                        alpha=estimates.boot.fp.mayo[i,3])
   for(j in 1:30){
      roc.t.i <- ROC.CD.fp.gumbel(m.M=seq(from=lim.mark.boot[1], 
                                          to=lim.mark.boot[2], length=50), 
                                  t.T=time.marginal[j], 
                                  theta.par=2, xi.M=estimates.boot.fp.mayo[i,1], 
                                  omega.M=exp(estimates.boot.fp.mayo[i,2]), 
                                  alpha.M=estimates.boot.fp.mayo[i,3], 
                                  shape.T=exp(estimates.boot.fp.mayo[i,4]), 
                                  scale.T=exp(estimates.boot.fp.mayo[i,5]))
      boot.auc.CD.fp.mayo[i,j] <- auc(roc.t.i$FP.D, roc.t.i$TP.C)
   }
}

#Boostrap quantiles
quantile.boot.auc.CD.fp.mayo <- apply(boot.auc.CD.fp.mayo, 
                                      2, quantile, probs=c(0.025,0.975))
#Inferior and superior limits
sup.auc.CD.fp.mayo <- 2*auc.CD.fp.mayo - quantile.boot.auc.CD.fp.mayo[1,]
inf.auc.CD.fp.mayo <- 2*auc.CD.fp.mayo - quantile.boot.auc.CD.fp.mayo[2,]

#AUC plot
plot(time.marginal, auc.CD.fp.mayo, type="l", ylim=c(0.7,1), lwd=2)
lines(time.marginal, inf.auc.CD.fp.mayo, col="gray", lwd=2)
lines(time.marginal, sup.auc.CD.fp.mayo, col="gray", lwd=2)

### Fully-parametric Cumulative/Dynamic ROC curves 

In [ ]:
%%R 
#Point estimates
q.roc.plot <- seq(from=0.01, to=0.99, length=23)

roc.CD.fp.mayo.3plots <- matrix(1:(23*3), ncol=3)
actual.roc.CD.fp.mayo.3plots <- matrix(1:(227*3), ncol=3)
aqtual.q.roc.CD.fp.mayo.3plots <- matrix(1:(227*3), ncol=3)
par(mfrow=c(1,3))
times.3 <- c(365,4*365,6*365)
for(i in 1:3){
   roc.CD.fp.mayo <- ROC.CD.fp.gumbel(m.M=mayo.marginal, t.T=times.3[i], 
                                      theta.par=2, xi.M=mle.fp.gumbel.tau0.5.mayo$estimate[1], 
                                      omega.M=exp(mle.fp.gumbel.tau0.5.mayo$estimate[2]), 
                                      alpha.M=mle.fp.gumbel.tau0.5.mayo$estimate[3], 
                                      shape.T=exp(mle.fp.gumbel.tau0.5.mayo$estimate[4]), 
                                      scale.T=exp(mle.fp.gumbel.tau0.5.mayo$estimate[5]))
   cat(auc(roc.CD.fp.mayo$FP.D, roc.CD.fp.mayo$TP.C),"\n")
   aqtual.q.roc.CD.fp.mayo.3plots[,i] <- roc.CD.fp.mayo$FP.D
   actual.roc.CD.fp.mayo.3plots[,i] <- roc.CD.fp.mayo$TP.C
   roc.approx <- approx(roc.CD.fp.mayo$FP.D, roc.CD.fp.mayo$TP.C,
                        xout = q.roc.plot)
   lines(roc.approx$x, roc.approx$y, col=2)
   roc.CD.fp.mayo.3plots[,i] <- roc.approx$y
}

# Bootstrap for confidence bounds
boot.ROC.CD.fp.mayo <- array(1:(200*3*23), dim=c(200, 3, 23))
times.3 <- c(365,4*365,6*365)
for(i in 1:200){
   lim.mark.boot <- qsn(c(0.001,0.999), 
                        xi=estimates.boot.fp.mayo[i,1], 
                        omega=exp(estimates.boot.fp.mayo[i,2]), 
                        alpha=estimates.boot.fp.mayo[i,3])
   for(j in 1:3){
      roc.CD.fp.mayo.boot <- ROC.CD.fp.gumbel(m.M=seq(from=lim.mark.boot[1], 
                                                      to=lim.mark.boot[2], length=225), 
                                              t.T=times.3[j],
                                              theta.par=2, xi.M=estimates.boot.fp.mayo[i,1], 
                                              omega.M=exp(estimates.boot.fp.mayo[i,2]), 
                                              alpha.M=estimates.boot.fp.mayo[i,3], 
                                              shape.T=exp(estimates.boot.fp.mayo[i,4]), 
                                              scale.T=exp(estimates.boot.fp.mayo[i,5]))
      roc.approx <- approx(roc.CD.fp.mayo.boot$FP.D, roc.CD.fp.mayo.boot$TP.C,
                           xout = q.roc.plot)
      boot.ROC.CD.fp.mayo[i,j,] <- roc.approx$y
   }}

#Boostrap quantiles
q.boot.ROC.CD.fp.mayo <- apply(boot.ROC.CD.fp.mayo, 
                               2:3, quantile, probs=c(0.025,0.975))

#Inferior and superior limits
roc.sup.CD.fp.mayo.3plots <- matrix(1:(23*3), ncol=3)
roc.inf.CD.fp.mayo.3plots <- matrix(1:(23*3), ncol=3)

#ROC curves plots
par(mfrow=c(1,3))
for(i in 1:3){
   plot(c(0, q.roc.plot, 1), c(0,roc.CD.fp.mayo.3plots[,i],1), type="l", ylim=c(0,1))
   roc.sup.CD.fp.mayo.3plots[,i] <- 2*roc.CD.fp.mayo.3plots[,i] - 
      q.boot.ROC.CD.fp.mayo[1,i,]
   roc.inf.CD.fp.mayo.3plots[,i] <- 2*roc.CD.fp.mayo.3plots[,i] - 
      q.boot.ROC.CD.fp.mayo[2,i,]
   lines(c(0,q.roc.plot,1), c(0,roc.sup.CD.fp.mayo.3plots[,i],1), 
         lwd=2, col="gray")
   lines(c(0,q.roc.plot,1), c(0,roc.inf.CD.fp.mayo.3plots[,i],1), 
         lwd=2, col="gray")
   lines(c(0, q.roc.plot, 1), c(0,roc.CD.fp.mayo.3plots[,i],1))
}


### Fully-parametric Incident/Dynamic AUC 

In [ ]:
%%R

#Point estimate
auc.ID.fp.mayo <- 1:30
for(i in 1:30){
   roc.ID.t.i <- ROC.ID.fp.gumbel(m.M=mayo.marginal, t.T=time.marginal[i], 
                                  theta.par=2, xi.M=mle.fp.gumbel.tau0.5.mayo$estimate[1], 
                                  omega.M=exp(mle.fp.gumbel.tau0.5.mayo$estimate[2]), 
                                  alpha.M=mle.fp.gumbel.tau0.5.mayo$estimate[3], 
                                  shape.T=exp(mle.fp.gumbel.tau0.5.mayo$estimate[4]), 
                                  scale.T=exp(mle.fp.gumbel.tau0.5.mayo$estimate[5]))
   auc.ID.fp.mayo[i] <- auc(roc.ID.t.i$FP.D, roc.ID.t.i$TP.I)
}

# Bootstrap for ID AUC 
boot.auc.ID.fp.mayo <- matrix(1:(200*30), ncol=30)
for(i in 1:200){
   lim.mark.boot <- qsn(c(0.001,0.999), 
                        xi=estimates.boot.fp.mayo[i,1], 
                        omega=exp(estimates.boot.fp.mayo[i,2]), 
                        alpha=estimates.boot.fp.mayo[i,3])
   for(j in 1:30){
      roc.t.i <- ROC.ID.fp.gumbel(m.M=seq(from=lim.mark.boot[1], 
                                          to=lim.mark.boot[2], length=30), 
                                  t.T=time.marginal[j], 
                                  theta.par=2, xi.M=estimates.boot.fp.mayo[i,1], 
                                  omega.M=exp(estimates.boot.fp.mayo[i,2]), 
                                  alpha.M=estimates.boot.fp.mayo[i,3], 
                                  shape.T=exp(estimates.boot.fp.mayo[i,4]), 
                                  scale.T=exp(estimates.boot.fp.mayo[i,5]))
      boot.auc.ID.fp.mayo[i,j] <- auc(roc.t.i$FP.D, roc.t.i$TP.I)
   }
}

#Boostrap quantiles
quantile.boot.auc.ID.fp.mayo <- apply(boot.auc.ID.fp.mayo, 
                                      2, quantile, probs=c(0.025,0.975))
#Inferior and superior limits
sup.auc.ID.fp.mayo <- 2*auc.ID.fp.mayo - quantile.boot.auc.ID.fp.mayo[1,]
inf.auc.ID.fp.mayo <- 2*auc.ID.fp.mayo - quantile.boot.auc.ID.fp.mayo[2,]

#AUC plot
plot(time.marginal, auc.ID.fp.mayo, type="l", ylim=c(0,1), lwd=2)
lines(time.marginal, inf.auc.ID.fp.mayo, col="gray", lwd=2)
lines(time.marginal, sup.auc.ID.fp.mayo, col="gray", lwd=2)

### Fully-parametric Incident/Dynamic ROC curves

In [ ]:
%%R

#Point estimates
roc.ID.fp.mayo.3plots <- matrix(1:(23*3), ncol=3)
par(mfrow=c(1,3))
times.3 <- c(365,4*365,6*365)
for(i in 1:3){
   roc.ID.fp.mayo <- ROC.ID.fp.gumbel(m.M=mayo.marginal, t.T=times.3[i], 
                                      theta.par=2, xi.M=mle.fp.gumbel.tau0.5.mayo$estimate[1], 
                                      omega.M=exp(mle.fp.gumbel.tau0.5.mayo$estimate[2]), 
                                      alpha.M=mle.fp.gumbel.tau0.5.mayo$estimate[3], 
                                      shape.T=exp(mle.fp.gumbel.tau0.5.mayo$estimate[4]), 
                                      scale.T=exp(mle.fp.gumbel.tau0.5.mayo$estimate[5]))
   roc.approx <- approx(roc.ID.fp.mayo$FP.D, roc.ID.fp.mayo$TP.I,
                        xout = q.roc.plot)
   lines(roc.approx$x, roc.approx$y, col=2)
   roc.ID.fp.mayo.3plots[,i] <- roc.approx$y
}

#Bootstrap for confidence bounds
boot.ROC.ID.fp.mayo <- array(1:(200*3*23), dim=c(200, 3, 23))
times.3 <- c(365,4*365,6*365)
for(i in 1:200){
   lim.mark.boot <- qsn(c(0.001,0.999), 
                        xi=estimates.boot.fp.mayo[i,1], 
                        omega=exp(estimates.boot.fp.mayo[i,2]), 
                        alpha=estimates.boot.fp.mayo[i,3])
   for(j in 1:3){
      roc.ID.fp.mayo.boot <- ROC.ID.fp.gumbel(m.M=seq(from=lim.mark.boot[1], 
                                                      to=lim.mark.boot[2], length=225), 
                                              t.T=times.3[j],
                                              theta.par=2, xi.M=estimates.boot.fp.mayo[i,1], 
                                              omega.M=exp(estimates.boot.fp.mayo[i,2]), 
                                              alpha.M=estimates.boot.fp.mayo[i,3], 
                                              shape.T=exp(estimates.boot.fp.mayo[i,4]), 
                                              scale.T=exp(estimates.boot.fp.mayo[i,5]))
      roc.approx <- approx(roc.ID.fp.mayo.boot$FP.D, roc.ID.fp.mayo.boot$TP.I,
                           xout = q.roc.plot)
      boot.ROC.ID.fp.mayo[i,j,] <- roc.approx$y
   }}

#Boostrap quantiles
q.boot.ROC.ID.fp.mayo <- apply(boot.ROC.ID.fp.mayo, 
                               2:3, quantile, probs=c(0.025,0.975))

#Inferior and superior limits
roc.sup.ID.fp.mayo.3plots <- matrix(1:(23*3), ncol=3)
roc.inf.ID.fp.mayo.3plots <- matrix(1:(23*3), ncol=3)

#ROC curve plots
par(mfrow=c(1,3))
for(i in 1:3){
   plot(c(0, q.roc.plot, 1), c(0,roc.ID.fp.mayo.3plots[,i],1), 
        type="l", ylim=c(0,1))
   roc.sup.ID.fp.mayo.3plots[,i] <- 2*roc.ID.fp.mayo.3plots[,i] - 
      q.boot.ROC.ID.fp.mayo[1,i,]
   roc.inf.ID.fp.mayo.3plots[,i] <- 2*roc.ID.fp.mayo.3plots[,i] - 
      q.boot.ROC.ID.fp.mayo[2,i,]
   lines(c(0,q.roc.plot,1), c(0,roc.sup.ID.fp.mayo.3plots[,i],1), 
         lwd=2, col="gray")
   lines(c(0,q.roc.plot,1), c(0,roc.inf.ID.fp.mayo.3plots[,i],1), 
         lwd=2, col="gray")
   lines(c(0, q.roc.plot, 1), c(0,roc.ID.fp.mayo.3plots[,i],1))
}


### Fully-parametric Predictiveness curves

In [ ]:
%%R


#Point estimate
PRED.fp.mayo.3plots <- matrix(1:(23*3), ncol=3)
par(mfrow=c(1,3))
times.3 <- c(365,4*365,6*365)

for(i in 1:3){
   PRED.fp.mayo <- PRED.fp.gumbel(m.M=mayo.marginal, t.T=times.3[i], 
                                  theta.par=2, xi.M=mle.fp.gumbel.tau0.5.mayo$estimate[1], 
                                  omega.M=exp(mle.fp.gumbel.tau0.5.mayo$estimate[2]), 
                                  alpha.M=mle.fp.gumbel.tau0.5.mayo$estimate[3], 
                                  shape.T=exp(mle.fp.gumbel.tau0.5.mayo$estimate[4]), 
                                  scale.T=exp(mle.fp.gumbel.tau0.5.mayo$estimate[5]))
   PRED.approx <- approx(PRED.fp.mayo$F.M.sn, PRED.fp.mayo$Risk.MT,
                         xout = q.roc.plot)
   lines(PRED.approx$x, PRED.approx$y, col=2)
   PRED.fp.mayo.3plots[,i] <- PRED.approx$y
}

#Bootstrap for confidence bounds
boot.PRED.fp.mayo <- array(1:(200*3*23), dim=c(200, 3, 23))
times.3 <- c(365,4*365,6*365)
for(i in 1:200){
   lim.mark.boot <- qsn(c(0.001,0.999), 
                        xi=estimates.boot.fp.mayo[i,1], 
                        omega=exp(estimates.boot.fp.mayo[i,2]), 
                        alpha=estimates.boot.fp.mayo[i,3])
   for(j in 1:3){
      PRED.fp.mayo.boot <- PRED.fp.gumbel(m.M=seq(from=lim.mark.boot[1], 
                                                  to=lim.mark.boot[2], length=225), 
                                          t.T=times.3[j],
                                          theta.par=2, xi.M=estimates.boot.fp.mayo[i,1], 
                                          omega.M=exp(estimates.boot.fp.mayo[i,2]), 
                                          alpha.M=estimates.boot.fp.mayo[i,3], 
                                          shape.T=exp(estimates.boot.fp.mayo[i,4]), 
                                          scale.T=exp(estimates.boot.fp.mayo[i,5]))
      PRED.approx <- approx(PRED.fp.mayo.boot$F.M.sn, PRED.fp.mayo.boot$Risk.MT,
                            xout = q.roc.plot)
      boot.PRED.fp.mayo[i,j,] <- PRED.approx$y
   }}

#Boostrap quantiles
q.boot.PRED.fp.mayo <- apply(boot.PRED.fp.mayo, 
                             2:3, quantile, probs=c(0.025,0.975))

#Inferior and superior limits
roc.sup.PRED.fp.mayo.3plots <- matrix(1:(23*3), ncol=3)
roc.inf.PRED.fp.mayo.3plots <- matrix(1:(23*3), ncol=3)
par(mfrow=c(1,3))
for(i in 1:3){
   plot(q.roc.plot, PRED.fp.mayo.3plots[,i], 
        type="l", ylim=c(0,1))
   roc.sup.PRED.fp.mayo.3plots[,i] <- 2*PRED.fp.mayo.3plots[,i] - 
      q.boot.PRED.fp.mayo[1,i,]
   roc.inf.PRED.fp.mayo.3plots[,i] <- 2*PRED.fp.mayo.3plots[,i] - 
      q.boot.PRED.fp.mayo[2,i,]
   lines(q.roc.plot, roc.sup.PRED.fp.mayo.3plots[,i], 
         lwd=2, col="gray")
   lines(q.roc.plot, roc.inf.PRED.fp.mayo.3plots[,i], 
         lwd=2, col="gray")
   lines(q.roc.plot, PRED.fp.mayo.3plots[,i])
}



### Fully-parametric Total Gain

In [ ]:
%%R

#Point estimate
STG.fp.mayo <- 1:30
for(i in 1:30){
   STG.fp.mayo[i] <- std.total.gain.fp.gumbel(m.M=mayo.marginal, t.T=time.marginal[i], 
                                              theta.par=2, xi.M=mle.fp.gumbel.tau0.5.mayo$estimate[1], 
                                              omega.M=exp(mle.fp.gumbel.tau0.5.mayo$estimate[2]), 
                                              alpha.M=mle.fp.gumbel.tau0.5.mayo$estimate[3], 
                                              shape.T=exp(mle.fp.gumbel.tau0.5.mayo$estimate[4]), 
                                              scale.T=exp(mle.fp.gumbel.tau0.5.mayo$estimate[5]))
}

#Bootstrap for confidence bounds
boot.STG.fp.mayo <- matrix(1:(200*30), ncol=30)
for(i in 1:200){
   lim.mark.boot <- qsn(c(0.001,0.999), 
                        xi=estimates.boot.fp.mayo[i,1], 
                        omega=exp(estimates.boot.fp.mayo[i,2]), 
                        alpha=estimates.boot.fp.mayo[i,3])
   for(j in 1:30){
      stg.t.i <- std.total.gain.fp.gumbel(m.M=seq(from=lim.mark.boot[1], 
                                                  to=lim.mark.boot[2], length=225), 
                                          t.T=time.marginal[j], 
                                          theta.par=2, xi.M=estimates.boot.fp.mayo[i,1], 
                                          omega.M=exp(estimates.boot.fp.mayo[i,2]), 
                                          alpha.M=estimates.boot.fp.mayo[i,3], 
                                          shape.T=exp(estimates.boot.fp.mayo[i,4]), 
                                          scale.T=exp(estimates.boot.fp.mayo[i,5]))
      boot.STG.fp.mayo[i,j] <- stg.t.i
   }
}

#Boostrap quantiles
quantile.boot.STG.fp.mayo <- apply(boot.STG.fp.mayo, 
                                   2, quantile, probs=c(0.025,0.975))

#Inferior and superior limits
sup.STG.fp.mayo <- 2*STG.fp.mayo - quantile.boot.STG.fp.mayo[1,]
inf.STG.fp.mayo <- 2*STG.fp.mayo - quantile.boot.STG.fp.mayo[2,]

#Total Gain plot
plot(time.marginal, STG.fp.mayo, type="l", ylim=c(0,1))
lines(time.marginal, inf.STG.fp.mayo, col="gray", lwd=2)
lines(time.marginal, sup.STG.fp.mayo, col="gray", lwd=2)




### Semi-parametric Cumulative/Dynamic AUC

In [ ]:
%%R


#Point estimate
auc.CD.sp.mayo <- 1:30
tp.cd.sp.mayo <- NULL
fp.cd.sp.mayo <- NULL

for(i in 1:30){
   roc.CD.t.i <- ROC.CD.sp.gumbel(c.M=mayo.marginal, t.T=time.marginal[i],
                                  theta.par=2, 
                                  xi.M=estimates.sn.mayo[1], 
                                  omega.M=estimates.sn.mayo[2], 
                                  alpha.M=estimates.sn.mayo[3],
                                  times.np=KM.mayo.obj$time, 
                                  F.T.np=1 - KM.mayo.obj$surv, KM.mayo.obj$n.event)
   auc.CD.sp.mayo[i] <- auc(roc.CD.t.i$FP.D, roc.CD.t.i$TP.C)
}

# Bootstrap procedure
sample.size <- 312
boot.auc.CD.sp.mayo <- matrix(1:(200*30), ncol=30)
for(i in 1:200){
   estimated.C.sp.mayo.w.dependence <- mvdc(gumbelCopula(2, dim= 2), 
                                            c("unif", "unif"), 
                                            list(list(min = 0, max = 1), 
                                                 list(min = 0, max = 1)), 
                                            marginsIdentical = T)
   sim.U.C <- rMvdc(sample.size, estimated.C.sp.mayo.w.dependence)
   simulated.marker.boot <- qsn(sim.U.C[,1], 
                                xi=estimates.sn.mayo[1], 
                                omega=estimates.sn.mayo[2], 
                                alpha=estimates.sn.mayo[3])
   fit.sn.sim.boot <- selm(marker~1, family="SN", 
                           data=data.frame(marker=simulated.marker.boot))
   estimates.sn.sim.boot <- coef(fit.sn.sim.boot, "DP")
   lim.mark.boot <- qsn(c(0.001,0.999), 
                        xi=estimates.sn.sim.boot[1], 
                        omega=estimates.sn.sim.boot[2], 
                        alpha=estimates.sn.sim.boot[3])
   
   cat("n.times=", length(KM.sim.boot$time), "\n")
   cat("n.event=", length(KM.sim.boot$n.event), "\n")
   for(j in 1:30){
      roc.t.i <- ROC.CD.sp.gumbel(c.M=seq(from=lim.mark.boot[1], 
                                          to=lim.mark.boot[2], length=41), 
                                  t.T=time.marginal[j],
                                  theta.par=2, 
                                  xi.M=estimates.sn.sim.boot[1], 
                                  omega.M=estimates.sn.sim.boot[2], 
                                  alpha.M=estimates.sn.sim.boot[3],
                                  times.np=KM.sim.boot$time, 
                                  F.T.np=1 - KM.sim.boot$surv, KM.sim.boot$n.event)
      boot.auc.CD.sp.mayo[i,j] <- auc(roc.t.i$FP.D, roc.t.i$TP.C)
   }
}

#Boostrap quantiles
quantile.boot.auc.CD.sp.mayo <- apply(boot.auc.CD.sp.mayo, 
                                      2, quantile, probs=c(0.025,0.975))

#Inferior and superior limits
sup.auc.CD.sp.mayo <- 2*auc.CD.sp.mayo - quantile.boot.auc.CD.sp.mayo[1,]
inf.auc.CD.sp.mayo <- 2*auc.CD.sp.mayo - quantile.boot.auc.CD.sp.mayo[2,]

#AUC plot
plot(time.marginal, auc.CD.sp.mayo, type="l", ylim=c(0.7,1))
lines(time.marginal, inf.auc.CD.sp.mayo, col="gray", lwd=2)
lines(time.marginal, sup.auc.CD.sp.mayo, col="gray", lwd=2)
lines(time.marginal, auc.CD.sp.mayo, lwd=2)

### Semi-parametric Cumulative/Dynamic ROC curves

In [ ]:
%%R

#Point estimate
roc.CD.sp.mayo.3plots <- matrix(1:(23*3), ncol=3)
actual.roc.CD.sp.mayo.3plots <- matrix(1:(227*3), ncol=3)
aqtual.q.roc.CD.sp.mayo.3plots <- matrix(1:(227*3), ncol=3)

par(mfrow=c(1,3))
times.3 <- c(365,4*365,6*365)
for(i in 1:3){
   roc.CD.sp.mayo <- ROC.CD.sp.gumbel(c.M=mayo.marginal, t.T=times.3[i], 
                                      theta.par=2, 
                                      xi.M=estimates.sn.mayo[1], 
                                      omega.M=estimates.sn.mayo[2], 
                                      alpha.M=estimates.sn.mayo[3],
                                      times.np=KM.mayo.obj$time, 
                                      F.T.np=1 - KM.mayo.obj$surv, KM.mayo.obj$n.event)
   cat(auc(roc.CD.sp.mayo$FP.D, roc.CD.sp.mayo$TP.C),"\n")
   aqtual.q.roc.CD.sp.mayo.3plots[,i] <- roc.CD.fp.mayo$FP.D
   actual.roc.CD.sp.mayo.3plots[,i] <- roc.CD.fp.mayo$TP.C
   roc.approx <- approx(roc.CD.sp.mayo$FP.D, roc.CD.sp.mayo$TP.C,
                        xout = q.roc.plot)
   lines(roc.approx$x, roc.approx$y, col=2)
   roc.CD.sp.mayo.3plots[,i] <- roc.approx$y
}

#Bootstrap for confidence bounds
boot.ROC.CD.sp.mayo <- array(1:(200*3*50), dim=c(200, 3, 23))
times.3 <- c(365,4*365,6*365)

for(i in 1:200){
   for(j in 1:3){
      estimated.C.sp.mayo.w.dependence <- mvdc(gumbelCopula(2, dim= 2), 
                                               c("unif", "unif"), 
                                               list(list(min = 0, max = 1), 
                                                    list(min = 0, max = 1)), 
                                               marginsIdentical = T)
      sim.U.C <- rMvdc(sample.size, estimated.C.sp.mayo.w.dependence)
      simulated.marker.boot <- qsn(sim.U.C[,1], 
                                   xi=estimates.sn.mayo[1], 
                                   omega=estimates.sn.mayo[2], 
                                   alpha=estimates.sn.mayo[3])
      fit.sn.sim.boot <- selm(marker~1, family="SN", 
                              data=data.frame(marker=simulated.marker.boot))
      estimates.sn.sim.boot <- coef(fit.sn.sim.boot, "DP")
      lim.mark.boot <- qsn(c(0.001,0.999), 
                           xi=estimates.sn.sim.boot[1], 
                           omega=estimates.sn.sim.boot[2], 
                           alpha=estimates.sn.sim.boot[3])
      ##########
      cuantil.t <- 1-sim.U.C[,2]
      t.km <- quantile(KM.mayo.obj, probs = cuantil.t, conf.int = F)
      cuantil.c <- runif(sample.size)
      C.sim <- survfit(Surv(time-0.001*vital.status, 1-vital.status) ~ 1, 
                       data = pbc.mayo)
      c.km <- quantile(C.sim, probs = cuantil.c, conf.int = F)
      t.and.c <- matrix(c(t.km, c.km), ncol=2, byrow=F)
      t.star <- apply(t.and.c, 1, min, na.rm = TRUE)
      substract.sim <- t.and.c-t.star
      cens.in.sim <- rep(0, nrow(substract.sim))
      cens.in.sim[substract.sim[,1] == 0] <- 1
      sim.set.boot <- data.frame(ztimes=t.star, status=cens.in.sim)
      KM.sim.boot <- survfit(Surv(ztimes, status) ~ 1, data = sim.set.boot)
      ##############
      cat("n.times=", length(KM.sim.boot$time), "\n")
      cat("n.event=", length(KM.sim.boot$n.event), "\n")
      roc.CD.sp.mayo.boot <- ROC.CD.sp.gumbel(c.M=seq(from=lim.mark.boot[1], 
                                                      to=lim.mark.boot[2], length=41), 
                                              t.T=times.3[j],
                                              theta.par=2, 
                                              xi.M=estimates.sn.sim.boot[1], 
                                              omega.M=estimates.sn.sim.boot[2], 
                                              alpha.M=estimates.sn.sim.boot[3],
                                              times.np=KM.sim.boot$time, 
                                              F.T.np=1 - KM.sim.boot$surv, KM.sim.boot$n.event)
      roc.approx <- approx(roc.CD.sp.mayo.boot$FP.D, roc.CD.sp.mayo.boot$TP.C,
                           xout = q.roc.plot)
      boot.ROC.CD.sp.mayo[i,j,] <- roc.approx$y
   }
}

#Boostrap quantiles
q.boot.ROC.CD.sp.mayo <- apply(boot.ROC.CD.sp.mayo, 
                               2:3, quantile, probs=c(0.025,0.975))

#Inferior and superior limits
roc.sup.CD.sp.mayo.3plots <- matrix(1:(23*3), ncol=3)
roc.inf.CD.sp.mayo.3plots <- matrix(1:(23*3), ncol=3)

#ROC curve plots
par(mfrow=c(1,3))
for(i in 1:3){
   plot(c(0, q.roc.plot, 1), c(0,roc.CD.sp.mayo.3plots[,i],1), type="l", ylim=c(0,1))
   roc.sup.CD.sp.mayo.3plots[,i] <- 2*roc.CD.sp.mayo.3plots[,i] - 
      q.boot.ROC.CD.sp.mayo[1,i,]
   roc.inf.CD.sp.mayo.3plots[,i] <- 2*roc.CD.sp.mayo.3plots[,i] - 
      q.boot.ROC.CD.sp.mayo[2,i,]
   lines(c(0,q.roc.plot,1), c(0,roc.sup.CD.sp.mayo.3plots[,i],1), 
         lwd=2, col="gray")
   lines(c(0,q.roc.plot,1), c(0,roc.inf.CD.sp.mayo.3plots[,i],1), 
         lwd=2, col="gray")
   lines(c(0, q.roc.plot, 1), c(0,roc.CD.sp.mayo.3plots[,i],1))
}


### Semi-parametric Incident/Dynamic AUC

In [ ]:
%%R

#Point estimator
auc.ID.sp.mayo <- 1:30
system.time
for(i in 1:30){
   cat(i, "\n")
   roc.ID.t.i <- ROC.ID.sp.gumbel(c.M=mayo.marginal, t.T=time.marginal[i],
                                  theta.par=2, 
                                  xi.M=estimates.sn.mayo[1], 
                                  omega.M=estimates.sn.mayo[2], 
                                  alpha.M=estimates.sn.mayo[3],
                                  times.np=KM.mayo.obj$time, 
                                  F.T.np=1 - KM.mayo.obj$surv, KM.mayo.obj$n.event)
   auc.ID.sp.mayo[i] <- auc(roc.ID.t.i$FP.D, roc.ID.t.i$TP.I)
}

# Bootstrap
sample.size <- 312
boot.auc.ID.sp.mayo <- matrix(1:(200*30), ncol=30)

for(i in 1:200){
   estimated.C.sp.mayo.w.dependence <- mvdc(gumbelCopula(2, dim= 2), 
                                            c("unif", "unif"), 
                                            list(list(min = 0, max = 1), 
                                                 list(min = 0, max = 1)), 
                                            marginsIdentical = T)
   sim.U.C <- rMvdc(sample.size, estimated.C.sp.mayo.w.dependence)
   simulated.marker.boot <- qsn(sim.U.C[,1], 
                                xi=estimates.sn.mayo[1], 
                                omega=estimates.sn.mayo[2], 
                                alpha=estimates.sn.mayo[3])
   fit.sn.sim.boot <- selm(marker~1, family="SN", 
                           data=data.frame(marker=simulated.marker.boot))
   estimates.sn.sim.boot <- coef(fit.sn.sim.boot, "DP")
   lim.mark.boot <- qsn(c(0.001,0.999), 
                        xi=estimates.sn.sim.boot[1], 
                        omega=estimates.sn.sim.boot[2], 
                        alpha=estimates.sn.sim.boot[3])
   ##
   cuantil.t <- 1-sim.U.C[,2]
   t.km <- quantile(KM.mayo.obj, probs = cuantil.t, conf.int = F)
   cuantil.c <- runif(sample.size)
   C.sim <- survfit(Surv(time-0.001*vital.status, 1-vital.status) ~ 1, 
                    data = pbc.mayo)
   c.km <- quantile(C.sim, probs = cuantil.c, conf.int = F)
   t.and.c <- matrix(c(t.km, c.km), ncol=2, byrow=F)
   t.star <- apply(t.and.c, 1, min, na.rm = TRUE)
   substract.sim <- t.and.c-t.star
   cens.in.sim <- rep(0, nrow(substract.sim))
   cens.in.sim[substract.sim[,1] == 0] <- 1
   sim.set.boot <- data.frame(ztimes=t.star, status=cens.in.sim)
   KM.sim.boot <- survfit(Surv(ztimes, status) ~ 1, data = sim.set.boot)
   ##
   cat("n.times=", length(KM.sim.boot$time), "\n")
   cat("n.event=", length(KM.sim.boot$n.event), "\n")
   for(j in 1:30){
      roc.t.i <- ROC.ID.sp.gumbel(c.M=seq(from=lim.mark.boot[1], 
                                          to=lim.mark.boot[2], length=30), 
                                  t.T=time.marginal[j],
                                  theta.par=2, 
                                  xi.M=estimates.sn.sim.boot[1], 
                                  omega.M=estimates.sn.sim.boot[2], 
                                  alpha.M=estimates.sn.sim.boot[3],
                                  times.np=KM.sim.boot$time, 
                                  F.T.np=1 - KM.sim.boot$surv, KM.sim.boot$n.event)
      boot.auc.ID.sp.mayo[i,j] <- auc(roc.t.i$FP.D, roc.t.i$TP.I)
   }
}

#Boostrap quantiles
quantile.boot.auc.ID.sp.mayo <- apply(boot.auc.ID.sp.mayo, 
                                      2, quantile, probs=c(0.025,0.975))

#Inferior and superior limits
sup.auc.ID.sp.mayo <- 2*auc.ID.sp.mayo -   quantile.boot.auc.ID.sp.mayo[1,]
inf.auc.ID.sp.mayo <- 2*auc.ID.sp.mayo -   quantile.boot.auc.ID.sp.mayo[2,]

#AUC plot
plot(time.marginal, auc.ID.sp.mayo, type="l", ylim=c(0,1))
lines(time.marginal, inf.auc.ID.sp.mayo, col="gray", lwd=2)
lines(time.marginal, sup.auc.ID.sp.mayo, col="gray", lwd=2)
lines(time.marginal, auc.ID.sp.mayo, lwd=2)

### Semi-parametric Incident/Dynamic ROC curves

In [ ]:
%%R

#Point estimate
roc.ID.sp.mayo.3plots <- matrix(1:(23*3), ncol=3)
par(mfrow=c(1,3))
times.3 <- c(365,4*365,6*365)

for(i in 1:3){
   roc.ID.sp.mayo <- ROC.ID.sp.gumbel(c.M=mayo.marginal, t.T=times.3[i],
                                      theta.par=2, 
                                      xi.M=estimates.sn.mayo[1], 
                                      omega.M=estimates.sn.mayo[2], 
                                      alpha.M=estimates.sn.mayo[3],
                                      times.np=KM.mayo.obj$time, 
                                      F.T.np=1 - KM.mayo.obj$surv, KM.mayo.obj$n.event)
   roc.approx <- approx(roc.ID.sp.mayo$FP.D, roc.ID.sp.mayo$TP.I,
                        xout = q.roc.plot)
   lines(roc.approx$x, roc.approx$y, col=2)
   roc.ID.sp.mayo.3plots[,i] <- roc.approx$y
}


#Bootstrap for confidence bounds
boot.ROC.ID.sp.mayo <- array(1:(200*3*23), dim=c(200, 3, 23))
times.3 <- c(365,4*365,6*365)

for(i in 1:200){
   for(j in 1:3){
      estimated.C.sp.mayo.w.dependence <- mvdc(gumbelCopula(2, dim= 2), 
                                               c("unif", "unif"), 
                                               list(list(min = 0, max = 1), 
                                                    list(min = 0, max = 1)), 
                                               marginsIdentical = T)
      sim.U.C <- rMvdc(sample.size, estimated.C.sp.mayo.w.dependence)
      simulated.marker.boot <- qsn(sim.U.C[,1], 
                                   xi=estimates.sn.mayo[1], 
                                   omega=estimates.sn.mayo[2], 
                                   alpha=estimates.sn.mayo[3])
      fit.sn.sim.boot <- selm(marker~1, family="SN", 
                              data=data.frame(marker=simulated.marker.boot))
      estimates.sn.sim.boot <- coef(fit.sn.sim.boot, "DP")
      lim.mark.boot <- qsn(c(0.001,0.999), 
                           xi=estimates.sn.sim.boot[1], 
                           omega=estimates.sn.sim.boot[2], 
                           alpha=estimates.sn.sim.boot[3])
      #
      cuantil.t <- 1-sim.U.C[,2]
      t.km <- quantile(KM.mayo.obj, probs = cuantil.t, conf.int = F)
      cuantil.c <- runif(sample.size)
      C.sim <- survfit(Surv(time-0.001*vital.status, 1-vital.status) ~ 1, 
                       data = pbc.mayo)
      c.km <- quantile(C.sim, probs = cuantil.c, conf.int = F)
      t.and.c <- matrix(c(t.km, c.km), ncol=2, byrow=F)
      t.star <- apply(t.and.c, 1, min, na.rm = TRUE)
      substract.sim <- t.and.c-t.star
      cens.in.sim <- rep(0, nrow(substract.sim))
      cens.in.sim[substract.sim[,1] == 0] <- 1
      sim.set.boot <- data.frame(ztimes=t.star, status=cens.in.sim)
      KM.sim.boot <- survfit(Surv(ztimes, status) ~ 1, data = sim.set.boot)
      #
      cat("n.times=", length(KM.sim.boot$time), "\n")
      cat("n.event=", length(KM.sim.boot$n.event), "\n")
      roc.ID.sp.mayo.boot <- ROC.ID.sp.gumbel(c.M=seq(from=lim.mark.boot[1], 
                                                      to=lim.mark.boot[2], length=225), 
                                              t.T=times.3[j],
                                              theta.par=2, 
                                              xi.M=estimates.sn.sim.boot[1], 
                                              omega.M=estimates.sn.sim.boot[2], 
                                              alpha.M=estimates.sn.sim.boot[3],
                                              times.np=KM.sim.boot$time, 
                                              F.T.np=1 - KM.sim.boot$surv, KM.sim.boot$n.event)
      roc.approx <- approx(roc.ID.sp.mayo.boot$FP.D, roc.ID.sp.mayo.boot$TP.I,
                           xout = q.roc.plot)
      boot.ROC.ID.sp.mayo[i,j,] <- roc.approx$y
   }
}

#Boostrap quantiles
q.boot.ROC.ID.sp.mayo <- apply(boot.ROC.ID.sp.mayo, 2:3, quantile, probs=c(0.025,0.975))

#Inferior and superior limits
roc.sup.ID.sp.mayo.3plots <- matrix(1:(23*3), ncol=3)
roc.inf.ID.sp.mayo.3plots <- matrix(1:(23*3), ncol=3)

#ROC curve plots
par(mfrow=c(1,3))
for(i in 1:3){
   plot(c(0, q.roc.plot, 1), c(0,roc.ID.sp.mayo.3plots[,i],1), type="l", ylim=c(0,1))
   roc.sup.ID.sp.mayo.3plots[,i] <- 2*roc.ID.sp.mayo.3plots[,i] - 
      q.boot.ROC.ID.sp.mayo[1,i,]
   roc.inf.ID.sp.mayo.3plots[,i] <- 2*roc.ID.sp.mayo.3plots[,i] - 
      q.boot.ROC.ID.sp.mayo[2,i,]
   lines(c(0,q.roc.plot,1), c(0,roc.sup.ID.sp.mayo.3plots[,i],1), 
         lwd=2, col="gray")
   lines(c(0,q.roc.plot,1), c(0,roc.inf.ID.sp.mayo.3plots[,i],1), 
         lwd=2, col="gray")
   lines(c(0, q.roc.plot, 1), c(0,roc.ID.sp.mayo.3plots[,i],1))
}



### Semi-parametric Predictiveness curves

In [ ]:
%%R


#Point estimate
PRED.sp.mayo.3plots <- matrix(1:(23*3), ncol=3)
par(mfrow=c(1,3))
times.3 <- c(365,4*365,6*365)

for(i in 1:3){
   PRED.sp.mayo <- PRED.sp.gumbel(c.M=mayo.marginal, t.T=times.3[i],
                                  theta.par=2, 
                                  xi.M=estimates.sn.mayo[1], 
                                  omega.M=estimates.sn.mayo[2], 
                                  alpha.M=estimates.sn.mayo[3],
                                  times.np=KM.mayo.obj$time, 
                                  F.T.np=1 - KM.mayo.obj$surv, KM.mayo.obj$n.event)
   PRED.approx <- approx(PRED.sp.mayo$F.M, PRED.sp.mayo$Risk.MT,
                         xout = q.roc.plot)
   lines(PRED.approx$x, PRED.approx$y, col=2)
   PRED.sp.mayo.3plots[,i] <- PRED.approx$y
}

#Bootstrap for confidence bounds
boot.PRED.sp.mayo <- array(1:(200*3*23), dim=c(200, 3, 23))
times.3 <- c(365,4*365,6*365)

for(i in 1:200){
   for(j in 1:3){
      estimated.C.sp.mayo.w.dependence <- mvdc(gumbelCopula(2, dim= 2), 
                                               c("unif", "unif"), 
                                               list(list(min = 0, max = 1), 
                                                    list(min = 0, max = 1)), 
                                               marginsIdentical = T)
      sim.U.C <- rMvdc(sample.size, estimated.C.sp.mayo.w.dependence)
      simulated.marker.boot <- qsn(sim.U.C[,1], 
                                   xi=estimates.sn.mayo[1], 
                                   omega=estimates.sn.mayo[2], 
                                   alpha=estimates.sn.mayo[3])
      fit.sn.sim.boot <- selm(marker~1, family="SN", 
                              data=data.frame(marker=simulated.marker.boot))
      estimates.sn.sim.boot <- coef(fit.sn.sim.boot, "DP")
      lim.mark.boot <- qsn(c(0.001,0.999), 
                           xi=estimates.sn.sim.boot[1], 
                           omega=estimates.sn.sim.boot[2], 
                           alpha=estimates.sn.sim.boot[3])
      #
      cuantil.t <- 1-sim.U.C[,2]
      t.km <- quantile(KM.mayo.obj, probs = cuantil.t, conf.int = F)
      cuantil.c <- runif(sample.size)
      C.sim <- survfit(Surv(time-0.001*vital.status, 1-vital.status) ~ 1, 
                       data = pbc.mayo)
      c.km <- quantile(C.sim, probs = cuantil.c, conf.int = F)
      t.and.c <- matrix(c(t.km, c.km), ncol=2, byrow=F)
      t.star <- apply(t.and.c, 1, min, na.rm = TRUE)
      substract.sim <- t.and.c-t.star
      cens.in.sim <- rep(0, nrow(substract.sim))
      cens.in.sim[substract.sim[,1] == 0] <- 1
      sim.set.boot <- data.frame(ztimes=t.star, status=cens.in.sim)
      KM.sim.boot <- survfit(Surv(ztimes, status) ~ 1, data = sim.set.boot)
      #
      cat("n.times=", length(KM.sim.boot$time), "\n")
      cat("n.event=", length(KM.sim.boot$n.event), "\n")
      PRED.sp.mayo.boot <- PRED.sp.gumbel(c.M=seq(from=lim.mark.boot[1], 
                                                  to=lim.mark.boot[2], length=225), 
                                          t.T=times.3[j],
                                          theta.par=2, 
                                          xi.M=estimates.sn.sim.boot[1], 
                                          omega.M=estimates.sn.sim.boot[2], 
                                          alpha.M=estimates.sn.sim.boot[3],
                                          times.np=KM.sim.boot$time, 
                                          F.T.np=1 - KM.sim.boot$surv, KM.sim.boot$n.event)
      roc.approx <- approx(PRED.sp.mayo.boot$F.M, PRED.sp.mayo.boot$Risk.MT,
                           xout = q.roc.plot)
      boot.PRED.sp.mayo[i,j,] <- roc.approx$y
   }
}

#Boostrap quantiles
q.boot.PRED.sp.mayo <- apply(boot.PRED.sp.mayo, 
                             2:3, quantile, probs=c(0.025,0.975))
#Inferior and superior limits
roc.sup.PRED.sp.mayo.3plots <- matrix(1:(23*3), ncol=3)
roc.inf.PRED.sp.mayo.3plots <- matrix(1:(23*3), ncol=3)

#Predictiveness curve plots
par(mfrow=c(1,3))
for(i in 1:3){
   plot(q.roc.plot, PRED.sp.mayo.3plots[,i], type="l", ylim=c(0,1))
   roc.sup.PRED.sp.mayo.3plots[,i] <- 2*PRED.sp.mayo.3plots[,i] - 
      q.boot.PRED.sp.mayo[1,i,]
   roc.inf.PRED.sp.mayo.3plots[,i] <- 2*PRED.sp.mayo.3plots[,i] - 
      q.boot.PRED.sp.mayo[2,i,]
   lines(q.roc.plot, roc.sup.PRED.sp.mayo.3plots[,i], 
         lwd=2, col="gray")
   lines(q.roc.plot, roc.inf.PRED.sp.mayo.3plots[,i], 
         lwd=2, col="gray")
   lines(q.roc.plot, PRED.sp.mayo.3plots[,i])
}



### Semi-parametric Total Gain

In [ ]:
%%R

#Point estimate
STG.sp.mayo <- 1:30
for(i in 1:30){
   STG.sp.mayo[i] <- std.total.gain.sp.gumbel(c.M=mayo.marginal, 
                                              t.T=time.marginal[i], 
                                              theta.par=2, 
                                              xi.M=estimates.sn.mayo[1], 
                                              omega.M=estimates.sn.mayo[2], 
                                              alpha.M=estimates.sn.mayo[3],
                                              times.np=KM.mayo.obj$time, 
                                              F.T.np=1 - KM.mayo.obj$surv, KM.mayo.obj$n.event)
}

# Bootstrap for confidence bounds
sample.size <- 312
boot.STG.sp.mayo <- matrix(1:(200*30), ncol=30)

for(i in 1:200){
   estimated.C.sp.mayo.w.dependence <- mvdc(gumbelCopula(2, dim= 2), 
                                            c("unif", "unif"), 
                                            list(list(min = 0, max = 1), 
                                                 list(min = 0, max = 1)), 
                                            marginsIdentical = T)
   sim.U.C <- rMvdc(sample.size, estimated.C.sp.mayo.w.dependence)
   simulated.marker.boot <- qsn(sim.U.C[,1], 
                                xi=estimates.sn.mayo[1], 
                                omega=estimates.sn.mayo[2], 
                                alpha=estimates.sn.mayo[3])
   fit.sn.sim.boot <- selm(marker~1, family="SN", 
                           data=data.frame(marker=simulated.marker.boot))
   estimates.sn.sim.boot <- coef(fit.sn.sim.boot, "DP")
   lim.mark.boot <- qsn(c(0.001,0.999), 
                        xi=estimates.sn.sim.boot[1], 
                        omega=estimates.sn.sim.boot[2], 
                        alpha=estimates.sn.sim.boot[3])
   #
   cuantil.t <- 1-sim.U.C[,2]
   t.km <- quantile(KM.mayo.obj, probs = cuantil.t, conf.int = F)
   cuantil.c <- runif(sample.size)
   C.sim <- survfit(Surv(time-0.001*vital.status, 1-vital.status) ~ 1, 
                    data = pbc.mayo)
   c.km <- quantile(C.sim, probs = cuantil.c, conf.int = F)
   t.and.c <- matrix(c(t.km, c.km), ncol=2, byrow=F)
   t.star <- apply(t.and.c, 1, min, na.rm = TRUE)
   substract.sim <- t.and.c-t.star
   cens.in.sim <- rep(0, nrow(substract.sim))
   cens.in.sim[substract.sim[,1] == 0] <- 1
   sim.set.boot <- data.frame(ztimes=t.star, status=cens.in.sim)
   KM.sim.boot <- survfit(Surv(ztimes, status) ~ 1, data = sim.set.boot)
   #
   cat("n.times=", length(KM.sim.boot$time), "\n")
   cat("n.event=", length(KM.sim.boot$n.event), "\n")
   for(j in 1:30){
      boot.STG.sp.mayo[i,j] <- std.total.gain.sp.gumbel(c.M=seq(from=lim.mark.boot[1], 
                                                                to=lim.mark.boot[2], length=50), 
                                                        t.T=time.marginal[j],
                                                        theta.par=2, 
                                                        xi.M=estimates.sn.sim.boot[1], 
                                                        omega.M=estimates.sn.sim.boot[2], 
                                                        alpha.M=estimates.sn.sim.boot[3],
                                                        times.np=KM.sim.boot$time, 
                                                        F.T.np=1 - KM.sim.boot$surv, KM.sim.boot$n.event)
   }
}

#Boostrap quantiles
quantile.boot.STG.sp.mayo <- apply(boot.STG.sp.mayo, 
                                   2, quantile, probs=c(0.025,0.975))
#Inferior and superior limits
sup.STG.sp.mayo <- 2*STG.sp.mayo -  quantile.boot.STG.sp.mayo[1,]
inf.STG.sp.mayo <- 2*STG.sp.mayo - quantile.boot.STG.sp.mayo[2,]

#Total Gain plot
plot(time.marginal, STG.sp.mayo, type="l", ylim=c(0,1))
lines(time.marginal, inf.STG.sp.mayo, col="gray", lwd=2)
lines(time.marginal, sup.STG.sp.mayo, col="gray", lwd=2)
lines(time.marginal, STG.sp.mayo, lwd=2)


# Section 5: Final Graphical objects

### ROC and Predictiveness plots 

In [ ]:
%%R

# ROC Cumulative/Dynamic: Two biomarkers, two approaches, three times:

q.axis <- rep(c(0, q.roc.plot, 1), 2*2)
type.marker.time <- factor(rep(2:1, each=2*25), labels=c("mayo", "4cov"))
appr.time <- factor(c(rep(1, 25), rep(2, 25), 
                      rep(1, 25), rep(2, 25)),
                    labels=c("Parametric", "Semi-parametric"))

roc.CD.time.1 <- c(c(0,roc.CD.fp.4cov.3plots[,1],1), c(0,roc.CD.sp.4cov.3plots[,1],1),
                   c(0,roc.CD.fp.mayo.3plots[,1],1), c(0,roc.CD.sp.mayo.3plots[,1],1))
roc.sup.CD.time.1 <- c(c(0,roc.sup.CD.fp.4cov.3plots[,1],1), c(0,roc.sup.CD.sp.4cov.3plots[,1],1),
                       c(0,roc.sup.CD.fp.mayo.3plots[,1],1), c(0,roc.sup.CD.sp.mayo.3plots[,1],1))
roc.inf.CD.time.1 <- c(c(0,roc.inf.CD.fp.4cov.3plots[,1],1), c(0,roc.inf.CD.sp.4cov.3plots[,1],1),
                       c(0,roc.inf.CD.fp.mayo.3plots[,1],1), c(0,roc.inf.CD.sp.mayo.3plots[,1],1))

roc.CD.1 <- xyplot(roc.inf.CD.time.1 + roc.CD.time.1 + roc.sup.CD.time.1 ~ q.axis | appr.time*type.marker.time, 
                   type="l", ylim=c(0,1), col=c("gray49", 1, "gray49"), 
                   lwd=c(1.5, 1.5, 1.5), lty=c(1,1,1), xlab="q", 
                   ylab=expression(paste("ROC"^"C/D", "(q, 365)")),
                   par.settings = list(strip.background=list(col="grey98")))

roc.CD.time.2 <- c(c(0,roc.CD.fp.4cov.3plots[,2],1), c(0,roc.CD.sp.4cov.3plots[,2],1),
                   c(0,roc.CD.fp.mayo.3plots[,2],1), c(0,roc.CD.sp.mayo.3plots[,2],1))
roc.sup.CD.time.2 <- c(c(0,roc.sup.CD.fp.4cov.3plots[,2],1), c(0,roc.sup.CD.sp.4cov.3plots[,2],1),
                       c(0,roc.sup.CD.fp.mayo.3plots[,2],1), c(0,roc.sup.CD.sp.mayo.3plots[,2],1))
roc.inf.CD.time.2 <- c(c(0,roc.inf.CD.fp.4cov.3plots[,2],1), c(0,roc.inf.CD.sp.4cov.3plots[,2],1),
                       c(0,roc.inf.CD.fp.mayo.3plots[,2],1), c(0,roc.inf.CD.sp.mayo.3plots[,2],1))

roc.CD.2 <- xyplot(roc.inf.CD.time.2 + roc.CD.time.2 + roc.sup.CD.time.2 ~ q.axis | appr.time*type.marker.time, 
                   type="l", ylim=c(0,1), col=c("gray49", 1, "gray49"), 
                   lwd=c(1.5, 1.5, 1.5), lty=c(1,1,1), xlab="q", 
                   ylab=expression(paste("ROC"^"C/D", "(q, 1460)")),
                   par.settings = list(strip.background=list(col="grey98")))

roc.CD.time.3 <- c(c(0,roc.CD.fp.4cov.3plots[,3],1), c(0,roc.CD.sp.4cov.3plots[,3],1),
                   c(0,roc.CD.fp.mayo.3plots[,3],1), c(0,roc.CD.sp.mayo.3plots[,3],1))
roc.sup.CD.time.3 <- c(c(0,roc.sup.CD.fp.4cov.3plots[,3],1), c(0,roc.sup.CD.sp.4cov.3plots[,3],1),
                       c(0,roc.sup.CD.fp.mayo.3plots[,3],1), c(0,roc.sup.CD.sp.mayo.3plots[,3],1))
roc.inf.CD.time.3 <- c(c(0,roc.inf.CD.fp.4cov.3plots[,3],1), c(0,roc.inf.CD.sp.4cov.3plots[,3],1),
                       c(0,roc.inf.CD.fp.mayo.3plots[,3],1), c(0,roc.inf.CD.sp.mayo.3plots[,3],1))

roc.CD.3 <- xyplot(roc.inf.CD.time.3 + roc.CD.time.3 + roc.sup.CD.time.3 ~ q.axis | appr.time*type.marker.time, 
                   type="l", ylim=c(0,1), col=c("gray49", 1, "gray49"), 
                   lwd=c(1.5, 1.5, 1.5), lty=c(1,1,1), xlab="q", 
                   ylab=expression(paste("ROC"^"C/D", "(q, 2190)")),
                   par.settings = list(strip.background=list(col="grey98")))

# ROC Incident/Dynamic: Two biomarkers, two approaches, three times:

roc.ID.time.1 <- c(c(0,roc.ID.fp.4cov.3plots[,1],1), c(0,roc.ID.sp.4cov.3plots[,1],1),
                   c(0,roc.ID.fp.mayo.3plots[,1],1), c(0,roc.ID.sp.mayo.3plots[,1],1))
roc.sup.ID.time.1 <- c(c(0,roc.sup.ID.fp.4cov.3plots[,1],1), c(0,roc.sup.ID.sp.4cov.3plots[,1],1),
                       c(0,roc.sup.ID.fp.mayo.3plots[,1],1), c(0,roc.sup.ID.sp.mayo.3plots[,1],1))
roc.inf.ID.time.1 <- c(c(0,roc.inf.ID.fp.4cov.3plots[,1],1), c(0,roc.inf.ID.sp.4cov.3plots[,1],1),
                       c(0,roc.inf.ID.fp.mayo.3plots[,1],1), c(0,roc.inf.ID.sp.mayo.3plots[,1],1))

roc.ID.1 <- xyplot(roc.inf.ID.time.1 + roc.ID.time.1 + roc.sup.ID.time.1 ~ q.axis | appr.time*type.marker.time, 
                   type="l", ylim=c(0,1), col=c("gray49", 1, "gray49"), 
                   lwd=c(1.5, 1.5, 1.5), lty=c(1,1,1), xlab="q", 
                   ylab=expression(paste("ROC"^"I/D", "(q, 365)")),
                   par.settings = list(strip.background=list(col="grey98")))

roc.ID.time.2 <- c(c(0,roc.ID.fp.4cov.3plots[,2],1), c(0,roc.ID.sp.4cov.3plots[,2],1),
                   c(0,roc.ID.fp.mayo.3plots[,2],1), c(0,roc.ID.sp.mayo.3plots[,2],1))
roc.sup.ID.time.2 <- c(c(0,roc.sup.ID.fp.4cov.3plots[,2],1), c(0,roc.sup.ID.sp.4cov.3plots[,2],1),
                       c(0,roc.sup.ID.fp.mayo.3plots[,2],1), c(0,roc.sup.ID.sp.mayo.3plots[,2],1))
roc.inf.ID.time.2 <- c(c(0,roc.inf.ID.fp.4cov.3plots[,2],1), c(0,roc.inf.ID.sp.4cov.3plots[,2],1),
                       c(0,roc.inf.ID.fp.mayo.3plots[,2],1), c(0,roc.inf.ID.sp.mayo.3plots[,2],1))

roc.ID.2 <- xyplot(roc.inf.ID.time.2 + roc.ID.time.2 + roc.sup.ID.time.2 ~ q.axis | appr.time*type.marker.time, 
                   type="l", ylim=c(0,1), col=c("gray49", 1, "gray49"), 
                   lwd=c(1.5, 1.5, 1.5), lty=c(1,1,1), xlab="q", 
                   ylab=expression(paste("ROC"^"I/D", "(q, 1460)")),
                   par.settings = list(strip.background=list(col="grey98")))

roc.ID.time.3 <- c(c(0,roc.ID.fp.4cov.3plots[,3],1), c(0,roc.ID.sp.4cov.3plots[,3],1),
                   c(0,roc.ID.fp.mayo.3plots[,3],1), c(0,roc.ID.sp.mayo.3plots[,3],1))
roc.sup.ID.time.3 <- c(c(0,roc.sup.ID.fp.4cov.3plots[,3],1), c(0,roc.sup.ID.sp.4cov.3plots[,3],1),
                       c(0,roc.sup.ID.fp.mayo.3plots[,3],1), c(0,roc.sup.ID.sp.mayo.3plots[,3],1))
roc.inf.ID.time.3 <- c(c(0,roc.inf.ID.fp.4cov.3plots[,3],1), c(0,roc.inf.ID.sp.4cov.3plots[,3],1),
                       c(0,roc.inf.ID.fp.mayo.3plots[,3],1), c(0,roc.inf.ID.sp.mayo.3plots[,3],1))

roc.ID.3 <- xyplot(roc.inf.ID.time.3 + roc.ID.time.3 + roc.sup.ID.time.3 ~ q.axis | appr.time*type.marker.time, 
                   type="l", ylim=c(0,1), col=c("gray49", 1, "gray49"), 
                   lwd=c(1.5, 1.5, 1.5), lty=c(1,1,1), xlab="q", 
                   ylab=expression(paste("ROC"^"I/D", "(q, 2190)")),
                   par.settings = list(strip.background=list(col="grey98")))

# Predictiveness Incident/Dynamic: Two biomarkers, two approaches, three times:

q.axis.PRED <- rep(q.roc.plot, 2*2)
type.marker.time.PRED <- factor(rep(2:1, each=2*23), labels=c("mayo", "4cov"))
appr.time.PRED <- factor(c(rep(1, 23), rep(2, 23), 
                           rep(1, 23), rep(2, 23)),
                         labels=c("Parametric", "Semi-parametric"))


PRED.time.1 <- c(PRED.fp.4cov.3plots[,1], PRED.sp.4cov.3plots[,1],
                 PRED.fp.mayo.3plots[,1], PRED.sp.mayo.3plots[,1])
sup.PRED.time.1 <- c(roc.sup.PRED.fp.4cov.3plots[,1],
                     pred.sup.sp.4cov.3plots[,1],
                     roc.sup.PRED.sp.mayo.3plots[,1],
                     roc.sup.PRED.sp.mayo.3plots[,1])
inf.PRED.time.1 <- c(roc.inf.PRED.fp.4cov.3plots[,1],
                     pred.inf.sp.4cov.3plots[,1],
                     roc.inf.PRED.sp.mayo.3plots[,1],
                     roc.inf.PRED.sp.mayo.3plots[,1])

PRED.1 <- xyplot(inf.PRED.time.1+ PRED.time.1 +
                    sup.PRED.time.1 ~ q.axis.PRED | appr.time.PRED*type.marker.time.PRED, 
                 type="l", ylim=c(0,1), col=c("gray49", 1, "gray49"), 
                 lwd=c(1.5, 1.5, 1.5), lty=c(1,1,1), xlab="q", 
                 ylab=expression(italic(R)(q, 365)),
                 par.settings = list(strip.background=list(col="grey98")))

PRED.time.2 <- c(PRED.fp.4cov.3plots[,2], PRED.sp.4cov.3plots[,2],
                 PRED.fp.mayo.3plots[,2], PRED.sp.mayo.3plots[,2])
sup.PRED.time.2 <- c(roc.sup.PRED.fp.4cov.3plots[,2],
                     pred.sup.sp.4cov.3plots[,2],
                     roc.sup.PRED.sp.mayo.3plots[,2],
                     roc.sup.PRED.sp.mayo.3plots[,2])
inf.PRED.time.2 <- c(roc.inf.PRED.fp.4cov.3plots[,2],
                     pred.inf.sp.4cov.3plots[,2],
                     roc.inf.PRED.sp.mayo.3plots[,2],
                     roc.inf.PRED.sp.mayo.3plots[,2])

PRED.2 <- xyplot(inf.PRED.time.2+ PRED.time.2 +
                    sup.PRED.time.2 ~ q.axis.PRED | appr.time.PRED*type.marker.time.PRED, 
                 type="l", ylim=c(0,1), col=c("gray49", 1, "gray49"), 
                 lwd=c(1.5, 1.5, 1.5), lty=c(1,1,1), xlab="q", 
                 ylab=expression(italic(R)(q, 1460)),
                 par.settings = list(strip.background=list(col="grey98")))

PRED.time.3 <- c(PRED.fp.4cov.3plots[,3], PRED.sp.4cov.3plots[,3],
                 PRED.fp.mayo.3plots[,3], PRED.sp.mayo.3plots[,3])
sup.PRED.time.3 <- c(roc.sup.PRED.fp.4cov.3plots[,3],
                     pred.sup.sp.4cov.3plots[,3],
                     roc.sup.PRED.sp.mayo.3plots[,3],
                     roc.sup.PRED.sp.mayo.3plots[,3])
inf.PRED.time.3 <- c(roc.inf.PRED.fp.4cov.3plots[,3],
                     pred.inf.sp.4cov.3plots[,3],
                     roc.inf.PRED.sp.mayo.3plots[,3],
                     roc.inf.PRED.sp.mayo.3plots[,3])

PRED.3 <- xyplot(inf.PRED.time.3+ PRED.time.3 +
                    sup.PRED.time.3 ~ q.axis.PRED | appr.time.PRED*type.marker.time.PRED, 
                 type="l", ylim=c(0,1), col=c("gray49", 1, "gray49"), 
                 lwd=c(1.5, 1.5, 1.5), lty=c(1,1,1), xlab="q", 
                 ylab=expression(italic(R)(q, 2190)),
                 par.settings = list(strip.background=list(col="grey98")))

#Final plots

#Graphical object with CD ROC curves plots
grid.arrange(roc.CD.1, roc.CD.2, roc.CD.3, ncol=3) 

#Graphical object with IC ROC curves plots
grid.arrange(roc.ID.1, roc.ID.2, roc.ID.3, ncol=3)

#Graphical object with predictiveness plots
grid.arrange(PRED.1, PRED.2, PRED.3, ncol=3)


#Graphical object to create figure 2 (ROC curves and predictiveness curves)
grid.arrange(roc.CD.1, roc.CD.2, roc.CD.3, 
             roc.ID.1, roc.ID.2, roc.ID.3,
             PRED.1, PRED.2, PRED.3, ncol=3)


### AUC and STG plots 

In [ ]:
%%R 

marker.is <- factor(rep(2:1, each=60), labels=c("mayo", "4cov"))
appr <- factor(c(rep(1, 30), rep(2, 30), rep(1, 30), rep(2, 30)),
               labels=c("Parametric", "Semi-parametric"))

auc.CD.all <- c(auc.CD.fp.4cov, auc.CD.sp.4cov, auc.CD.fp.mayo, auc.CD.sp.mayo)
inf.auc.CD.all <- c(inf.auc.CD.fp.4cov, inf.auc.CD.sp.4cov, inf.auc.CD.fp.mayo, inf.auc.CD.sp.mayo)
sup.auc.CD.all <- c(sup.auc.CD.fp.4cov, sup.auc.CD.sp.4cov, sup.auc.CD.fp.mayo, sup.auc.CD.sp.mayo)

time.all <- rep(time.marginal, 4)

auc.ID.all <- c(auc.ID.fp.4cov, auc.ID.sp.4cov, auc.ID.fp.mayo, auc.ID.sp.mayo)
inf.auc.ID.all <- c(inf.auc.ID.fp.4cov, inf.auc.ID.sp.4cov, inf.auc.ID.fp.mayo, inf.auc.ID.sp.mayo)
sup.auc.ID.all <- c(sup.auc.ID.fp.4cov, sup.auc.ID.sp.4cov, sup.auc.ID.fp.mayo, sup.auc.ID.sp.mayo)

STG.all <- c(STG.fp.4cov, STG.sp.4cov, STG.fp.mayo, STG.sp.mayo)
inf.STG.all <- c(inf.STG.fp.4cov, inf.STG.sp.4cov,
                 inf.STG.fp.mayo, inf.STG.sp.mayo)
sup.STG.all <- c(sup.STG.fp.4cov, sup.STG.sp.4cov,
                 sup.STG.fp.mayo, sup.STG.sp.mayo)


plot.AUC.CD <- xyplot(inf.auc.CD.all+ auc.CD.all + sup.auc.CD.all ~ time.all |  appr*marker.is, 
                      type="l", ylim=c(0,1), lty=c(1,1,1), col=c("gray49", 1, "gray49"), 
                      lwd=c(1.5, 1.5, 1.5), xlab="t  (days)", 
                      ylab=expression(paste("AUC"^"C/D", "(t)")),
                      main="Cumulative/Dynamic AUC",
                      par.settings = list(strip.background=list(col="grey98")))      

plot.AUC.ID <- xyplot(inf.auc.ID.all+ auc.ID.all + sup.auc.ID.all ~ time.all |  appr*marker.is, 
                      type="l", ylim=c(0,1), lty=c(1,1,1), col=c("gray49", 1, "gray49"), 
                      lwd=c(1.5, 1.5, 1.5), xlab="t  (days)", 
                      ylab=expression(paste("AUC"^"I/D", "(t)")),
                      main="Incident/Dynamic AUC",
                      par.settings = list(strip.background=list(col="grey98")))      

plot.STG <- xyplot(inf.STG.all+ STG.all + sup.STG.all ~ time.all |  appr*marker.is, 
                   type="l", ylim=c(0,1), lty=c(1,1,1), col=c("gray49", 1, "gray49"), 
                   lwd=c(1.5, 1.5, 1.5), xlab="t  (days)", 
                   ylab=expression(paste("STG", "(t)")),
                   main="Standardized Total Gain",
                   par.settings = list(strip.background=list(col="grey98")))      

#Graphical object to create figure 3
grid.arrange(plot.AUC.CD, plot.AUC.ID, plot.STG, ncol=3)
